In [1]:
import datetime as dt
import os
import io
import re
from copy import deepcopy
from pathlib import Path

import requests
import pandas as pd


# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

# Payload base que você viu no DevTools (os campos Date/FinalDate serão sobrescritos)
PAYLOAD_BASE = {
    "Name": "IPCACoupon",
    "Date": "2025-11-10",
    "FinalDate": "2025-11-10",
    "ClientId": "",
    "Filters": {},
}


def montar_payload(name: str, data: dt.date) -> dict:
    """Monta o JSON no mesmo formato da requisição do navegador."""
    p = deepcopy(PAYLOAD_BASE)
    p["Name"] = name
    d_str = data.strftime("%Y-%m-%d")
    p["Date"] = d_str
    p["FinalDate"] = d_str
    return p


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    """Faz o POST na B3 e devolve o CSV (texto) para aquela 'Name' e data."""
    payload = montar_payload(name, data)
    r = requests.post(URL, headers=HEADERS, json=payload)

    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:500]}")

    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    return r.text


# ==========================
# 2) Helpers de parsing
# ==========================

def ptbr_to_float(s):
    """Converte '98.315,57' -> 98315.57. Devolve None se vazio ou '-'."""
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s == "" or s == "-":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    """
    Acha o cabeçalho 'Vencimento;Contratos em Aberto;...' e retorna
    apenas esse bloco até antes da linha 'F26=...' ou de uma linha em branco.
    """
    lines = csv_text.splitlines()

    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break

    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado no CSV.")

    block_lines = [lines[start]]
    for line in lines[start + 1:]:
        if not line.strip():
            break
        # linha de preços corrigidos tipo 'F26=98.315,57 F27=...'
        if ";" not in line and "=" in line:
            break
        block_lines.append(line)

    return "\n".join(block_lines)


def extrair_precos_corrigidos(csv_text: str) -> dict:
    """
    Acha a linha 'F26=98.315,57 F27=...' e devolve
    { 'F26': 98315.57, 'F27': ... }.
    """
    lines = csv_text.splitlines()
    linha_alvo = None
    for line in lines:
        if "=" in line and ";" not in line:
            linha_alvo = line
            break

    if not linha_alvo:
        return {}

    pares = re.findall(r"([A-Z]\d{2})=([\d\.,]+)", linha_alvo)
    return {cod: ptbr_to_float(valor) for cod, valor in pares}


def extrair_di_over_e_ipca(csv_text: str) -> dict:
    """
    Pega (se existir no texto):
      - Taxa DI Over para DD/MM/AAAA: 0,0551%
      - Valor Índice Ipca pro Rata Tempore: 7.367,7
    """
    di_over = None
    di_over_data = None
    ipca_index = None

    for line in csv_text.splitlines():
        line = line.strip()
        if line.startswith("Taxa DI Over"):
            m = re.search(
                r"Taxa DI Over para (\d{2}/\d{2}/\d{4}):\s*([\d\.,]+)%",
                line
            )
            if m:
                di_over_data = dt.datetime.strptime(m.group(1), "%d/%m/%Y").date()
                di_over = ptbr_to_float(m.group(2))  # em %
        elif line.startswith("Valor Índice Ipca pro Rata Tempore"):
            m = re.search(r":\s*([\d\.,]+)", line)
            if m:
                ipca_index = ptbr_to_float(m.group(1))

    return {
        "di_over_data": di_over_data,
        "di_over_pct": di_over,
        "ipca_index_pro_rata": ipca_index,
    }


def parse_mercado_futuro(csv_text: str,
                          data_referencia: dt.date,
                          name: str) -> tuple[pd.DataFrame, dict]:
    """
    A partir do CSV completo:
      - devolve DataFrame com Mercado Futuro (cada linha = código tipo F26/K27…)
      - e um dict com di_over/ipca_index (se existirem).

    Adiciona colunas:
      - Data_Referencia
      - Name (IPCACoupon, DI1Day, etc.)
      - Preco_Ajuste_Corrigido (se existir linha F26=...)
      - DI_Over_pct / IPCA_Index (repetidas em todas as linhas do dia)
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)
    df = pd.read_csv(
        io.StringIO(bloco),
        sep=";",
        decimal=",",
        thousands=".",
        dtype=str
    )

    # normalizar nomes de colunas (sem espaços, com underscore)
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("ç", "c")
        .str.replace("ã", "a")
        .str.replace("á", "a")
        .str.replace("é", "e")
        .str.replace("í", "i")
        .str.replace("ó", "o")
        .str.replace("ú", "u")
        .str.replace("%", "pct")
    )

    cols_numericas = [
        "Contratos_em_Aberto",
        "Negócios_Realizados",
        "Contratos_Negociados",
        "Volume",
        "Ajuste_Anterior",
        "Preço_de_Abertura",
        "Preço_Mínimo",
        "Preço_Máximo",
        "Preço_Médio",
        "Último_Preço",
        "Ajuste",
        "Variação_em_Pontos",
        "Última_Oferta_de_Compra",
        "Última_Oferta_de_Venda",
    ]
    for c in cols_numericas:
        if c in df.columns:
            df[c] = df[c].apply(ptbr_to_float)

    # infos de contexto
    df["Data_Referencia"] = pd.to_datetime(data_referencia)
    df["Name"] = name  # IPCACoupon, DI1Day, BusinessDollar, etc.

    # preços de ajuste corrigidos (linha F26=…)
    precos_corr = extrair_precos_corrigidos(csv_text)
    if precos_corr:
        df["Preco_Ajuste_Corrigido"] = df["Vencimento"].map(precos_corr)

    meta = extrair_di_over_e_ipca(csv_text)
    # replicar meta por linha (facilita no uso depois)
    df["DI_Over_pct"] = meta.get("di_over_pct")
    df["IPCA_Index_Pro_Rata"] = meta.get("ipca_index_pro_rata")

    return df, meta


# ==========================
# 3) Base em parquet
# ==========================

def carregar_base_parquet(path_parquet: str) -> pd.DataFrame:
    path = Path(path_parquet)
    if path.exists():
        return pd.read_parquet(path)
    else:
        return pd.DataFrame()


def incrementar_base_preco_ajuste(
    path_parquet: str,
    df_novo: pd.DataFrame,
    chaves=("Data_Referencia", "Name", "Vencimento")
) -> pd.DataFrame:
    """
    Lê a base existente, concatena com df_novo e remove duplicatas
    pelas chaves (por padrão: Data_Referencia + Name + Vencimento).
    """
    df_base = carregar_base_parquet(path_parquet)

    if not df_base.empty:
        df_comb = pd.concat([df_base, df_novo], ignore_index=True)
    else:
        df_comb = df_novo.copy()

    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# 4) Exemplo de uso
# ==========================

if __name__ == "__main__":
    # Ativos/tabelas que precisam ser carregados
    NAMES = [
        "IPCACoupon",     # DAP
        "DI1Day",         # DI
        "BusinessDollar",
        "WDOMiniFuture",
        "USTNOTEFuture",
    ]

    # range de datas
    inicio = dt.date(2025, 11, 3)
    fim = dt.date(2025, 11, 12)

    path_parquet = "df_preco_de_ajuste_atual.parquet"

    d = inicio
    while d <= fim:
        df_todos_names = []

        for name in NAMES:
            try:
                csv_text = baixar_csv_bdi(name, d)
                data_referencia = d  # ajuste se preferir d+1 etc.
                df_dia, meta_dia = parse_mercado_futuro(csv_text, data_referencia, name)
                df_todos_names.append(df_dia)

                print(
                    f"{d} / {name}: {len(df_dia)} contratos | "
                    f"meta={meta_dia}"
                )
            except Exception as e:
                print(f"{d} / {name}: erro -> {e}")

        if df_todos_names:
            df_dia_full = pd.concat(df_todos_names, ignore_index=True)
            df_base = incrementar_base_preco_ajuste(path_parquet, df_dia_full)
            print(f"{d}: base total {len(df_base)} linhas")

        d += dt.timedelta(days=1)


2025-11-03 / IPCACoupon: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-03 / DI1Day: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-03 / BusinessDollar: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-03 / WDOMiniFuture: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-03 / USTNOTEFuture: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-04 / IPCACoupon: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-04 / DI1Day: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-04 / BusinessDollar: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-04 / WDOMiniFuture: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-04 / USTNOTEFuture: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-05 / IPCACoupon: erro -> Cabeçalho de Mercado Futuro não encontrado no CSV.
2025-11-05 / DI1Day: erro -> Cabeçalho de Mercado Futuro não enc

In [6]:
dados = pd.read_parquet("df_preco_de_ajuste_atual.parquet")
dados

,Vencimento,Contratos_em_Aberto,NegÃ³cios_Realizados,Contratos_Negociados,Volume,Ajuste_Anterior,PreÃ§o_de_Abertura,PreÃ§o_MÃ­nimo,PreÃ§o_MÃ¡ximo,PreÃ§o_MÃ©dio,Ãltimo_PreÃ§o,Ajuste,VariaÃ§Ã£o_em_Pontos,Ãltima_Oferta_de_Compra,Ãltima_Oferta_de_Venda,Data_Referencia,Preco_Ajuste_Corrigido,Name,DI_Over_pct,IPCA_Index_Pro_Rata
0,F26,61171.0,11,202.0,3.646451e+07,98065.21,"10,51","10,51","10,51","10,51","10,51",98009.420,"-55,79â","10,30","11,03",2025-11-03,98057.05,None,NaN,None
1,F27,8597.0,1,4.0,6.628490e+05,90013.08,"9,29","9,29","9,29","9,29","9,29",90003.010,"-10,07â",-,"9,29",2025-11-03,90046.75,None,NaN,None
2,G26,NaN,-,NaN,NaN,97267.77,-,-,-,-,-,97278.240,"10,47â","9,92","10,36",2025-11-03,97325.52,None,NaN,None
3,H26,NaN,-,NaN,NaN,96781.44,-,-,-,-,-,96791.800,"10,36â","9,34","9,78",2025-11-03,96838.84,None,NaN,None
4,J26,NaN,-,NaN,NaN,96038.87,-,-,-,-,-,96042.710,"3,84â","9,38","9,82",2025-11-03,96089.39,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,X26,NaN,-,NaN,NaN,NaN,-,-,-,-,-,5736.921,"-38,9110â",-,-,2025-11-10,NaN,WDOMiniFuture,NaN,None
789,Z25,890267.0,453.655,2137536.0,1.140360e+11,NaN,"5.340,000","5.308,500","5.347,000","5.334,926","5.314,500",5328.396,"-33,9510â","5.313,500","5.314,500",2025-11-10,NaN,WDOMiniFuture,NaN,None
790,Z26,NaN,-,NaN,NaN,NaN,-,-,-,-,-,5768.780,"-39,3120â",-,-,2025-11-10,NaN,WDOMiniFuture,NaN,None
791,H26,NaN,-,NaN,NaN,NaN,-,-,-,-,-,112.610,"-0,17â",-,-,2025-11-10,NaN,USTNOTEFuture,NaN,None


In [7]:
dados.to_csv("df_preco_de_ajuste_atual.csv", index=False)

In [8]:
import datetime as dt
import os
import io
from copy import deepcopy
from pathlib import Path

import requests
import pandas as pd


# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

# Payload base visto no DevTools (Date/FinalDate serão sobrescritos)
PAYLOAD_BASE = {
    "Name": "IPCACoupon",
    "Date": "2025-11-10",
    "FinalDate": "2025-11-10",
    "ClientId": "",
    "Filters": {},
}


def montar_payload(name: str, data: dt.date) -> dict:
    """Monta o JSON no mesmo formato da requisição do navegador."""
    p = deepcopy(PAYLOAD_BASE)
    p["Name"] = name
    d_str = data.strftime("%Y-%m-%d")
    p["Date"] = d_str
    p["FinalDate"] = d_str
    return p


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    """
    Faz o POST na B3 e devolve o CSV (texto) para aquela 'Name' e data.
    Decodifica como latin-1 para preservar acentos.
    """
    payload = montar_payload(name, data)
    r = requests.post(URL, headers=HEADERS, json=payload)

    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")

    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    # B3 costuma vir em latin-1
    return r.content.decode("latin-1")


# ==========================
# 2) Parsing — só Ajuste
# ==========================

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    """
    Acha o cabeçalho 'Vencimento;Contratos em Aberto;...' e retorna
    apenas esse bloco até antes de linha em branco ou linha de 'F26=...'.
    """
    lines = csv_text.splitlines()

    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break

    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado no CSV.")

    block_lines = [lines[start]]
    for line in lines[start + 1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            # linha de preços corrigidos tipo 'F26=98.315,57 F27=...'
            break
        block_lines.append(line)

    return "\n".join(block_lines)


def ptbr_to_float(s):
    """Converte '98.315,57' -> 98315.57. Devolve None se vazio ou '-'."""
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s == "" or s == "-":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(csv_text: str,
                  data_referencia: dt.date,
                  name: str) -> pd.DataFrame:
    """
    A partir do CSV completo, devolve apenas:
      - Vencimento
      - Ajuste (float)
      - Data_Referencia
      - Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)

    # Lê só o bloco da tabela
    df_raw = pd.read_csv(
        io.StringIO(bloco),
        sep=";",
        decimal=",",
        thousands=".",
        dtype=str,
    )

    # Garante que as colunas tenham os nomes esperados
    # (na exportação vêm como 'Vencimento' e 'Ajuste')
    if "Vencimento" not in df_raw.columns or "Ajuste" not in df_raw.columns:
        raise ValueError(f"Colunas 'Vencimento'/'Ajuste' não encontradas. Colunas = {df_raw.columns.tolist()}")

    df = df_raw[["Vencimento", "Ajuste"]].copy()
    df["Ajuste"] = df["Ajuste"].apply(ptbr_to_float)

    # adiciona contexto
    df["Data_Referencia"] = pd.to_datetime(data_referencia)
    df["Name"] = name

    # opcional: remove linhas sem Ajuste (só vencimento fantasma)
    df = df[df["Ajuste"].notna()].reset_index(drop=True)

    return df


# ==========================
# 3) Base em parquet (só Ajuste)
# ==========================

def carregar_base_parquet(path_parquet: str) -> pd.DataFrame:
    path = Path(path_parquet)
    if path.exists():
        return pd.read_parquet(path)
    else:
        return pd.DataFrame(columns=["Data_Referencia", "Name", "Vencimento", "Ajuste"])


def incrementar_base_ajuste(
    path_parquet: str,
    df_novo: pd.DataFrame,
    chaves=("Data_Referencia", "Name", "Vencimento")
) -> pd.DataFrame:
    """
    Lê a base existente, concatena com df_novo e remove duplicatas
    pelas chaves (Data_Referencia + Name + Vencimento).
    """
    df_base = carregar_base_parquet(path_parquet)

    if not df_base.empty:
        df_comb = pd.concat([df_base, df_novo], ignore_index=True)
    else:
        df_comb = df_novo.copy()

    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# 4) Exemplo de uso
# ==========================

if __name__ == "__main__":
    # Ativos/tabelas que precisam ser carregados
    NAMES = [
        "IPCACoupon",     # DAP
        "DI1Day",         # DI
        "BusinessDollar",
        "WDOMiniFuture",
        "USTNOTEFuture",
    ]

    # range de datas
    inicio = dt.date(2025, 11, 3)
    fim = dt.date(2025, 11, 10)

    path_parquet = "df_ajustes_b3.parquet"

    d = inicio
    while d <= fim:
        df_todos_names = []

        for name in NAMES:
            try:
                csv_text = baixar_csv_bdi(name, d)
                data_referencia = d  # se quiser usar D+1, é só trocar aqui
                df_dia = parse_ajustes(csv_text, data_referencia, name)
                df_todos_names.append(df_dia)

                print(f"{d} / {name}: {len(df_dia)} ajustes carregados")
            except Exception as e:
                print(f"{d} / {name}: erro -> {e}")

        if df_todos_names:
            df_dia_full = pd.concat(df_todos_names, ignore_index=True)
            df_base = incrementar_base_ajuste(path_parquet, df_dia_full)
            print(f"{d}: base total {len(df_base)} linhas\n")

        d += dt.timedelta(days=1)


2025-11-03 / IPCACoupon: 20 ajustes carregados
2025-11-03 / DI1Day: 41 ajustes carregados
2025-11-03 / BusinessDollar: 27 ajustes carregados
2025-11-03 / WDOMiniFuture: 27 ajustes carregados
2025-11-03 / USTNOTEFuture: 2 ajustes carregados
2025-11-03: base total 117 linhas

2025-11-04 / IPCACoupon: 20 ajustes carregados
2025-11-04 / DI1Day: 40 ajustes carregados
2025-11-04 / BusinessDollar: 26 ajustes carregados
2025-11-04 / WDOMiniFuture: 26 ajustes carregados
2025-11-04 / USTNOTEFuture: 2 ajustes carregados
2025-11-04: base total 231 linhas

2025-11-05 / IPCACoupon: 20 ajustes carregados
2025-11-05 / DI1Day: 40 ajustes carregados
2025-11-05 / BusinessDollar: 26 ajustes carregados
2025-11-05 / WDOMiniFuture: 26 ajustes carregados
2025-11-05 / USTNOTEFuture: 2 ajustes carregados
2025-11-05: base total 345 linhas

2025-11-06 / IPCACoupon: 20 ajustes carregados
2025-11-06 / DI1Day: 40 ajustes carregados
2025-11-06 / BusinessDollar: 26 ajustes carregados
2025-11-06 / WDOMiniFuture: 26 aju

In [10]:
import datetime as dt
import io
from copy import deepcopy
from pathlib import Path

import requests
import pandas as pd


# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

# Payload base visto no DevTools (Date/FinalDate serão sobrescritos)
PAYLOAD_BASE = {
    "Name": "IPCACoupon",
    "Date": "2025-11-10",
    "FinalDate": "2025-11-10",
    "ClientId": "",
    "Filters": {},
}


def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    p["Name"] = name
    d_str = data.strftime("%Y-%m-%d")
    p["Date"] = d_str
    p["FinalDate"] = d_str
    return p


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    """
    Faz o POST na B3 e devolve o CSV (texto) para aquela 'Name' e data.
    Decodifica como latin-1 para preservar acentos.
    """
    payload = montar_payload(name, data)
    r = requests.post(URL, headers=HEADERS, json=payload)

    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")

    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    return r.content.decode("latin-1")


# ==========================
# 2) Parsing — só Ajuste
# ==========================

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    """
    Acha o cabeçalho 'Vencimento;Contratos em Aberto;...' e retorna
    apenas esse bloco até antes de linha em branco ou linha de 'F26=...'.
    """
    lines = csv_text.splitlines()

    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break

    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado no CSV.")

    block_lines = [lines[start]]
    for line in lines[start + 1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            # linha de preços corrigidos tipo 'F26=98.315,57 F27=...'
            break
        block_lines.append(line)

    return "\n".join(block_lines)


def ptbr_to_float(s):
    """Converte '98.315,57' -> 98315.57. Devolve None se vazio ou '-'."""
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s == "" or s == "-":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(csv_text: str,
                  data_referencia: dt.date,
                  name: str) -> pd.DataFrame:
    """
    A partir do CSV completo, devolve apenas:
      - Vencimento
      - Ajuste (float)
      - Data_Referencia
      - Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)

    df_raw = pd.read_csv(
        io.StringIO(bloco),
        sep=";",
        decimal=",",
        thousands=".",
        dtype=str,
    )

    if "Vencimento" not in df_raw.columns or "Ajuste" not in df_raw.columns:
        raise ValueError(
            f"Colunas 'Vencimento'/'Ajuste' não encontradas. "
            f"Colunas = {df_raw.columns.tolist()}"
        )

    df = df_raw[["Vencimento", "Ajuste"]].copy()
    df["Ajuste"] = df["Ajuste"].apply(ptbr_to_float)

    df["Data_Referencia"] = pd.to_datetime(data_referencia)
    df["Name"] = name

    df = df[df["Ajuste"].notna()].reset_index(drop=True)
    return df


# ==========================
# 3) Base longa em parquet
# ==========================

def carregar_base_parquet(path_parquet: str) -> pd.DataFrame:
    path = Path(path_parquet)
    if path.exists():
        return pd.read_parquet(path)
    else:
        return pd.DataFrame(columns=["Vencimento", "Ajuste", "Data_Referencia", "Name"])


def incrementar_base_ajuste(
    path_parquet: str,
    df_novo: pd.DataFrame,
    chaves=("Data_Referencia", "Name", "Vencimento")
) -> pd.DataFrame:
    df_base = carregar_base_parquet(path_parquet)

    if not df_base.empty:
        df_comb = pd.concat([df_base, df_novo], ignore_index=True)
    else:
        df_comb = df_novo.copy()

    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# 4) Exportar matriz (wide)
# ==========================

def exportar_matriz_ajuste(
    df_base: pd.DataFrame,
    name: str,
    path_parquet_out: str,
):
    """
    Gera um arquivo em formato “matriz”:
      - índice: Vencimento
      - colunas: Data_Referencia
      - valores: Ajuste

    Filtra apenas o Name desejado (ex: IPCACoupon, DI1Day).
    """
    df_name = df_base[df_base["Name"] == name].copy()
    if df_name.empty:
        print(f"Nenhum dado para Name={name}, não gerou matriz.")
        return

    # garante datetime
    df_name["Data_Referencia"] = pd.to_datetime(df_name["Data_Referencia"])

    matriz = df_name.pivot(
        index="Vencimento",
        columns="Data_Referencia",
        values="Ajuste",
    )

    # ordenar vencimentos e datas
    matriz = matriz.sort_index()
    matriz = matriz.reindex(sorted(matriz.columns), axis=1)

    # se quiser datas como string no cabeçalho:
    matriz.columns = [c.strftime("%Y-%m-%d") for c in matriz.columns]

    matriz.to_parquet(path_parquet_out)
    print(f"Matriz de ajustes ({name}) salva em: {path_parquet_out}")


# ==========================
# 5) Exemplo de uso
# ==========================

if __name__ == "__main__":
    # Ativos/tabelas que precisam ser carregados
    NAMES = [
        "IPCACoupon",     # DAP
        "DI1Day",         # DI
        "BusinessDollar",
        "WDOMiniFuture",
        "USTNOTEFuture",
    ]

    # range de datas
    inicio = dt.date(2025, 11, 3)
    fim = dt.date(2025, 11, 10)

    path_long = "df_ajustes_b3.parquet"

    d = inicio
    while d <= fim:
        df_todos_names = []

        for name in NAMES:
            try:
                csv_text = baixar_csv_bdi(name, d)
                data_referencia = d  # se quiser D+1, troque aqui
                df_dia = parse_ajustes(csv_text, data_referencia, name)
                df_todos_names.append(df_dia)

                print(f"{d} / {name}: {len(df_dia)} ajustes carregados")
            except Exception as e:
                print(f"{d} / {name}: erro -> {e}")

        if df_todos_names:
            df_dia_full = pd.concat(df_todos_names, ignore_index=True)
            df_base = incrementar_base_ajuste(path_long, df_dia_full)
            print(f"{d}: base longa total {len(df_base)} linhas\n")

        d += dt.timedelta(days=1)

    # Depois de atualizar a base longa, gera os arquivos em formato matriz
    df_base_final = carregar_base_parquet(path_long)

    # Exemplo: matriz só de IPCACoupon, igual o arquivo que você anexou
    exportar_matriz_ajuste(
        df_base_final,
        name="IPCACoupon",
        path_parquet_out="df_preco_de_ajuste_atual_completo.parquet",
    )

    # Se quiser também uma matriz análoga para DI1Day:
    exportar_matriz_ajuste(
        df_base_final,
        name="DI1Day",
        path_parquet_out="df_preco_de_ajuste_di1day_completo.parquet",
    )


2025-11-03 / IPCACoupon: 20 ajustes carregados
2025-11-03 / DI1Day: 41 ajustes carregados
2025-11-03 / BusinessDollar: 27 ajustes carregados
2025-11-03 / WDOMiniFuture: 27 ajustes carregados
2025-11-03 / USTNOTEFuture: 2 ajustes carregados
2025-11-03: base longa total 687 linhas

2025-11-04 / IPCACoupon: 20 ajustes carregados
2025-11-04 / DI1Day: 40 ajustes carregados
2025-11-04 / BusinessDollar: 26 ajustes carregados
2025-11-04 / WDOMiniFuture: 26 ajustes carregados
2025-11-04 / USTNOTEFuture: 2 ajustes carregados
2025-11-04: base longa total 687 linhas

2025-11-05 / IPCACoupon: 20 ajustes carregados
2025-11-05 / DI1Day: 40 ajustes carregados
2025-11-05 / BusinessDollar: 26 ajustes carregados
2025-11-05 / WDOMiniFuture: 26 ajustes carregados
2025-11-05 / USTNOTEFuture: 2 ajustes carregados
2025-11-05: base longa total 687 linhas

2025-11-06 / IPCACoupon: 20 ajustes carregados
2025-11-06 / DI1Day: 40 ajustes carregados
2025-11-06 / BusinessDollar: 26 ajustes carregados
2025-11-06 / WDO

In [15]:
import datetime as dt
import io
from copy import deepcopy
from pathlib import Path

import requests
import pandas as pd


# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

# Payload base visto no DevTools (Date/FinalDate serão sobrescritos)
PAYLOAD_BASE = {
    "Name": "IPCACoupon",
    "Date": "2025-11-10",
    "FinalDate": "2025-11-10",
    "ClientId": "",
    "Filters": {},
}


def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    p["Name"] = name
    d_str = data.strftime("%Y-%m-%d")
    p["Date"] = d_str
    p["FinalDate"] = d_str
    return p


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    """
    Faz o POST na B3 e devolve o CSV (texto) para aquela 'Name' e data.
    Decodifica como latin-1 para preservar acentos.
    """
    payload = montar_payload(name, data)
    r = requests.post(URL, headers=HEADERS, json=payload)

    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")

    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    # B3 geralmente vem em latin-1
    return r.content.decode("latin-1")


# ==========================
# 2) Parsing — só Ajuste
# ==========================

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    """
    Acha o cabeçalho 'Vencimento;Contratos em Aberto;...' e retorna
    apenas esse bloco até antes de linha em branco ou linha de 'F26=...'.
    """
    lines = csv_text.splitlines()

    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break

    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado no CSV.")

    block_lines = [lines[start]]
    for line in lines[start + 1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            # linha de preços corrigidos tipo 'F26=98.315,57 F27=...'
            break
        block_lines.append(line)

    return "\n".join(block_lines)


def ptbr_to_float(s):
    """Converte '98.315,57' -> 98315.57. Devolve None se vazio ou '-'."""
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s == "" or s == "-":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(
    csv_text: str,
    data_referencia: dt.date,
    name: str
) -> pd.DataFrame:
    """
    A partir do CSV completo, devolve apenas:
      - Vencimento
      - Ajuste (float)
      - Data_Referencia
      - Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)

    df_raw = pd.read_csv(
        io.StringIO(bloco),
        sep=";",
        decimal=",",
        thousands=".",
        dtype=str,
    )

    if "Vencimento" not in df_raw.columns or "Ajuste" not in df_raw.columns:
        raise ValueError(
            f"Colunas 'Vencimento'/'Ajuste' não encontradas. "
            f"Colunas = {df_raw.columns.tolist()}"
        )

    df = df_raw[["Vencimento", "Ajuste"]].copy()
    df["Ajuste"] = df["Ajuste"].apply(ptbr_to_float)

    df["Data_Referencia"] = pd.to_datetime(data_referencia)
    df["Name"] = name

    df = df[df["Ajuste"].notna()].reset_index(drop=True)
    return df


# ==========================
# 3) Base longa em parquet
# ==========================

def carregar_base_parquet(path_parquet: str) -> pd.DataFrame:
    path = Path(path_parquet)
    if path.exists():
        return pd.read_parquet(path)
    else:
        return pd.DataFrame(columns=["Vencimento", "Ajuste", "Data_Referencia", "Name"])


def incrementar_base_ajuste(
    path_parquet: str,
    df_novo: pd.DataFrame,
    chaves=("Data_Referencia", "Name", "Vencimento")
) -> pd.DataFrame:
    df_base = carregar_base_parquet(path_parquet)

    if not df_base.empty:
        df_comb = pd.concat([df_base, df_novo], ignore_index=True)
    else:
        df_comb = df_novo.copy()

    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# 4) Construir matriz com DI + DAP + WDO + TREASURY
#    usando a mesma lógica do código antigo
# ==========================

def construir_matriz_assets(
    df_base: pd.DataFrame,
    path_parquet_out: str,
):
    """
    Constrói matriz final:
      - índice: Assets (DAPxx, DI_xx, TREASURY, WDO1, etc.)
      - colunas: datas (YYYY-MM-DD)
      - valores: Ajuste
    Reaplica a lógica:
      - IPCACoupon -> 'DAP' + últimos 2 dígitos do Vencimento
      - DI1Day     -> 'DI_' + últimos 2 dígitos do Vencimento
      - BusinessDollar / WDOMiniFuture -> 'WDO1'
      - USTNOTEFuture -> 'TREASURY'
    """
    df = df_base.copy()
    df["Data_Referencia"] = pd.to_datetime(df["Data_Referencia"])
    df["Vencimento"] = df["Vencimento"].astype(str).str.strip()

    # últimos 2 dígitos do código de vencimento (F26 -> "26", K35 -> "35", etc.)
    last2 = df["Vencimento"].str[-2:]

    # mapeia Name + Vencimento para Assets (mesma regra do normalizar antigo)
    assets = []

    for name, venc, suf in zip(df["Name"], df["Vencimento"], last2):
        if name == "IPCACoupon":        # DAP - Cupom de DI x IPCA
            assets.append(f"DAP{suf}")
        elif name == "DI1Day":          # DI1 - DI de 1 dia
            assets.append(f"DI_{suf}")
        elif name in ("BusinessDollar", "WDOMiniFuture"):
            assets.append("WDO1")
        elif name == "USTNOTEFuture":
            assets.append("TREASURY")
        else:
            # se aparecer algum Name desconhecido, você decide:
            # aqui vou descartar configurando None
            assets.append(None)

    df["Assets"] = assets
    df = df[df["Assets"].notna()].copy()

    # agora pivot: índice=Assets, colunas=Data_Referencia, valores=Ajuste
    matriz = df.pivot_table(
        index="Assets",
        columns="Data_Referencia",
        values="Ajuste",
        aggfunc="first",     # se tiver mais de uma linha por asset/data, pega a primeira
    )

    # ordena colunas por data
    matriz = matriz.reindex(sorted(matriz.columns), axis=1)

    # datas como string no cabeçalho
    matriz.columns = [c.strftime("%Y-%m-%d") for c in matriz.columns]

    # opcional: reindexar na ordem desejada de ativos
    ordem_ativos = [
        "DAP25", "DAP26", "DAP27", "DAP28", "DAP29", "DAP30", "DAP32",
        "DAP33", "DAP35", "DAP40", "DAP45", "DAP50", "DAP55", "DAP60",
        "DI_26", "DI_27", "DI_28", "DI_29", "DI_30", "DI_31", "DI_32",
        "DI_33", "DI_34", "DI_35", "DI_36", "DI_37", "DI_38", "DI_39",
        "DI_40",
        "TREASURY",
        "WDO1",
        # Se em algum momento você trouxer NTNB de outra fonte,
        # é só garantir que os índices existam:
        "NTNB26", "NTNB27", "NTNB28", "NTNB30", "NTNB32",
        "NTNB35", "NTNB40", "NTNB45", "NTNB50", "NTNB55", "NTNB60",
    ]
    matriz = matriz.reindex(ordem_ativos)

    # salva em parquet no formato wide
    matriz.to_parquet(path_parquet_out)
    print(f"Matriz final (Assets x Datas) salva em: {path_parquet_out}")
    print("Shape:", matriz.shape)


# ==========================
# 5) Exemplo de uso
# ==========================

if __name__ == "__main__":
    # Ativos/tabelas que precisam ser carregados da B3
    NAMES = [
        "IPCACoupon",     # DAP
        "DI1Day",         # DI
        "BusinessDollar",
        "WDOMiniFuture",
        "USTNOTEFuture",
    ]

    # range de datas para buscar
    inicio = dt.date(2025, 11, 3)
    fim = dt.date(2025, 11, 10)

    path_long = "df_ajustes_b3.parquet"

    d = inicio
    while d <= fim:
        df_todos_names = []

        for name in NAMES:
            try:
                csv_text = baixar_csv_bdi(name, d)
                data_referencia = d  # se quiser usar D+1, troque aqui
                df_dia = parse_ajustes(csv_text, data_referencia, name)
                df_todos_names.append(df_dia)

                print(f"{d} / {name}: {len(df_dia)} ajustes carregados")
            except Exception as e:
                print(f"{d} / {name}: erro -> {e}")

        if df_todos_names:
            df_dia_full = pd.concat(df_todos_names, ignore_index=True)
            df_base = incrementar_base_ajuste(path_long, df_dia_full)
            print(f"{d}: base longa total {len(df_base)} linhas\n")

        d += dt.timedelta(days=1)

    # Depois de atualizar a base longa, gera a matriz DI + DAP + WDO + TREASURY
    df_base_final = carregar_base_parquet(path_long)

    construir_matriz_assets(
        df_base_final,
        path_parquet_out="df_preco_de_ajuste_atual.parquet",
    )


2025-11-03 / IPCACoupon: 20 ajustes carregados
2025-11-03 / DI1Day: 41 ajustes carregados
2025-11-03 / BusinessDollar: 27 ajustes carregados
2025-11-03 / WDOMiniFuture: 27 ajustes carregados
2025-11-03 / USTNOTEFuture: 2 ajustes carregados
2025-11-03: base longa total 687 linhas

2025-11-04 / IPCACoupon: 20 ajustes carregados
2025-11-04 / DI1Day: 40 ajustes carregados
2025-11-04 / BusinessDollar: 26 ajustes carregados
2025-11-04 / WDOMiniFuture: 26 ajustes carregados
2025-11-04 / USTNOTEFuture: 2 ajustes carregados
2025-11-04: base longa total 687 linhas

2025-11-05 / IPCACoupon: 20 ajustes carregados
2025-11-05 / DI1Day: 40 ajustes carregados
2025-11-05 / BusinessDollar: 26 ajustes carregados
2025-11-05 / WDOMiniFuture: 26 ajustes carregados
2025-11-05 / USTNOTEFuture: 2 ajustes carregados
2025-11-05: base longa total 687 linhas

2025-11-06 / IPCACoupon: 20 ajustes carregados
2025-11-06 / DI1Day: 40 ajustes carregados
2025-11-06 / BusinessDollar: 26 ajustes carregados
2025-11-06 / WDO

In [16]:
dados = pd.read_parquet("df_preco_de_ajuste_atual.parquet")
dados

,2025-11-03,2025-11-04,2025-11-05,2025-11-06,2025-11-07,2025-11-10
Assets,,,,,,
DAP25,99495.450,99554.810,99602.000,99650.89,99708.900,99757.990
DAP26,98009.420,98060.210,98121.230,98168.63,98220.720,98267.810
DAP27,90003.010,90066.880,90090.730,90081.71,90154.060,90238.740
DAP28,80469.970,80495.100,80458.770,80463.54,80590.980,80749.090
DAP29,76445.570,76407.190,76344.370,76404.75,76588.250,76734.890
DAP30,69826.150,69755.270,69761.040,69858.58,70032.760,70176.460
DAP32,60455.180,60340.990,60358.960,60527.84,60716.030,60885.670
DAP33,57414.480,57371.620,57368.640,57544.96,57781.790,57878.840
DAP35,50251.200,50221.530,50258.130,50605.10,50730.920,50879.330


In [ ]:
dados = pd.read_parquet("df_preco_de_ajuste_atual_completo.parquet")
dados

,2025-11-03,2025-11-04,2025-11-05,2025-11-06,2025-11-07,2025-11-10
Vencimento,,,,,,
F26,98009.42,98060.21,98121.23,98168.63,98220.72,98267.81
F27,90003.01,90066.88,90090.73,90081.71,90154.06,90238.74
G26,97278.24,97298.11,97345.47,97383.37,97439.42,97495.91
H26,96791.80,96820.63,96871.23,96905.92,96968.43,97027.38
J26,96042.71,96073.82,96123.99,96153.97,96231.04,96294.46
K27,88040.19,88112.37,88135.62,88086.02,88194.38,88296.45
K29,76445.57,76407.19,76344.37,76404.75,76588.25,76734.89
K33,57414.48,57371.62,57368.64,57544.96,57781.79,57878.84
K35,50251.20,50221.53,50258.13,50605.10,50730.92,50879.33


In [ ]:
dados = pd.read_parquet("DashRisco/Dados/df_preco_de_ajuste_atual_completo.parquet")
print(dados['Assets'])

0        DAP25
1        DAP26
2        DAP27
3        DAP28
4        DAP29
5        DAP30
6        DAP32
7        DAP33
8        DAP35
9        DAP40
10       DAP45
11       DAP50
12       DAP55
13       DAP60
14       DI_26
15       DI_27
16       DI_28
17       DI_29
18       DI_30
19       DI_31
20       DI_32
21       DI_33
22       DI_34
23       DI_35
24       DI_36
25       DI_37
26       DI_38
27       DI_39
28       DI_40
29    TREASURY
30        WDO1
0       NTNB26
1       NTNB27
2       NTNB28
3       NTNB30
4       NTNB32
5       NTNB35
6       NTNB40
7       NTNB45
8       NTNB50
9       NTNB55
10      NTNB60
Name: Assets, dtype: object


In [17]:
import requests
cod = '36482'
url = f'https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO=\'{cod}\')'
resp = requests.get(url)
data = resp.json()
# Agora data contém o objeto JSON com valores da série


In [18]:
data

{'@odata.context': 'http://www.ipeadata.gov.br/api/odata4/$metadata#Collection(Ipeadata.OData4.Models.Valor)',
 'value': []}

In [ ]:
import requests

codigo = "PRECOS12_IPCA12"
url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{codigo}')"

resp = requests.get(url)
data = resp.json()
serie = [
    (item["VALDATA"][:10], item["VALVALOR"])
    for item in data["value"]
]

# ordena pela data (string AAAA-MM-DD já ordena certo)
serie.sort(key=lambda x: x[0])

for data_str, valor in serie:
    print(data_str, valor)
    
data


1979-12-01 7.6183e-09
1980-01-01 8.1223e-09
1980-02-01 8.4973e-09
1980-03-01 9.0104e-09
1980-04-01 9.4867e-09
1980-05-01 1.00277e-08
1980-06-01 1.05597e-08
1980-07-01 1.11453e-08
1980-08-01 1.16965e-08
1980-09-01 1.21913e-08
1980-10-01 1.33471e-08
1980-11-01 1.42378e-08
1980-12-01 1.51795e-08
1981-01-01 1.62174e-08
1981-02-01 1.72555e-08
1981-03-01 1.81134e-08
1981-04-01 1.92839e-08
1981-05-01 2.0356e-08
1981-06-01 2.14792e-08
1981-07-01 2.28229e-08
1981-08-01 2.40772e-08
1981-09-01 2.53429e-08
1981-10-01 2.66297e-08
1981-11-01 2.80335e-08
1981-12-01 2.96946e-08
1982-01-01 3.17628e-08
1982-02-01 3.38708e-08
1982-03-01 3.58057e-08
1982-04-01 3.79137e-08
1982-05-01 4.04382e-08
1982-06-01 4.33113e-08
1982-07-01 4.6065e-08
1982-08-01 4.88164e-08
1982-09-01 5.12964e-08
1982-10-01 5.35752e-08
1982-11-01 5.64085e-08
1982-12-01 6.08117e-08
1983-01-01 6.60665e-08
1983-02-01 7.12581e-08
1983-03-01 7.64919e-08
1983-04-01 8.15245e-08
1983-05-01 8.68097e-08
1983-06-01 9.53868e-08
1983-07-01 1.05000

In [20]:
import requests
import pandas as pd

codigo = "PRECOS12_IPCA12"
url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{codigo}')"

resp = requests.get(url)
resp.raise_for_status()  # opcional, mas bom pra ver erro de HTTP
data = resp.json()

# A série está dentro da chave "value"
registros = data["value"]

# Monta um DataFrame
df = pd.DataFrame(registros)

# Converte a data para datetime e deixa só data (sem hora/fuso)
df["VALDATA"] = pd.to_datetime(df["VALDATA"]).dt.date

# Mantém só data e valor, e ordena
df = df[["VALDATA", "VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)

print(df)


C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_4184\545469039.py:18: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["VALDATA"] = pd.to_datetime(df["VALDATA"]).dt.date


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
import requests
import pandas as pd

codigo = "PRECOS12_IPCA12"
url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{codigo}')"

resp = requests.get(url)
resp.raise_for_status()
data = resp.json()

registros = data["value"]

df = pd.DataFrame(registros)

# 1) Garante que é string e pega só AAAA-MM-DD
df["VALDATA"] = df["VALDATA"].astype(str).str[:10]

# 2) Converte para datetime
df["VALDATA"] = pd.to_datetime(df["VALDATA"], errors="coerce")

# 3) Mantém só as colunas relevantes e ordena
df = (
    df[["VALDATA", "VALVALOR"]]
    .sort_values("VALDATA")
    .reset_index(drop=True)
)

# 4) Renomeia para algo mais legível
df = df.rename(columns={
    "VALDATA": "Data",
    "VALVALOR": "IPCA_Indice"
})

# (Opcional) deixar a data como índice
# df = df.set_index("Data")
# IPCA INDICE
print(df.head())
print(df.dtypes)

IPCA_PREVISTO = 0.15 # PROJEÇÃO IPCA DO MÊS
Valor_Ponto = 100000
Reais_por_ponto = 0.00025
#today = data de refencia do dia scrapado 
# preciso multiplicar o ajuste capturado pelo código pela váriavel Valor Ponto que foi construida com os dados de IPCA, projeção e etc


        Data   IPCA_Indice
0 1979-12-01  7.618300e-09
1 1980-01-01  8.122300e-09
2 1980-02-01  8.497300e-09
3 1980-03-01  9.010400e-09
4 1980-04-01  9.486700e-09
Data           datetime64[ns]
IPCA_Indice           float64
dtype: object


In [29]:
import datetime as dt
import io
import re
from copy import deepcopy
from pathlib import Path

import requests
import pandas as pd


# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO = "PRECOS12_IPCA12"
IPCA_PREVISTO = 0.15       # Projeção ANBIMA do mês (em %)
VALOR_PONTOS_CONTRATO = 100_000
REAIS_POR_PONTO = 0.00025  # R$ PT


# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

PAYLOAD_BASE = {
    "Name": "IPCACoupon",
    "Date": "2025-11-10",
    "FinalDate": "2025-11-10",
    "ClientId": "",
    "Filters": {},
}


def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    p["Name"] = name
    d_str = data.strftime("%Y-%m-%d")
    p["Date"] = d_str
    p["FinalDate"] = d_str
    return p


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    """
    Faz o POST na B3 e devolve o CSV (texto).
    Tenta decodificar em UTF-8 e cai para latin-1 se der erro.
    """
    payload = montar_payload(name, data)
    r = requests.post(URL, headers=HEADERS, json=payload)

    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")

    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    try:
        return r.content.decode("utf-8")
    except UnicodeDecodeError:
        return r.content.decode("latin-1")


# ==========================
# 2) Parsing — usando Variação em Pontos
# ==========================

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()

    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break

    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado no CSV.")

    block_lines = [lines[start]]
    for line in lines[start + 1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block_lines.append(line)

    return "\n".join(block_lines)


def ptbr_to_float(s):
    """
    Converte strings do tipo '98.315,57', '-55,79↓', '64,82↑' -> 98315.57 / -55.79 / 64.82.
    Remove qualquer caractere que não seja dígito, vírgula, ponto ou sinal de menos.
    """
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s == "" or s == "-":
        return None

    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(
    csv_text: str,
    data_referencia: dt.date,
    name: str
) -> pd.DataFrame:
    """
    A partir do CSV completo, devolve apenas:
      - Vencimento
      - Ajuste (float)  [vem da coluna 'Variação em Pontos']
      - Data_Referencia
      - Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)

    df_raw = pd.read_csv(
        io.StringIO(bloco),
        sep=";",
        decimal=",",
        thousands=".",
        dtype=str,
    )

    # achar o nome real da coluna de variação em pontos (com ou sem mojibake)
    col_var = None
    for c in df_raw.columns:
        # normaliza tudo pra minúsculo e sem acento óbvio
        c_norm = c.lower()
        if "varia" in c_norm and "ponto" in c_norm:
            col_var = c
            break

    if "Vencimento" not in df_raw.columns or col_var is None:
        raise ValueError(
            f"Colunas 'Vencimento'/'Variação em Pontos' não encontradas. "
            f"Colunas = {df_raw.columns.tolist()}"
        )

    # usa Vencimento + coluna de Variação em Pontos encontrada
    df = df_raw[["Vencimento", col_var]].copy()
    df["Ajuste"] = df[col_var].apply(ptbr_to_float)

    df["Data_Referencia"] = pd.to_datetime(data_referencia)
    df["Name"] = name

    df = df[df["Ajuste"].notna()].reset_index(drop=True)
    return df


# ==========================
# 3) IPCA – baixar da IPEA e helpers
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    codigo = IPCA_SERIE_CODIGO
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{codigo}')"

    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()["value"]

    df = pd.DataFrame(data)
    df["VALDATA"] = df["VALDATA"].astype(str).str[:10]
    df["VALDATA"] = pd.to_datetime(df["VALDATA"], errors="coerce")

    df = (
        df[["VALDATA", "VALVALOR"]]
        .sort_values("VALDATA")
        .reset_index(drop=True)
        .rename(columns={"VALDATA": "Data", "VALVALOR": "IPCA_Indice"})
    )

    return df


def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA disponível até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])


def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:  # 5=sábado, 6=domingo
        d += dt.timedelta(days=1)
    return d


def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev_month = data_ref.month
        prev_year = data_ref.year
        if data_ref.month == 12:
            next_month = 1
            next_year = data_ref.year + 1
        else:
            next_month = data_ref.month + 1
            next_year = data_ref.year
    else:
        if data_ref.month == 1:
            prev_month = 12
            prev_year = data_ref.year - 1
        else:
            prev_month = data_ref.month - 1
            prev_year = data_ref.year
        next_month = data_ref.month
        next_year = data_ref.year

    prev = dt.date(prev_year, prev_month, 15)
    nxt = dt.date(next_year, next_month, 15)
    return prev, nxt


def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float = IPCA_PREVISTO,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    data_ref = pd.to_datetime(data_ref).date()

    prev_15, next_15 = datas_ipca_referencia(data_ref)

    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)

    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")

    ni_ref = obter_ipca_ref(df_ipca, prev_adj)

    ipca_pro_rata = ni_ref * (1 + ipca_previsto / 100) ** (du_desde / du_entre)

    valor_ponto = ipca_pro_rata * reais_por_ponto

    return valor_ponto


# ==========================
# 4) Base longa em parquet
# ==========================

def carregar_base_parquet(path_parquet: str) -> pd.DataFrame:
    path = Path(path_parquet)
    if path.exists():
        return pd.read_parquet(path)
    else:
        return pd.DataFrame(columns=["Vencimento", "Ajuste", "Data_Referencia", "Name"])


def incrementar_base_ajuste(
    path_parquet: str,
    df_novo: pd.DataFrame,
    chaves=("Data_Referencia", "Name", "Vencimento")
) -> pd.DataFrame:
    df_base = carregar_base_parquet(path_parquet)

    if not df_base.empty:
        df_comb = pd.concat([df_base, df_novo], ignore_index=True)
    else:
        df_comb = df_novo.copy()

    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# 5) Construir matriz com DI + DAP + WDO + TREASURY
# ==========================

def construir_matriz_assets(
    df_base: pd.DataFrame,
    path_parquet_out: str,
):
    df = df_base.copy()
    df["Data_Referencia"] = pd.to_datetime(df["Data_Referencia"])
    df["Vencimento"] = df["Vencimento"].astype(str).str.strip()

    last2 = df["Vencimento"].str[-2:]

    assets = []
    for name, venc, suf in zip(df["Name"], df["Vencimento"], last2):
        if name == "IPCACoupon":
            assets.append(f"DAP{suf}")
        elif name == "DI1Day":
            assets.append(f"DI_{suf}")
        elif name in ("BusinessDollar", "WDOMiniFuture"):
            assets.append("WDO1")
        elif name == "USTNOTEFuture":
            assets.append("TREASURY")
        else:
            assets.append(None)

    df["Assets"] = assets
    df = df[df["Assets"].notna()].copy()

    matriz = df.pivot_table(
        index="Assets",
        columns="Data_Referencia",
        values="Ajuste",
        aggfunc="first",
    )

    matriz = matriz.reindex(sorted(matriz.columns), axis=1)
    matriz.columns = [c.strftime("%Y-%m-%d") for c in matriz.columns]

    ordem_ativos = [
        "DAP25", "DAP26", "DAP27", "DAP28", "DAP29", "DAP30", "DAP32",
        "DAP33", "DAP35", "DAP40", "DAP45", "DAP50", "DAP55", "DAP60",
        "DI_26", "DI_27", "DI_28", "DI_29", "DI_30", "DI_31", "DI_32",
        "DI_33", "DI_34", "DI_35", "DI_36", "DI_37", "DI_38", "DI_39",
        "DI_40",
        "TREASURY",
        "WDO1",
        "NTNB26", "NTNB27", "NTNB28", "NTNB30", "NTNB32",
        "NTNB35", "NTNB40", "NTNB45", "NTNB50", "NTNB55", "NTNB60",
    ]
    matriz = matriz.reindex(ordem_ativos)

    matriz.to_parquet(path_parquet_out)
    print(f"Matriz final (Assets x Datas) salva em: {path_parquet_out}")
    print("Shape:", matriz.shape)


# ==========================
# 6) Exemplo de uso
# ==========================

if __name__ == "__main__":
    NAMES = [
        "IPCACoupon",
        "DI1Day",
        "BusinessDollar",
        "WDOMiniFuture",
        "USTNOTEFuture",
    ]

    inicio = dt.date(2025, 11, 3)
    fim = dt.date(2025, 11, 10)

    path_long = "df_ajustes_b3.parquet"

    df_ipca = carregar_ipca_ipeadata()

    d = inicio
    while d <= fim:
        df_todos_names = []

        for name in NAMES:
            try:
                csv_text = baixar_csv_bdi(name, d)
                data_referencia = d
                df_dia = parse_ajustes(csv_text, data_referencia, name)

                # DAP: variação em pontos * valor do ponto (R$) calculado via IPCA
                if name == "IPCACoupon":
                    valor_ponto = calcular_valor_ponto_dap_para_data(
                        df_ipca=df_ipca,
                        data_ref=data_referencia,
                        ipca_previsto=IPCA_PREVISTO,
                        reais_por_ponto=REAIS_POR_PONTO,
                    )
                    df_dia["Ajuste"] = df_dia["Ajuste"] * valor_ponto

                df_todos_names.append(df_dia)

                print(f"{d} / {name}: {len(df_dia)} ajustes carregados")
            except Exception as e:
                print(f"{d} / {name}: erro -> {e}")

        if df_todos_names:
            df_dia_full = pd.concat(df_todos_names, ignore_index=True)
            df_base = incrementar_base_ajuste(path_long, df_dia_full)
            print(f"{d}: base longa total {len(df_base)} linhas\n")

        d += dt.timedelta(days=1)

    df_base_final = carregar_base_parquet(path_long)

    construir_matriz_assets(
        df_base_final,
        path_parquet_out="df_preco_de_ajuste_atual.parquet",
    )


2025-11-03 / IPCACoupon: 20 ajustes carregados
2025-11-03 / DI1Day: 41 ajustes carregados
2025-11-03 / BusinessDollar: 27 ajustes carregados
2025-11-03 / WDOMiniFuture: 27 ajustes carregados
2025-11-03 / USTNOTEFuture: 2 ajustes carregados
2025-11-03: base longa total 687 linhas

2025-11-04 / IPCACoupon: 20 ajustes carregados
2025-11-04 / DI1Day: 40 ajustes carregados
2025-11-04 / BusinessDollar: 26 ajustes carregados
2025-11-04 / WDOMiniFuture: 26 ajustes carregados
2025-11-04 / USTNOTEFuture: 2 ajustes carregados
2025-11-04: base longa total 687 linhas

2025-11-05 / IPCACoupon: 20 ajustes carregados
2025-11-05 / DI1Day: 40 ajustes carregados
2025-11-05 / BusinessDollar: 26 ajustes carregados
2025-11-05 / WDOMiniFuture: 26 ajustes carregados
2025-11-05 / USTNOTEFuture: 2 ajustes carregados
2025-11-05: base longa total 687 linhas

2025-11-06 / IPCACoupon: 20 ajustes carregados
2025-11-06 / DI1Day: 40 ajustes carregados
2025-11-06 / BusinessDollar: 26 ajustes carregados
2025-11-06 / WDO

In [ ]:
import datetime as dt
import io
import re
from copy import deepcopy
from pathlib import Path

import requests
import pandas as pd


# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO = "PRECOS12_IPCA12"
IPCA_PREVISTO = 0.15       # Projeção ANBIMA do mês (em %)
VALOR_PONTOS_CONTRATO = 100_000
REAIS_POR_PONTO = 0.00025  # R$ PT


# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

PAYLOAD_BASE = {
    "Name": "IPCACoupon",
    "Date": "2025-11-10",
    "FinalDate": "2025-11-10",
    "ClientId": "",
    "Filters": {},
}


def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    p["Name"] = name
    d_str = data.strftime("%Y-%m-%d")
    p["Date"] = d_str
    p["FinalDate"] = d_str
    return p


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    """
    Faz o POST na B3 e devolve o CSV (texto).
    Tenta decodificar em UTF-8 e cai para latin-1 se der erro.
    """
    payload = montar_payload(name, data)
    r = requests.post(URL, headers=HEADERS, json=payload)

    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")

    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    try:
        return r.content.decode("utf-8")
    except UnicodeDecodeError:
        return r.content.decode("latin-1")


# ==========================
# 2) Parsing — usando Variação em Pontos
# ==========================

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()

    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break

    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado no CSV.")

    block_lines = [lines[start]]
    for line in lines[start + 1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block_lines.append(line)

    return "\n".join(block_lines)


def ptbr_to_float(s):
    """
    Converte strings do tipo '98.315,57', '-55,79↓', '64,82↑' -> 98315.57 / -55.79 / 64.82.
    Remove qualquer caractere que não seja dígito, vírgula, ponto ou sinal de menos.
    """
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s == "" or s == "-":
        return None

    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(
    csv_text: str,
    data_referencia: dt.date,
    name: str
) -> pd.DataFrame:
    """
    A partir do CSV completo, devolve apenas:
      - Vencimento
      - Ajuste (float)  [vem da coluna 'Variação em Pontos']
      - Data_Referencia
      - Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)

    df_raw = pd.read_csv(
        io.StringIO(bloco),
        sep=";",
        decimal=",",
        thousands=".",
        dtype=str,
    )

    # achar o nome real da coluna de variação em pontos (com ou sem mojibake)
    col_var = None
    for c in df_raw.columns:
        # normaliza tudo pra minúsculo e sem acento óbvio
        c_norm = c.lower()
        if "varia" in c_norm and "ponto" in c_norm:
            col_var = c
            break

    if "Vencimento" not in df_raw.columns or col_var is None:
        raise ValueError(
            f"Colunas 'Vencimento'/'Variação em Pontos' não encontradas. "
            f"Colunas = {df_raw.columns.tolist()}"
        )

    # usa Vencimento + coluna de Variação em Pontos encontrada
    df = df_raw[["Vencimento", col_var]].copy()
    df["Ajuste"] = df[col_var].apply(ptbr_to_float)

    df["Data_Referencia"] = pd.to_datetime(data_referencia)
    df["Name"] = name

    df = df[df["Ajuste"].notna()].reset_index(drop=True)
    return df


# ==========================
# 3) IPCA – baixar da IPEA e helpers
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    codigo = IPCA_SERIE_CODIGO
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{codigo}')"

    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()["value"]

    df = pd.DataFrame(data)
    df["VALDATA"] = df["VALDATA"].astype(str).str[:10]
    df["VALDATA"] = pd.to_datetime(df["VALDATA"], errors="coerce")

    df = (
        df[["VALDATA", "VALVALOR"]]
        .sort_values("VALDATA")
        .reset_index(drop=True)
        .rename(columns={"VALDATA": "Data", "VALVALOR": "IPCA_Indice"})
    )

    return df


def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA disponível até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])


def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:  # 5=sábado, 6=domingo
        d += dt.timedelta(days=1)
    return d


def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev_month = data_ref.month
        prev_year = data_ref.year
        if data_ref.month == 12:
            next_month = 1
            next_year = data_ref.year + 1
        else:
            next_month = data_ref.month + 1
            next_year = data_ref.year
    else:
        if data_ref.month == 1:
            prev_month = 12
            prev_year = data_ref.year - 1
        else:
            prev_month = data_ref.month - 1
            prev_year = data_ref.year
        next_month = data_ref.month
        next_year = data_ref.year

    prev = dt.date(prev_year, prev_month, 15)
    nxt = dt.date(next_year, next_month, 15)
    return prev, nxt


def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float = IPCA_PREVISTO,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    data_ref = pd.to_datetime(data_ref).date()

    prev_15, next_15 = datas_ipca_referencia(data_ref)

    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)

    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")

    ni_ref = obter_ipca_ref(df_ipca, prev_adj)

    ipca_pro_rata = ni_ref * (1 + ipca_previsto / 100) ** (du_desde / du_entre)

    valor_ponto = ipca_pro_rata * reais_por_ponto

    return valor_ponto


# ==========================
# 4) Base longa em parquet
# ==========================

def carregar_base_parquet(path_parquet: str) -> pd.DataFrame:
    path = Path(path_parquet)
    if path.exists():
        return pd.read_parquet(path)
    else:
        return pd.DataFrame(columns=["Vencimento", "Ajuste", "Data_Referencia", "Name"])


def incrementar_base_ajuste(
    path_parquet: str,
    df_novo: pd.DataFrame,
    chaves=("Data_Referencia", "Name", "Vencimento")
) -> pd.DataFrame:
    df_base = carregar_base_parquet(path_parquet)

    if not df_base.empty:
        df_comb = pd.concat([df_base, df_novo], ignore_index=True)
    else:
        df_comb = df_novo.copy()

    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# 5) Construir matriz com DI + DAP + WDO + TREASURY
# ==========================

def construir_matriz_assets(
    df_base: pd.DataFrame,
    path_parquet_out: str,
):
    df = df_base.copy()
    df["Data_Referencia"] = pd.to_datetime(df["Data_Referencia"])
    df["Vencimento"] = df["Vencimento"].astype(str).str.strip()

    last2 = df["Vencimento"].str[-2:]

    assets = []
    for name, venc, suf in zip(df["Name"], df["Vencimento"], last2):
        if name == "IPCACoupon":
            assets.append(f"DAP{suf}")
        elif name == "DI1Day":
            assets.append(f"DI_{suf}")
        elif name in ("BusinessDollar", "WDOMiniFuture"):
            assets.append("WDO1")
        elif name == "USTNOTEFuture":
            assets.append("TREASURY")
        else:
            assets.append(None)

    df["Assets"] = assets
    df = df[df["Assets"].notna()].copy()

    matriz = df.pivot_table(
        index="Assets",
        columns="Data_Referencia",
        values="Ajuste",
        aggfunc="first",
    )

    matriz = matriz.reindex(sorted(matriz.columns), axis=1)
    matriz.columns = [c.strftime("%Y-%m-%d") for c in matriz.columns]

    ordem_ativos = [
        "DAP25", "DAP26", "DAP27", "DAP28", "DAP29", "DAP30", "DAP32",
        "DAP33", "DAP35", "DAP40", "DAP45", "DAP50", "DAP55", "DAP60",
        "DI_26", "DI_27", "DI_28", "DI_29", "DI_30", "DI_31", "DI_32",
        "DI_33", "DI_34", "DI_35", "DI_36", "DI_37", "DI_38", "DI_39",
        "DI_40",
        "TREASURY",
        "WDO1",
        "NTNB26", "NTNB27", "NTNB28", "NTNB30", "NTNB32",
        "NTNB35", "NTNB40", "NTNB45", "NTNB50", "NTNB55", "NTNB60",
    ]
    matriz = matriz.reindex(ordem_ativos)

    matriz.to_parquet(path_parquet_out)
    print(f"Matriz final (Assets x Datas) salva em: {path_parquet_out}")
    print("Shape:", matriz.shape)


# ==========================
# 6) Exemplo de uso
# ==========================

NAMES = [
    "IPCACoupon",
    "DI1Day",
    "BusinessDollar",
    "WDOMiniFuture",
    "USTNOTEFuture",
]

inicio = dt.date(2025, 11, 3)
fim = dt.date(2025, 11, 10)

path_long = "df_ajustes_b3.parquet"

df_ipca = carregar_ipca_ipeadata()

d = inicio
while d <= fim:
    df_todos_names = []

    for name in NAMES:
        try:
            csv_text = baixar_csv_bdi(name, d)
            data_referencia = d
            df_dia = parse_ajustes(csv_text, data_referencia, name)

            # DAP: variação em pontos * valor do ponto (R$) calculado via IPCA
            if name == "IPCACoupon":
                valor_ponto = calcular_valor_ponto_dap_para_data(
                    df_ipca=df_ipca,
                    data_ref=data_referencia,
                    ipca_previsto=IPCA_PREVISTO,
                    reais_por_ponto=REAIS_POR_PONTO,
                )
                df_dia["Ajuste"] = df_dia["Ajuste"] * valor_ponto

            df_todos_names.append(df_dia)

            print(f"{d} / {name}: {len(df_dia)} ajustes carregados")
        except Exception as e:
            print(f"{d} / {name}: erro -> {e}")

    if df_todos_names:
        df_dia_full = pd.concat(df_todos_names, ignore_index=True)
        df_base = incrementar_base_ajuste(path_long, df_dia_full)
        print(f"{d}: base longa total {len(df_base)} linhas\n")

    d += dt.timedelta(days=1)

df_base_final = carregar_base_parquet(path_long)

construir_matriz_assets(
    df_base_final,
    path_parquet_out="df_preco_de_ajuste_atual.parquet",
)


Data Ref: 2025-11-03 | NI Ref: 7359.0600 | IPCA Pro Rata: 7365.2972 | Valor Ponto: 1.8413
1.841324292494741
2025-11-03 / IPCACoupon: 20 ajustes carregados
2025-11-03 / DI1Day: 41 ajustes carregados
2025-11-03 / BusinessDollar: 27 ajustes carregados
2025-11-03 / WDOMiniFuture: 27 ajustes carregados
2025-11-03 / USTNOTEFuture: 2 ajustes carregados
2025-11-03: base longa total 687 linhas

Data Ref: 2025-11-04 | NI Ref: 7359.0600 | IPCA Pro Rata: 7365.7772 | Valor Ponto: 1.8414
1.8414442927969104
2025-11-04 / IPCACoupon: 20 ajustes carregados
2025-11-04 / DI1Day: 40 ajustes carregados
2025-11-04 / BusinessDollar: 26 ajustes carregados
2025-11-04 / WDOMiniFuture: 26 ajustes carregados
2025-11-04 / USTNOTEFuture: 2 ajustes carregados
2025-11-04: base longa total 687 linhas

Data Ref: 2025-11-05 | NI Ref: 7359.0600 | IPCA Pro Rata: 7366.2572 | Valor Ponto: 1.8416
1.8415643009195777
2025-11-05 / IPCACoupon: 20 ajustes carregados
2025-11-05 / DI1Day: 40 ajustes carregados
2025-11-05 / BusinessD

In [ ]:
# -*- coding: utf-8 -*-
import datetime as dt
import io, re, os, json
from copy import deepcopy
from pathlib import Path

import pandas as pd
import requests
import pandas_market_calendars as mcal


# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO   = "PRECOS12_IPCA12"
IPCA_PREVISTO       = 0.15        # % ANBIMA do mês
REAIS_POR_PONTO     = 0.00025     # R$ PT (valor do ponto)
BACKOFF_LIM         = 5           # qtde máx. de dias ÚTEIS B3 que voltamos ao tentar preencher um dia vazio

# arquivos
PATH_LONG   = "df_ajustes_b3.parquet"
PATH_WIDE   = "df_preco_de_ajuste_atual_completo.parquet"
PATH_JSON   = "df_preco_de_ajuste_atual_completo.json"  # NOVO: JSON pt-BR


# ============================================================
# A) OPÇÃO B — Normalizador p/ JSON com strings pt-BR (helper)
#    + salvaguarda: se HOJE existir e estiver vazio, não mexe
# ============================================================

import re
from datetime import datetime
try:
    from zoneinfo import ZoneInfo  # Python 3.9+
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None  # fallback sem timezone

_PTBR_NUM_RE = re.compile(r':\s*"(\d{1,3}(?:\.\d{3})*,\d+)"')
_OBJ_GLUE_RE = re.compile(r'}\s*{\s*')  # "}{"

def _fmt_ptbr(num: float) -> str:
    """Formata float em string pt-BR '12.345,67' com 2 casas."""
    s = f"{num:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

def _prepare_as_array(text: str) -> str:
    """Se houver múltiplos objetos colados '}{', envolve em [ ... ] e separa por vírgulas."""
    t = text.strip()
    if t.startswith("{") and _OBJ_GLUE_RE.search(t):
        return "[" + _OBJ_GLUE_RE.sub("},{", t) + "]"
    return t

def _has_today_all_blank(text: str, today_iso: str) -> bool:
    """
    True se **todas** as ocorrências de hoje são vazias: "YYYY-MM-DD":"" (ou só espaços).
    Se houver ao menos uma com número / string não vazia => False.
    Se não houver a chave hoje => False (processa normalmente).
    """
    pattern = re.compile(rf'"{re.escape(today_iso)}"\s*:\s*"(.*?)"')
    matches = pattern.findall(text)
    if not matches:
        pattern_num = re.compile(rf'"{re.escape(today_iso)}"\s*:\s*[-+]?\d+(?:\.\d+)?')
        if pattern_num.search(text):
            return False
        return False
    return all((m.strip() == "") for m in matches)

def fix_to_ptbr_string_json_with_today_guard(text: str, *, today: str | None = None) -> str:
    """
    Opção B (strings pt-BR) + salvaguarda do dia de hoje.
    - Mantém tudo como string "12.345,67".
    - Corrige objetos colados.
    - Se hoje existir e estiver **vazio** em todas as ocorrências, devolve texto original.
    """
    if today is None:
        if _TZ is not None:
            today_iso = datetime.now(_TZ).date().isoformat()
        else:
            today_iso = datetime.now().date().isoformat()
    else:
        today_iso = today

    raw = text

    # 0) Guard: hoje duplicado mas sem dados => não altera nada
    if _has_today_all_blank(raw, today_iso):
        return raw

    # 1) Envolver em array se houver "}{"
    t = _prepare_as_array(raw)

    # 2) Converter strings pt-BR "12.345,67" -> número temporário para validar JSON
    t_num = _PTBR_NUM_RE.sub(lambda m: f":{float(m.group(1).replace('.','').replace(',','.'))}", t)

    # 3) Parse seguro
    try:
        data = json.loads(t_num)
    except json.JSONDecodeError as e:
        raise ValueError(
            f"JSON inválido após normalização. Posição {e.pos}: {e.msg}. "
            "Verifique vírgulas, aspas e chaves."
        ) from e

    # 4) Voltar tudo p/ string pt-BR
    def walk(v):
        if isinstance(v, dict):
            return {k: walk(x) for k, x in v.items()}
        if isinstance(v, list):
            return [walk(x) for x in v]
        if isinstance(v, (int, float)):
            return _fmt_ptbr(float(v))
        return v

    data = walk(data)

    # 5) Dump final
    return json.dumps(data, ensure_ascii=False)


# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}
PAYLOAD_BASE = {"Name": "IPCACoupon","Date":"2025-11-10","FinalDate":"2025-11-10","ClientId":"","Filters":{}}

NAMES = ["IPCACoupon","DI1Day","BusinessDollar","WDOMiniFuture","USTNOTEFuture"]


def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Name"], p["Date"], p["FinalDate"] = name, s, s
    return p


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    r = requests.post(URL, headers=HEADERS, json=montar_payload(name, data))
    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")
    try:
        return r.content.decode("utf-8")
    except UnicodeDecodeError:
        return r.content.decode("latin-1")


# ==========================
# 2) Parsing — bloco e variação em pontos
# ==========================

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()
    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break
    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado.")
    block = [lines[start]]
    for line in lines[start+1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block.append(line)
    return "\n".join(block)


def ptbr_to_float(s):
    # '98.315,57' | '-55,79↓' | '64,82↑' → float
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s in {"", "-"}:
        return None
    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(csv_text: str, data_ref: dt.date, name: str) -> pd.DataFrame:
    bloco = extrair_bloco_mercado_futuro(csv_text)
    df_raw = pd.read_csv(io.StringIO(bloco), sep=";", decimal=",", thousands=".", dtype=str)

    # detecta a coluna de "Variação em Pontos" mesmo com mojibake
    col_var = None
    for c in df_raw.columns:
        cn = c.lower()
        if "varia" in cn and "ponto" in cn:
            col_var = c
            break
    if "Vencimento" not in df_raw.columns or col_var is None:
        raise ValueError(f"Colunas 'Vencimento'/'Variação em Pontos' não encontradas. Colunas = {df_raw.columns.tolist()}")

    df = df_raw[["Vencimento", col_var]].copy()
    df["Ajuste"] = df[col_var].apply(ptbr_to_float)
    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df = df[df["Ajuste"].notna()].reset_index(drop=True)
    return df


# ==========================
# 3) IPCA helpers (IPEA)
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{IPCA_SERIE_CODIGO}')"
    resp = requests.get(url); resp.raise_for_status()
    df = pd.DataFrame(resp.json()["value"])
    df["VALDATA"] = pd.to_datetime(df["VALDATA"].astype(str).str[:10], errors="coerce")
    df = df[["VALDATA","VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)
    return df.rename(columns={"VALDATA":"Data","VALVALOR":"IPCA_Indice"})


def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])


def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:  # sábado/domingo
        d += dt.timedelta(days=1)
    return d


def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev = dt.date(data_ref.year, data_ref.month, 15)
        nxt_m = 1 if data_ref.month == 12 else data_ref.month + 1
        nxt_y = data_ref.year + 1 if data_ref.month == 12 else data_ref.year
        nxt = dt.date(nxt_y, nxt_m, 15)
    else:
        pm = 12 if data_ref.month == 1 else data_ref.month - 1
        py = data_ref.year - 1 if data_ref.month == 1 else data_ref.year
        prev = dt.date(py, pm, 15)
        nxt  = dt.date(data_ref.year, data_ref.month, 15)
    return prev, nxt


def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float = IPCA_PREVISTO,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    data_ref = pd.to_datetime(data_ref).date()
    prev_15, next_15 = datas_ipca_referencia(data_ref)
    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)
    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")
    ni_ref = obter_ipca_ref(df_ipca, prev_adj)
    ipca_pro_rata = ni_ref * (1 + ipca_previsto/100) ** (du_desde/du_entre)
    return ipca_pro_rata * reais_por_ponto


# ==========================
# 4) Base longa (append dedup)
# ==========================

def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        return pd.read_parquet(p)
    return pd.DataFrame(columns=["Vencimento","Ajuste","Data_Referencia","Name"])


def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                            chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# 5) Wide: leitura, escrita e utilidades
# ==========================

def ler_wide_assets(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")
    df = pd.read_parquet(path)
    # garantir formato: primeira coluna é índice "Assets"
    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"
    # garantir ordenação de colunas por data
    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass
    # remover índices duplicados
    df = df[~df.index.duplicated(keep="last")]
    return df


def salvar_wide_assets(df: pd.DataFrame, path: str):
    df = df.copy()
    df.index.name = "Assets"
    out = df.reset_index().rename(columns={df.reset_index().columns[0]:"Assets"})
    out.to_parquet(path, index=False)


def _fmt_ptbr_2dec(x) -> str:
    """Formata número (ou NaN) para string pt-BR com 2 casas; NaN -> ''."""
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return ""
    try:
        v = float(x)
    except Exception:
        return str(x)
    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")


def wide_to_ptbr_json_text(wide_df: pd.DataFrame, today_iso: str | None = None) -> str:
    """
    Constrói um JSON (lista de objetos) com:
      {"Assets":"DAP60","YYYY-MM-DD":"12.345,67", ...}
    Todas as células como **string** pt-BR.
    """
    if today_iso is None:
        today_iso = (dt.date.today()).strftime("%Y-%m-%d")

    # Garante que colunas estejam como YYYY-MM-DD
    cols = []
    for c in wide_df.columns:
        try:
            cols.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols.append(c)
    wide = wide_df.copy()
    wide.columns = cols

    # Monta lista de dicts
    records = []
    for asset, row in wide.iterrows():
        d = {"Assets": str(asset)}
        for c in wide.columns:
            d[c] = _fmt_ptbr_2dec(row[c])
        records.append(d)

    # JSON final (já correto). NÃO usamos o guard aqui porque geramos conteúdo válido.
    # Se você estiver normalizando um texto vindo de fora, use fix_to_ptbr_string_json_with_today_guard.
    return json.dumps(records, ensure_ascii=False)


def mapear_asset(name: str, venc: str) -> str | None:
    suf = (venc or "").strip()[-2:]
    if name == "IPCACoupon":     return f"DAP{suf}"
    if name == "DI1Day":         return f"DI_{suf}"
    if name in ("BusinessDollar","WDOMiniFuture"): return "WDO1"
    if name == "USTNOTEFuture":  return "TREASURY"
    return None


def construir_coluna_wide(df_long_dia: pd.DataFrame, data_ref: dt.date) -> pd.Series:
    df = df_long_dia.copy()
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()]
    s = df.groupby("Asset")["Ajuste"].first()
    s.name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    return s


# ==========================
# 6) Calendário B3 + busca com backoff
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")

def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]

def ultimo_dia_util_antes_hoje() -> dt.date:
    today = dt.date.today()  # tz local
    val = b3_calendar().valid_days(today - dt.timedelta(days=15), today)
    return val[-1].date()

def buscar_um_dia(name: str, d: dt.date) -> pd.DataFrame | None:
    """Tenta baixar e parsear um name/data; devolve df (pode estar vazio)."""
    csv_text = baixar_csv_bdi(name, d)
    return parse_ajustes(csv_text, d, name)

def buscar_dia_com_backoff(target_d: dt.date, df_ipca: pd.DataFrame) -> pd.DataFrame:
    """
    Busca todos os NAMES para target_d. Se algum vier vazio/erro,
    volta dias ÚTEIS até BACKOFF_LIM e usa o último disponível,
    carimbando a Data_Referencia como target_d (para manter continuidade).
    """
    dfs = []
    validos_back = b3_valid_days(target_d - dt.timedelta(days=40), target_d)[::-1]  # dias úteis anteriores
    for name in NAMES:
        ok = False
        tentativa = 0
        for prev_d in [target_d] + validos_back:
            try:
                df_n = buscar_um_dia(name, prev_d)
                if df_n is not None and not df_n.empty:
                    # DAP: converter pontos→R$ no momento da INCORPORAÇÃO
                    if name == "IPCACoupon":
                        valor_ponto = calcular_valor_ponto_dap_para_data(df_ipca, target_d)
                        df_n["Ajuste"] = df_n["Ajuste"] * valor_ponto
                    # carimbar a Data_Referencia como o TARGET (não o prev_d)
                    df_n["Data_Referencia"] = pd.to_datetime(target_d)
                    dfs.append(df_n)
                    ok = True
                    if prev_d != target_d:
                        print(f"  • {name}: {target_d} vazio → usando {prev_d} (backfill)")
                    break
            except Exception:
                pass
            tentativa += 1
            if tentativa > BACKOFF_LIM:
                break
        if not ok:
            print(f"  ! {name}: sem dados até {BACKOFF_LIM} dias úteis atrás para {target_d}")
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame(columns=["Vencimento","Ajuste","Data_Referencia","Name"])


# ==========================
# 7) Pipeline principal (incremental + duplicação) + JSON pt-BR
# ==========================

def main():
    # 1) carregar IPCA e base wide existente
    df_ipca = carregar_ipca_ipeadata()
    wide = ler_wide_assets(PATH_WIDE)     # índice=Assets, colunas=YYYY-MM-DD (strings)

    # 2) definir range de datas a atualizar (do último col registrado até último útil antes de hoje)
    if wide.empty or len(wide.columns) == 0:
        start_dt = dt.date(2025, 1, 2)  # seed inicial se arquivo não existe
    else:
        try:
            cols_dt = sorted(pd.to_datetime(wide.columns))
            start_dt = cols_dt[-1].date()   # último dia já gravado
        except Exception:
            start_dt = dt.date(2025, 1, 2)

    end_dt = ultimo_dia_util_antes_hoje()
    dias_util = b3_valid_days(start_dt, end_dt)

    print(f"Atualizando de {start_dt} até {end_dt} (dias úteis B3: {len(dias_util)})")

    # 3) iterar dias úteis e preencher
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref, df_ipca)
        if df_dia_all.empty:
            print(f"{dref}: nenhum dado disponível (pula)")
            continue

        # atualizar base longa
        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)

        # montar coluna wide do dia
        s_col = construir_coluna_wide(df_dia_all, dref)

        # unir na wide existente
        if wide.empty:
            wide = pd.DataFrame(s_col)
        else:
            # garante que todos assets existam
            wide = wide.reindex(wide.index.union(s_col.index))
            col_name = s_col.name
            wide[col_name] = s_col

        print(f"{dref}: atualizada coluna {s_col.name} | base longa = {len(df_long)} linhas")

    # 4) lógica de duplicação (duas últimas colunas)
    if wide.shape[1] >= 2:
        cols_dt = sorted(pd.to_datetime(wide.columns))
        prev_col = cols_dt[-2].strftime("%Y-%m-%d")
        last_col = cols_dt[-1].strftime("%Y-%m-%d")

        iguais_preco = wide[prev_col].equals(wide[last_col])

        if iguais_preco:
            print("Última coluna já é cópia idêntica — não duplica de novo.")
        else:
            # duplica para o próximo dia útil após a última coluna real
            prox = b3_valid_days(cols_dt[-1].date(), cols_dt[-1].date() + dt.timedelta(days=10))
            if len(prox) >= 2:
                prox_dt = prox[1]  # o primeiro é o próprio last; o segundo é o próximo útil
                prox_col = prox_dt.strftime("%Y-%m-%d")
                if prox_col not in wide.columns:
                    wide[prox_col] = wide[last_col]
                    print(f"Coluna duplicada para o próximo dia útil: {prox_col}")
            else:
                print("Não foi possível encontrar próximo dia útil para duplicação.")
    else:
        print("Matriz wide tem menos de 2 colunas — sem duplicação.")

    # 5) ordena colunas por data e salva Parquet
    try:
        cols_dt = sorted(pd.to_datetime(wide.columns))
        wide = wide[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass

    salvar_wide_assets(wide, PATH_WIDE)
    print(f"Salvo: {PATH_WIDE} | shape {wide.shape}")

    # 6) Gera JSON pt-BR (todos valores como strings "12.345,67")
    json_text = wide_to_ptbr_json_text(wide)
    # Opcional: se você estiver alimentando este texto com um “dump” externo e quiser aplicar a salvaguarda:
    # json_text = fix_to_ptbr_string_json_with_today_guard(json_text)
    with open(PATH_JSON, "w", encoding="utf-8") as f:
        f.write(json_text)
    print(f"Salvo JSON pt-BR: {PATH_JSON} (strings com 2 casas decimais)")


if __name__ == "__main__":
    main()


Atualizando de 2025-11-12 até 2025-11-12 (dias úteis B3: 1)
  • IPCACoupon: 2025-11-12 vazio → usando 2025-11-11 (backfill)
  • DI1Day: 2025-11-12 vazio → usando 2025-11-11 (backfill)
  • BusinessDollar: 2025-11-12 vazio → usando 2025-11-11 (backfill)
  • WDOMiniFuture: 2025-11-12 vazio → usando 2025-11-11 (backfill)
  • USTNOTEFuture: 2025-11-12 vazio → usando 2025-11-11 (backfill)
2025-11-12: atualizada coluna 2025-11-12 | base longa = 801 linhas
Coluna duplicada para o próximo dia útil: 2025-11-13
Salvo: df_preco_de_ajuste_atual_completo.parquet | shape (43, 220)
Salvo JSON pt-BR: df_preco_de_ajuste_atual_completo.json (strings com 2 casas decimais)


In [3]:
# -*- coding: utf-8 -*-
import datetime as dt
import io, re, os, json
from copy import deepcopy
from pathlib import Path

import pandas as pd
import requests
import pandas_market_calendars as mcal

# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO   = "PRECOS12_IPCA12"
IPCA_PREVISTO       = 0.15        # % ANBIMA do mês
REAIS_POR_PONTO     = 0.00025     # R$ PT (valor do ponto) — aplicado no DAP
BACKOFF_LIM         = 5           # qtde máx. de dias ÚTEIS B3 que voltamos ao tentar preencher um dia vazio

# arquivos (bases)
PATH_LONG    = "df_ajustes_b3.parquet"  # base longa (depuração)
PATH_PRECO   = "df_preco_de_ajuste_atual_completo.parquet"  # PREÇOS (wide)
PATH_VALOR   = "df_valor_ajuste_contrato.parquet"           # VALOR AJUSTE (DAP em R$; demais = pontos até definirmos multiplicadores)
PATH_JSON    = "df_preco_de_ajuste_atual_completo.json"     # JSON pt-BR dos PREÇOS (opcional)

# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}
PAYLOAD_BASE = {"Name": "IPCACoupon","Date":"2025-11-10","FinalDate":"2025-11-10","ClientId":"","Filters":{}}

# Mercadorias/nomes para buscar
NAMES = ["IPCACoupon","DI1Day","BusinessDollar","WDOMiniFuture","USTNOTEFuture"]

# ==========================
# Helpers de JSON pt-BR (para export opcional)
# ==========================

try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None

_PTBR_NUM_RE = re.compile(r':\s*"(\d{1,3}(?:\.\d{3})*,\d+)"')
_OBJ_GLUE_RE = re.compile(r'}\s*{\s*')

def _fmt_ptbr_2dec(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return ""
    try:
        v = float(x)
    except Exception:
        return str(x)
    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    cols = []
    for c in wide_df.columns:
        try:
            cols.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols.append(c)
    wide = wide_df.copy()
    wide.columns = cols
    records = []
    for asset, row in wide.iterrows():
        d = {"Assets": str(asset)}
        for c in wide.columns:
            d[c] = _fmt_ptbr_2dec(row[c])
        records.append(d)
    return json.dumps(records, ensure_ascii=False)

# ==========================
# 2) Parsing — pontos e preço
# ==========================

def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Name"], p["Date"], p["FinalDate"] = name, s, s
    return p

def baixar_csv_bdi(name: str, data: dt.date) -> str:
    r = requests.post(URL, headers=HEADERS, json=montar_payload(name, data))
    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")
    try:
        return r.content.decode("utf-8")
    except UnicodeDecodeError:
        return r.content.decode("latin-1")

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()
    start = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Vencimento;Contratos em Aberto"):
            start = i
            break
    if start is None:
        raise ValueError("Cabeçalho de Mercado Futuro não encontrado.")
    block = [lines[start]]
    for line in lines[start+1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block.append(line)
    return "\n".join(block)

def ptbr_to_float(s):
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s in {"", "-"}:
        return None
    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def parse_ajustes(csv_text: str, data_ref: dt.date, name: str) -> pd.DataFrame:
    """
    Retorna colunas padronizadas:
      Vencimento | Pontos (variação) | PrecoAjusteAtual | Data_Referencia | Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)
    df_raw = pd.read_csv(io.StringIO(bloco), sep=";", decimal=",", thousands=".", dtype=str)

    # detectar colunas-alvo com tolerância a mojibake
    col_var = None
    col_preco = None
    for c in df_raw.columns:
        cn = c.lower()
        if col_var is None and ("varia" in cn and "ponto" in cn):
            col_var = c
        if col_preco is None and ("preço" in cn and "atual" in cn):
            col_preco = c
    if "Vencimento" not in df_raw.columns or col_var is None or col_preco is None:
        raise ValueError(f"Colunas não encontradas. Tenho: {df_raw.columns.tolist()}")

    df = df_raw[["Vencimento", col_var, col_preco]].copy()
    df.rename(columns={col_var: "Pontos", col_preco: "PrecoAjusteAtual"}, inplace=True)
    df["Pontos"] = df["Pontos"].apply(ptbr_to_float)  # variação em pontos
    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].apply(ptbr_to_float)
    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df = df[df["Pontos"].notna() | df["PrecoAjusteAtual"].notna()].reset_index(drop=True)
    return df

# ==========================
# 3) IPCA helpers (IPEA)
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{IPCA_SERIE_CODIGO}')"
    resp = requests.get(url); resp.raise_for_status()
    df = pd.DataFrame(resp.json()["value"])
    df["VALDATA"] = pd.to_datetime(df["VALDATA"].astype(str).str[:10], errors="coerce")
    df = df[["VALDATA","VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)
    return df.rename(columns={"VALDATA":"Data","VALVALOR":"IPCA_Indice"})

def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])

def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:
        d += dt.timedelta(days=1)
    return d

def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev = dt.date(data_ref.year, data_ref.month, 15)
        nxt_m = 1 if data_ref.month == 12 else data_ref.month + 1
        nxt_y = data_ref.year + 1 if data_ref.month == 12 else data_ref.year
        nxt = dt.date(nxt_y, nxt_m, 15)
    else:
        pm = 12 if data_ref.month == 1 else data_ref.month - 1
        py = data_ref.year - 1 if data_ref.month == 1 else data_ref.year
        prev = dt.date(py, pm, 15)
        nxt  = dt.date(data_ref.year, data_ref.month, 15)
    return prev, nxt

def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float = IPCA_PREVISTO,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    data_ref = pd.to_datetime(data_ref).date()
    prev_15, next_15 = datas_ipca_referencia(data_ref)
    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)
    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")
    ni_ref = obter_ipca_ref(df_ipca, prev_adj)
    ipca_pro_rata = ni_ref * (1 + ipca_previsto/100) ** (du_desde/du_entre)
    return ipca_pro_rata * reais_por_ponto

# ==========================
# 4) Base longa (append dedup)
# ==========================

def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        return pd.read_parquet(p)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name"])

def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                            chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb

# ==========================
# 5) Wides (preço e valor) — leitura/escrita
# ==========================

def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")
    df = pd.read_parquet(path)
    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"
    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass
    df = df[~df.index.duplicated(keep="last")]
    return df

def salvar_wide(df: pd.DataFrame, path: str):
    df2 = df.copy()
    df2.index.name = "Assets"
    out = df2.reset_index().rename(columns={df2.reset_index().columns[0]:"Assets"})
    out.to_parquet(path, index=False)

def mapear_asset(name: str, venc: str) -> str | None:
    suf = (venc or "").strip()[-2:]
    if name == "IPCACoupon":     return f"DAP{suf}"
    if name == "DI1Day":         return f"DI_{suf}"
    if name in ("BusinessDollar","WDOMiniFuture"): return "WDO1"
    if name == "USTNOTEFuture":  return "TREASURY"
    return None

def construir_colunas_wide_duplas(df_long_dia: pd.DataFrame, data_ref: dt.date, df_ipca: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    """
    Retorna:
      s_preco -> série por Asset com PrecoAjusteAtual
      s_valor -> série por Asset com Valor de Ajuste (DAP em R$; demais = pontos)
    """
    df = df_long_dia.copy()
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()]

    # preço (pega o primeiro por asset)
    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()

    # valor do ajuste: DAP em R$, demais ficam em pontos até definirmos multiplicadores
    s_valor = df.groupby("Asset")["Pontos"].first()  # começa em pontos
    if not df[df["Name"] == "IPCACoupon"].empty:
        valor_ponto_dap = calcular_valor_ponto_dap_para_data(df_ipca, data_ref)
        # converte apenas DAPxx
        for k in s_valor.index:
            if str(k).startswith("DAP") and pd.notna(s_valor.loc[k]):
                s_valor.loc[k] = float(s_valor.loc[k]) * valor_ponto_dap

    # nome da coluna
    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name

    return s_preco, s_valor

# ==========================
# 6) Calendário B3 + backoff
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")

def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]

def ultimo_dia_util_antes_hoje() -> dt.date:
    today = dt.date.today()
    val = b3_calendar().valid_days(today - dt.timedelta(days=15), today)
    return val[-1].date()

def buscar_um_dia(name: str, d: dt.date) -> pd.DataFrame | None:
    csv_text = baixar_csv_bdi(name, d)
    return parse_ajustes(csv_text, d, name)

def buscar_dia_com_backoff(target_d: dt.date, df_ipca: pd.DataFrame) -> pd.DataFrame:
    dfs = []
    validos_back = b3_valid_days(target_d - dt.timedelta(days=40), target_d)[::-1]
    for name in NAMES:
        ok = False
        tentativa = 0
        for prev_d in [target_d] + validos_back:
            try:
                df_n = buscar_um_dia(name, prev_d)
                if df_n is not None and not df_n.empty:
                    # carimbar Data_Referencia como TARGET
                    df_n["Data_Referencia"] = pd.to_datetime(target_d)
                    dfs.append(df_n)
                    ok = True
                    if prev_d != target_d:
                        print(f"  • {name}: {target_d} vazio → usando {prev_d} (backfill)")
                    break
            except Exception:
                pass
            tentativa += 1
            if tentativa > BACKOFF_LIM:
                break
        if not ok:
            print(f"  ! {name}: sem dados até {BACKOFF_LIM} dias úteis atrás para {target_d}")
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name"])

# ==========================
# 7) Pipeline (preço + valor) + duplicação
# ==========================

def main():
    # 1) carregar IPCA e bases existentes
    df_ipca = carregar_ipca_ipeadata()
    wide_preco = ler_wide(PATH_PRECO)   # preços (Preço de ajuste Atual)
    wide_valor = ler_wide(PATH_VALOR)   # valor do ajuste (DAP em R$; demais = pontos)

    # 2) definir range de datas
    def _last_col_date(df):
        if df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)
    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        # começa do mínimo last conhecido (mantém colunas alinhadas)
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)  # já temos até aqui; vamos em diante

    end_dt = ultimo_dia_util_antes_hoje()
    dias_util = b3_valid_days(start_dt, end_dt)

    print(f"Atualizando de {start_dt} até {end_dt} (dias úteis B3: {len(dias_util)})")

    # 3) iterar dias úteis
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref, df_ipca)
        if df_dia_all.empty:
            print(f"{dref}: nenhum dado disponível (pula)")
            continue

        # base longa (debug)
        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)

        # construir séries do dia
        s_preco, s_valor = construir_colunas_wide_duplas(df_dia_all, dref, df_ipca)

        # unir nas wides existentes
        for (wide, s) in ((wide_preco, s_preco), (wide_valor, s_valor)):
            if wide.empty:
                wide[s.name] = s
            else:
                wide = wide.reindex(wide.index.union(s.index))
                wide[s.name] = s
            # reatribui (porque a cópia retornada acima se perde)
            if s is s_preco:
                wide_preco = wide
            else:
                wide_valor = wide

        print(f"{dref}: preços/valores atualizados | base longa = {len(df_long)}")

    # 4) DUPLICAÇÃO — só duplica se AMBOS não forem cópias
    def _dup_if_needed(wp: pd.DataFrame, wv: pd.DataFrame):
        if wp.shape[1] < 2 or wv.shape[1] < 2:
            return wp, wv, False
        cols_p = sorted(pd.to_datetime(wp.columns))
        cols_v = sorted(pd.to_datetime(wv.columns))
        last_col_p, prev_col_p = cols_p[-1].strftime("%Y-%m-%d"), cols_p[-2].strftime("%Y-%m-%d")
        last_col_v, prev_col_v = cols_v[-1].strftime("%Y-%m-%d"), cols_v[-2].strftime("%Y-%m-%d")

        iguais_preco = wp[prev_col_p].equals(wp[last_col_p])
        iguais_valor = wv[prev_col_v].equals(wv[last_col_v])

        if iguais_preco and iguais_valor:
            print("Última coluna já é cópia idêntica — não duplica de novo.")
            return wp, wv, False

        # duplica ambos para o próximo útil do último dia de PREÇO (mantém grades alinhadas)
        last_dt = cols_p[-1].date()
        prox = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
        if len(prox) >= 2:
            prox_dt = prox[1]
            prox_col = prox_dt.strftime("%Y-%m-%d")
            if prox_col not in wp.columns:
                wp[prox_col] = wp[last_col_p]
            if prox_col not in wv.columns:
                wv[prox_col] = wv[last_col_v]
            print(f"Coluna duplicada para o próximo dia útil: {prox_col}")
            return wp, wv, True
        else:
            print("Não foi possível encontrar próximo dia útil para duplicação.")
            return wp, wv, False

    wide_preco, wide_valor, _ = _dup_if_needed(wide_preco, wide_valor)

    # 5) ordenar colunas e salvar
    def _order(df):
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO)
    salvar_wide(wide_valor, PATH_VALOR)
    print(f"Salvos:\n - {PATH_PRECO} {wide_preco.shape}\n - {PATH_VALOR} {wide_valor.shape}")

    # 6) JSON pt-BR dos PREÇOS (opcional)
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        print(f"Salvo JSON pt-BR de preços: {PATH_JSON}")
    except Exception as e:
        print(f"[warn] Falha ao gerar JSON pt-BR de preços: {e}")

if __name__ == "__main__":
    main()


Atualizando de 2025-11-12 até 2025-11-12 (dias úteis B3: 1)
  ! IPCACoupon: sem dados até 5 dias úteis atrás para 2025-11-12
  ! DI1Day: sem dados até 5 dias úteis atrás para 2025-11-12
  ! BusinessDollar: sem dados até 5 dias úteis atrás para 2025-11-12
  ! WDOMiniFuture: sem dados até 5 dias úteis atrás para 2025-11-12
  ! USTNOTEFuture: sem dados até 5 dias úteis atrás para 2025-11-12
2025-11-12: nenhum dado disponível (pula)
Última coluna já é cópia idêntica — não duplica de novo.
Salvos:
 - df_preco_de_ajuste_atual_completo.parquet (42, 219)
 - df_valor_ajuste_contrato.parquet (32, 219)
Salvo JSON pt-BR de preços: df_preco_de_ajuste_atual_completo.json


In [10]:
# -*- coding: utf-8 -*-
"""
Pipeline B3 (BDI) — IPCACoupon/DI/Dólar/WDO/Treasury
- Faz download do CSV (BDI export) por dia e instrumento.
- Faz parsing robusto (cabeçalhos variáveis). Para IPCACoupon, inclui parser
  específico que usa o bloco "Vencimento; ... ;Último Preço;Ajuste;Variação em Pontos"
  e linha-resumo (ex.: 'F26=98.168,92 ...') como fallback.
- Constrói bases longas e wide (Preço de Ajuste Atual e Valor de Ajuste),
  convertendo DAP (IPCACoupon) de pontos para R$.
- Atualiza para todos os dias úteis entre a última coluna salva e o último
  dia útil (B3) antes de hoje. Duplica a última coluna para o próximo dia útil
  se necessário.
"""

import datetime as dt
import io, re, os, json, unicodedata, hashlib, csv
from copy import deepcopy
from pathlib import Path

import pandas as pd
import requests
import pandas_market_calendars as mcal

# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO   = "PRECOS12_IPCA12"
IPCA_PREVISTO       = 0.15        # % ANBIMA do mês (fallback)
REAIS_POR_PONTO     = 0.00025     # R$ por ponto (coeficiente do contrato)
BACKOFF_LIM         = 15          # janela de backoff (dias ÚTEIS)

# Debug / inspeção
DEBUG_MAX_LINES = 40                       # quantas linhas imprimir como snippet
DEBUG_DUMP_DIR  = "debug_b3_csv"           # pasta para salvar CSVs brutos; use None para desativar

# arquivos (bases)
PATH_LONG    = "df_ajustes_b3.parquet"                       # base longa (depuração)
PATH_PRECO   = "df_preco_de_ajuste_atual_completo.parquet"   # PREÇOS (wide)
PATH_VALOR   = "df_valor_ajuste_contrato.parquet"            # VALOR AJUSTE (DAP em R$; demais = pontos)
PATH_JSON    = "df_preco_de_ajuste_atual_completo.json"      # JSON pt-BR de PREÇOS (opcional)

# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}
PAYLOAD_BASE = {"Name": "IPCACoupon","Date":"2025-11-10","FinalDate":"2025-11-10","ClientId":"","Filters":{}}

# Mercadorias/nomes para buscar
NAMES = ["IPCACoupon","DI1Day","BusinessDollar","WDOMiniFuture","USTNOTEFuture"]

# ==========================
# Helpers de JSON pt-BR (para export opcional)
# ==========================

try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None

def _fmt_ptbr_2dec(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return ""
    try:
        v = float(x)
    except Exception:
        return str(x)
    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    cols = []
    for c in wide_df.columns:
        try:
            cols.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols.append(c)
    wide = wide_df.copy()
    wide.columns = cols
    records = []
    for asset, row in wide.iterrows():
        d = {"Assets": str(asset)}
        for c in wide.columns:
            d[c] = _fmt_ptbr_2dec(row[c])
        records.append(d)
    return json.dumps(records, ensure_ascii=False)

# ==========================
# 2) Parsing — genérico + especial IPCACoupon
# ==========================

def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Name"], p["Date"], p["FinalDate"] = name, s, s
    return p

def _ensure_debug_dir():
    if DEBUG_DUMP_DIR:
        Path(DEBUG_DUMP_DIR).mkdir(parents=True, exist_ok=True)

def _dump_csv(name: str, data: dt.date, raw_bytes: bytes):
    if not DEBUG_DUMP_DIR:
        return
    _ensure_debug_dir()
    fn = Path(DEBUG_DUMP_DIR) / f"{name}_{data.strftime('%Y-%m-%d')}.csv"
    try:
        fn.write_bytes(raw_bytes)
        print(f"    [dump] CSV bruto salvo em: {fn}")
    except Exception as e:
        print(f"    [dump:fail] {e}")

def _print_snippet(tag: str, text: str, max_lines: int = DEBUG_MAX_LINES):
    lines = text.splitlines()
    header = f"----[ {tag} | primeiras {min(len(lines), max_lines)} de {len(lines)} linhas ]----"
    print(header)
    for ln in lines[:max_lines]:
        print(ln)
    print("-" * len(header))

def baixar_csv_bdi(name: str, data: dt.date) -> str:
    r = requests.post(URL, headers=HEADERS, json=montar_payload(name, data))
    clen = r.headers.get("Content-Length", "?")
    print(f"[HTTP] {name} {data} -> status={r.status_code} content-length={clen}")
    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    # salvar bruto (debug) + hash
    _dump_csv(name, data, r.content)
    md5 = hashlib.md5(r.content).hexdigest()  # noqa: S324 (ok p/ debug)
    print(f"        md5={md5} bytes={len(r.content)}")

    # tentar decodificar
    for enc in ("utf-8", "latin-1"):
        try:
            txt = r.content.decode(enc)
            break
        except UnicodeDecodeError:
            txt = None
    if txt is None:
        raise RuntimeError(f"{data} / {name}: falha ao decodificar (utf-8/latin-1)")

    # mostrar snippet bruto
    _print_snippet(f"RAW {name} {data}", txt)

    return txt

def _strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')

def _looks_like_header(line: str) -> bool:
    l = _strip_accents(line).lower()
    # Cabeçalhos típicos do bloco de Mercado Futuro
    return ("vencimento" in l) and (("preco" in l) or ("ajuste" in l)) and (("vari" in l) and ("ponto" in l))

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()
    start = None
    for i, line in enumerate(lines):
        if _looks_like_header(line):
            start = i
            break
    if start is None:
        return csv_text
    block = [lines[start]]
    for line in lines[start+1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block.append(line)
    return "\n".join(block)

def ptbr_to_float(s):
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s in {"", "-"}:
        return None
    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def parse_ajustes(csv_text: str, data_ref: dt.date, name: str) -> pd.DataFrame:
    """
    Parser genérico (cabeçalhos normalizados) — retorna colunas:
      Vencimento | Pontos (variação) | PrecoAjusteAtual | Data_Referencia | Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)
    target_text = bloco if ";" in bloco else csv_text

    # Log de cabeçalho detectado
    if bloco is csv_text:
        print(f"[parse] {name} {data_ref}: cabeçalho NÃO localizado — tentando CSV inteiro")
    else:
        first = bloco.splitlines()[0] if bloco else "<vazio>"
        print(f"[parse] {name} {data_ref}: cabeçalho localizado -> {first}")

    # Mostrar snippet do bloco que vamos realmente ler
    _print_snippet(f"PARSER_TARGET {name} {data_ref}", target_text)

    try:
        df_raw = pd.read_csv(io.StringIO(target_text), sep=";", dtype=str, engine="python", on_bad_lines="skip")
    except Exception:
        _print_snippet(f"PARSER_FAIL_{name}_{data_ref}", target_text)
        raise

    if df_raw.empty or df_raw.shape[1] < 3:
        _print_snippet(f"PARSER_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("CSV sem estrutura reconhecível para 'Ajustes do Pregão'.")

    # normalização
    colmap = {c: _strip_accents(c).lower() for c in df_raw.columns}
    print(f"[parse] colunas originais: {list(df_raw.columns)}")
    print(f"[parse] colunas normalizadas: {list(colmap.values())}")

    def _find_col(*must_have):
        for orig, norm in colmap.items():
            if all(x in norm for x in must_have):
                return orig
        return None

    c_venc  = _find_col("vencimento")
    # aceitar tanto "variacao em pontos" quanto "variacao pontos"
    c_var   = _find_col("vari", "ponto")
    # alguns instrumentos trazem "ajuste", outros "preco de ajuste atual"
    c_preco = _find_col("ajuste") or _find_col("preco", "ajuste", "atual")

    print(f"[parse] mapeadas -> Venc:{c_venc}  VarPts:{c_var}  Preco:{c_preco}")

    if c_venc is None or c_var is None or c_preco is None:
        _print_snippet(f"PARSER_HDR_MISSING_{name}_{data_ref}", "\n".join(df_raw.columns.astype(str)))
        raise ValueError(f"Colunas não encontradas. Cabeçalhos vistos: {list(df_raw.columns)}")

    df = df_raw[[c_venc, c_var, c_preco]].copy()
    df.columns = ["Vencimento", "Pontos", "PrecoAjusteAtual"]
    df["Pontos"] = df["Pontos"].apply(ptbr_to_float)
    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].apply(ptbr_to_float)

    print("[parse] amostra parseada (até 10 linhas):")
    print(df.head(10).to_string(index=False))

    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        _print_snippet(f"PARSER_CLEAN_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("Após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df.reset_index(drop=True, inplace=True)
    return df

# ---------- PARSER ESPECIAL IPCACoupon ----------
_PT_BR_NUM = re.compile(r'^-?\d{1,3}(\.\d{3})*(,\d+)?$')

def _ipc_to_float_ptbr(s: str):
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s in {"-", "–", "—"}:
        return None
    s = s.replace("↑", "").replace("↓", "")
    if _PT_BR_NUM.match(s):
        s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None

def _ipc_clean_text(t: str) -> str:
    return (t or "").replace("\ufeff", "").replace("\xa0", " ").replace("\r\n", "\n").replace("\r", "\n")

def _ipc_slice_table_block(text: str) -> tuple[str, str, float | None]:
    """
    Retorna (bloco_csv, linha_resumo, valor_indice_ipca_pro_rata)
    - bloco_csv inclui cabeçalho que inicia com 'Vencimento;'
    - linha_resumo é algo como 'F26=98.168,92 F27=...'
    - valor_indice_ipca_pro_rata vem da linha 'Valor Índice Ipca pro Rata Tempore: 7.367,7', se existir
    """
    txt = _ipc_clean_text(text)

    # 1) Encontrar cabeçalho da tabela
    m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação em Pontos;.*$', txt)
    if not m_head:
        # aceitar cabeçalhos mais curtos (quando a B3 corta colunas de oferta/demanda)
        m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação.*$', txt)
    if not m_head:
        raise ValueError("Cabeçalho 'Vencimento;' não encontrado no CSV da B3 (IPCACoupon).")

    start = m_head.start()

    # 2) Final do bloco normalmente antes de '* Preços de Ajustes Corrigidos ...'
    m_end = re.search(r'(?m)^\* Preços.*$', txt)
    end = m_end.start() if m_end else len(txt)
    bloco = txt[start:end].strip()

    # 3) Linha-resumo (F26=..., etc.)
    m_resumo = re.search(r'(?mi)^(?:[FGHJKMNQUVXZ]\d{2}=\d{1,3}\.\d{3},\d{2}\s*)+', txt)
    linha_resumo = m_resumo.group(0).strip() if m_resumo else ""

    # 4) Valor do Índice IPCA pro rata do próprio arquivo (prioritário p/ valor por ponto)
    m_ind = re.search(r'Valor\s+Índice\s+Ipca\s+pro\s+Rata\s+Tempore:\s*([0-9\.\,]+)', txt, re.IGNORECASE)
    valor_indice_ipca = _ipc_to_float_ptbr(m_ind.group(1)) if m_ind else None

    return bloco, linha_resumo, valor_indice_ipca

def parse_ipcacoupon_special(csv_text: str, data_ref: dt.date, name: str = "IPCACoupon") -> pd.DataFrame:
    """
    Parser dedicado para IPCACoupon. Retorna dataframe com colunas:
      Vencimento | Pontos | PrecoAjusteAtual | Data_Referencia | Name | ValorIndiceDia
    - PrecoAjusteAtual vem de 'Ajuste' (ou fallback da linha-resumo).
    - Pontos pode vir vazio/NaN (não é essencial para DAP em R$).
    - ValorIndiceDia é extraído da linha 'Valor Índice Ipca pro Rata Tempore: ...' (se houver).
    """
    bloco, linha_resumo, valor_indice_ipca = _ipc_slice_table_block(csv_text)
    linhas = [ln for ln in bloco.split("\n") if ln.strip()]
    reader = csv.reader(linhas, delimiter=';')
    header = next(reader)
    cols = [c.strip() for c in header]
    try:
        idx_venc    = cols.index("Vencimento")
        idx_ult     = cols.index("Último Preço")
        idx_aj      = cols.index("Ajuste")
        # "Variação em Pontos" pode ser truncado em alguns exports; usar 'Variação' e 'Pontos'
        idx_var = None
        for i, c in enumerate(cols):
            cc = c.replace(" ", "").lower()
            if ("vari" in cc) and ("ponto" in cc):
                idx_var = i
                break
    except ValueError as e:
        raise ValueError(f"Colunas esperadas (IPCACoupon) não encontradas no header {cols!r}: {e}")

    rows = []
    cod_venc_pat = re.compile(r'^[FGHJKMNQUVXZ]\d{2}$')

    for r in reader:
        if not r or len(r) <= idx_aj:
            continue
        venc = r[idx_venc].strip()
        if not cod_venc_pat.match(venc):
            continue

        ultimo_preco = _ipc_to_float_ptbr(r[idx_ult])
        ajuste = _ipc_to_float_ptbr(r[idx_aj])
        pontos = _ipc_to_float_ptbr(r[idx_var]) if idx_var is not None and idx_var < len(r) else None

        rows.append({
            "Vencimento": venc,
            "Pontos": pontos,
            "PrecoAjusteAtual": ajuste,   # 'Ajuste' = preço de ajuste do dia
            "UltimoPreco": ultimo_preco,
        })

    df = pd.DataFrame(rows).sort_values("Vencimento", ignore_index=True)

    # Fallback/validação com a linha-resumo -> preencher PrecoAjusteAtual faltante
    if linha_resumo:
        ajustes_resumo = {}
        for token in linha_resumo.split():
            if "=" in token:
                k, v = token.split("=", 1)
                k, v = k.strip(), v.strip().rstrip(";")
                fv = _ipc_to_float_ptbr(v)
                if cod_venc_pat.match(k) and fv is not None:
                    ajustes_resumo[k] = fv
        if not df.empty and ajustes_resumo:
            df["PrecoAjusteAtual"] = df.apply(
                lambda x: x["PrecoAjusteAtual"] if pd.notnull(x["PrecoAjusteAtual"]) else ajustes_resumo.get(x["Vencimento"]),
                axis=1
            )

    # Filtra linhas válidas
    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        raise ValueError("IPCACoupon: após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df["ValorIndiceDia"] = valor_indice_ipca  # pode ser None; wide trata fallback
    df.reset_index(drop=True, inplace=True)
    return df[["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"]]

# ==========================
# 3) IPCA helpers (IPEA) — fallback quando arquivo não trouxer o Índice do dia
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{IPCA_SERIE_CODIGO}')"
    resp = requests.get(url); resp.raise_for_status()
    df = pd.DataFrame(resp.json()["value"])
    df["VALDATA"] = pd.to_datetime(df["VALDATA"].astype(str).str[:10], errors="coerce")
    df = df[["VALDATA","VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)
    return df.rename(columns={"VALDATA":"Data","VALVALOR":"IPCA_Indice"})

def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])

def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:
        d += dt.timedelta(days=1)
    return d

def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev = dt.date(data_ref.year, data_ref.month, 15)
        nxt_m = 1 if data_ref.month == 12 else data_ref.month + 1
        nxt_y = data_ref.year + 1 if data_ref.month == 12 else data_ref.year
        nxt = dt.date(nxt_y, nxt_m, 15)
    else:
        pm = 12 if data_ref.month == 1 else data_ref.month - 1
        py = data_ref.year - 1 if data_ref.month == 1 else data_ref.year
        prev = dt.date(py, pm, 15)
        nxt  = dt.date(data_ref.year, data_ref.month, 15)
    return prev, nxt

def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float = IPCA_PREVISTO,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    """
    Fallback: valor por ponto (R$) = 0,00025 * Índice(IPCA) pro rata do dia.
    """
    data_ref = pd.to_datetime(data_ref).date()
    prev_15, next_15 = datas_ipca_referencia(data_ref)
    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)
    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")
    ni_ref = obter_ipca_ref(df_ipca, prev_adj)
    ipca_pro_rata = ni_ref * (1 + ipca_previsto/100) ** (du_desde/du_entre)
    return ipca_pro_rata * reais_por_ponto

# ==========================
# 4) Base longa (append dedup)
# ==========================

def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        return pd.read_parquet(p)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                            chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb

# ==========================
# 5) Wides (preço e valor) — leitura/escrita
# ==========================

def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")
    df = pd.read_parquet(path)
    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"
    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass
    df = df[~df.index.duplicated(keep="last")]
    return df

def salvar_wide(df: pd.DataFrame, path: str):
    df2 = df.copy()
    df2.index.name = "Assets"
    out = df2.reset_index().rename(columns={df2.reset_index().columns[0]:"Assets"})
    out.to_parquet(path, index=False)

def mapear_asset(name: str, venc: str) -> str | None:
    suf = (venc or "").strip()[-2:]
    if name == "IPCACoupon":     return f"DAP{suf}"
    if name == "DI1Day":         return f"DI_{suf}"
    if name in ("BusinessDollar","WDOMiniFuture"): return "WDO1"
    if name == "USTNOTEFuture":  return "TREASURY"
    return None

def _valor_por_ponto_dap(df_ipca: pd.DataFrame, data_ref: dt.date, df_long_dia: pd.DataFrame) -> float:
    """
    Calcula o valor por ponto (R$) para DAP:
    - 1º prioridade: 'ValorIndiceDia' do arquivo da B3 (linha "Valor Índice Ipca pro Rata Tempore").
      ValorPorPonto = 0.00025 * ValorIndiceDia
    - Fallback: cálculo via IPEA / projeção (método anterior).
    """
    vi = None
    try:
        vi = df_long_dia.loc[df_long_dia["Name"]=="IPCACoupon","ValorIndiceDia"].dropna().iloc[0]
    except Exception:
        vi = None
    if vi is not None:
        vpp = REAIS_POR_PONTO * float(vi)
        print(f"[DAP] Valor por ponto via arquivo B3: Índice={vi:.4f} -> R$ {vpp:.6f}")
        return vpp
    # fallback
    vpp = calcular_valor_ponto_dap_para_data(df_ipca, data_ref)
    print(f"[DAP] Valor por ponto via fallback IPEA: R$ {vpp:.6f}")
    return vpp

def construir_colunas_wide_duplas(
    df_long_dia: pd.DataFrame,
    data_ref: dt.date,
    df_ipca: pd.DataFrame
) -> tuple[pd.Series, pd.Series]:
    """
    Retorna:
      s_preco -> série por Asset com PrecoAjusteAtual
      s_valor -> série por Asset com Valor de Ajuste (DAP em R$; demais = pontos)
    """
    df = df_long_dia.copy()
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()]

    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()
    s_valor = df.groupby("Asset")["Pontos"].first()

    # Converter DAP (Pontos -> R$) usando o valor por ponto do próprio arquivo (ou fallback)
    if not df[df["Name"] == "IPCACoupon"].empty:
        valor_ponto_dap = _valor_por_ponto_dap(df_ipca, data_ref, df_long_dia)
        for k in list(s_valor.index):
            if str(k).startswith("DAP") and pd.notna(s_valor.loc[k]):
                try:
                    s_valor.loc[k] = float(s_valor.loc[k]) * valor_ponto_dap
                except Exception:
                    pass

    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name
    return s_preco, s_valor

# ==========================
# 6) Calendário B3 + backoff
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")

def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]

def ultimo_dia_util_antes_hoje() -> dt.date:
    today = dt.date.today()
    val = b3_calendar().valid_days(today - dt.timedelta(days=15), today)
    return val[-1].date()

def buscar_um_dia(name: str, d: dt.date) -> pd.DataFrame | None:
    csv_text = baixar_csv_bdi(name, d)
    # Preferir parser especial para IPCACoupon
    if name == "IPCACoupon":
        try:
            return parse_ipcacoupon_special(csv_text, d, name)
        except Exception as e_special:
            print(f"[warn] IPCACoupon parser especial falhou ({e_special}); tentando parser genérico…")
    return parse_ajustes(csv_text, d, name)

def buscar_dia_com_backoff(target_d: dt.date, df_ipca: pd.DataFrame) -> pd.DataFrame:
    dfs = []
    validos_back = b3_valid_days(target_d - dt.timedelta(days=60), target_d)[::-1]
    tentativas_max = max(BACKOFF_LIM, 15)

    for name in NAMES:
        ok = False
        tentativa = 0
        for prev_d in [target_d] + validos_back:
            try:
                csv_text = baixar_csv_bdi(name, prev_d)
                # Tenta parser especial se IPCACoupon; senão, parser genérico
                if name == "IPCACoupon":
                    try:
                        df_n = parse_ipcacoupon_special(csv_text, prev_d, name)
                    except Exception as e_special:
                        print(f"[warn] IPCACoupon parser especial falhou ({e_special}); usando genérico.")
                        df_n = parse_ajustes(csv_text, prev_d, name)
                else:
                    df_n = parse_ajustes(csv_text, prev_d, name)

                if df_n is not None and not df_n.empty:
                    # Normaliza Data_Referencia para o target (backfill)
                    df_n["Data_Referencia"] = pd.to_datetime(target_d)
                    dfs.append(df_n)
                    ok = True
                    if prev_d != target_d:
                        print(f"  • {name}: {target_d} vazio → usando {prev_d} (backfill)")
                    break
            except Exception as e:
                msg = str(e)
                print(f"  ! {name} @ {prev_d}: falhou parse ({msg[:120]})")
            tentativa += 1
            if tentativa > tentativas_max:
                break
        if not ok:
            print(f"  ! {name}: sem dados até {tentativas_max} dias úteis atrás para {target_d}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

# ==========================
# 7) Pipeline (preço + valor) + duplicação
# ==========================

def main():
    # 1) carregar IPCA e bases existentes
    df_ipca = carregar_ipca_ipeadata()
    wide_preco = ler_wide(PATH_PRECO)
    wide_valor = ler_wide(PATH_VALOR)

    # 2) definir range de datas
    def _last_col_date(df):
        if df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)
    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)

    end_dt = ultimo_dia_util_antes_hoje()
    dias_util = b3_valid_days(start_dt, end_dt)

    print(f"Atualizando de {start_dt} até {end_dt} (dias úteis B3: {len(dias_util)})")

    # 3) iterar dias úteis
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref, df_ipca)
        if df_dia_all.empty:
            print(f"{dref}: nenhum dado disponível (pula)")
            continue

        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)

        s_preco, s_valor = construir_colunas_wide_duplas(df_dia_all, dref, df_ipca)

        if wide_preco.empty:
            wide_preco = pd.DataFrame(s_preco)
        else:
            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_preco[s_preco.name] = s_preco

        if wide_valor.empty:
            wide_valor = pd.DataFrame(s_valor)
        else:
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_valor[s_valor.name] = s_valor

        print(f"{dref}: preços/valores atualizados | base longa = {len(df_long)}")

    # ====== TESTE: dropar últimas N colunas antes de duplicar (desativado por padrão) ======
    DROP_LAST_N = 0
    if DROP_LAST_N > 0:
        def _drop_last_n(df, n):
            if df.shape[1] <= n:
                return df
            cols = sorted(pd.to_datetime(df.columns))
            drops = [c.strftime("%Y-%m-%d") for c in cols[-n:]]
            print(f"[TEST] Dropando últimas {n} colunas -> {', '.join(drops)}")
            return df.drop(columns=drops, errors="ignore")
        wide_preco = _drop_last_n(wide_preco, DROP_LAST_N)
        wide_valor = _drop_last_n(wide_valor, DROP_LAST_N)
    # =======================================================================================

    # 4) DUPLICAÇÃO — só duplica se AMBOS não forem cópias
    def _dup_if_needed(wp: pd.DataFrame, wv: pd.DataFrame):
        if wp.shape[1] < 2 or wv.shape[1] < 2:
            return wp, wv, False
        cols_p = sorted(pd.to_datetime(wp.columns))
        cols_v = sorted(pd.to_datetime(wv.columns))
        last_col_p, prev_col_p = cols_p[-1].strftime("%Y-%m-%d"), cols_p[-2].strftime("%Y-%m-%d")
        last_col_v, prev_col_v = cols_v[-1].strftime("%Y-%m-%d"), cols_v[-2].strftime("%Y-%m-%d")

        iguais_preco = wp[prev_col_p].equals(wp[last_col_p])
        iguais_valor = wv[prev_col_v].equals(wv[last_col_v])

        if iguais_preco and iguais_valor:
            print("Última coluna já é cópia idêntica — não duplica de novo.")
            return wp, wv, False

        last_dt = cols_p[-1].date()
        prox = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
        if len(prox) >= 2:
            prox_dt = prox[1]
            prox_col = prox_dt.strftime("%Y-%m-%d")
            if prox_col not in wp.columns:
                wp[prox_col] = wp[last_col_p]
            if prox_col not in wv.columns:
                wv[prox_col] = wv[last_col_v]
            print(f"Coluna duplicada para o próximo dia útil: {prox_col}")
            return wp, wv, True
        else:
            print("Não foi possível encontrar próximo dia útil para duplicação.")
            return wp, wv, False

    wide_preco, wide_valor, _ = _dup_if_needed(wide_preco, wide_valor)

    # 5) ordenar colunas e salvar
    def _order(df):
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO)
    salvar_wide(wide_valor, PATH_VALOR)
    print(f"Salvos:\n - {PATH_PRECO} {wide_preco.shape}\n - {PATH_VALOR} {wide_valor.shape}")

    # 6) JSON pt-BR dos PREÇOS (opcional)
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        print(f"Salvo JSON pt-BR de preços: {PATH_JSON}")
    except Exception as e:
        print(f"[warn] Falha ao gerar JSON pt-BR de preços: {e}")

if __name__ == "__main__":
    main()


Atualizando de 2025-11-05 até 2025-11-12 (dias úteis B3: 6)
[HTTP] IPCACoupon 2025-11-05 -> status=200 content-length=1364
    [dump] CSV bruto salvo em: debug_b3_csv\IPCACoupon_2025-11-05.csv
        md5=844e201c528e32dcb063d4e22c1b257d bytes=2421
----[ RAW IPCACoupon 2025-11-05 | primeiras 28 de 28 linhas ]----
﻿Mercado Futuro

Vencimento;Contratos em Aberto;Negócios Realizados;Contratos Negociados;Volume;Ajuste Anterior;Preço de Abertura;Preço Mínimo;Preço Máximo;Preço Médio;Último Preço;Ajuste;Variação em Pontos;Última Oferta de Compra;Última Oferta de Venda
F26;61.257;6;114;20.600.481;98.107,88;10,44;10,44;10,44;10,44;10,44;98.121,23;13,35↑;10,37;11,10
F27;8.601;-;-;-;90.110,66;-;-;-;-;-;90.090,73;-19,93↓;-;-
G26;-;-;-;-;97.345,41;-;-;-;-;-;97.345,47;0,06↑;9,95;10,39
H26;-;-;-;-;96.867,69;-;-;-;-;-;96.871,23;3,54↑;9,31;9,75
J26;-;-;-;-;96.120,52;-;-;-;-;-;96.123,99;3,47↑;9,35;9,79
K27;331.851;52;14.887;2.416.438.427;88.155,20;8,74;8,74;8,78;8,76;8,76;88.135,62;-19,58↓;8,75;8,76
K2

In [ ]:
# -*- coding: utf-8 -*-
"""
Pipeline B3 (BDI) — IPCACoupon/DI/Dólar/WDO/Treasury
- Faz download do CSV (BDI export) por dia e instrumento.
- Faz parsing robusto (cabeçalhos variáveis). Para IPCACoupon, inclui parser
  específico que usa o bloco "Vencimento; ... ;Último Preço;Ajuste;Variação em Pontos"
  e linha-resumo (ex.: 'F26=98.168,92 ...') como fallback.
- Constrói bases longas e wide (Preço de Ajuste Atual e Valor de Ajuste),
  convertendo DAP (IPCACoupon) de pontos para R$.
- Atualiza para todos os dias úteis entre a última coluna salva e o último
  dia útil (B3) antes de hoje.
- Se não houver dados do dia, NÃO atualiza — apenas considera duplicar 1x a última
  coluna para o próximo dia útil (no máximo 1 duplicação).
- Exporta JSON/CSV com números em TEXTO pt-BR.
"""

import datetime as dt
import io, re, os, json, unicodedata, hashlib, csv
from copy import deepcopy
from pathlib import Path

import pandas as pd
import requests
import pandas_market_calendars as mcal

# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO   = "PRECOS12_IPCA12"
IPCA_PREVISTO       = 0.15        # % ANBIMA do mês (fallback)
REAIS_POR_PONTO     = 0.00025     # R$ por ponto (coeficiente do contrato)
BACKOFF_LIM         = 15          # janela de backoff (dias ÚTEIS)

# Debug / inspeção
DEBUG_MAX_LINES = 40                       # quantas linhas imprimir como snippet
DEBUG_DUMP_DIR  = "debug_b3_csv"           # pasta para salvar CSVs brutos; use None para desativar

# arquivos (bases)
PATH_LONG       = "df_ajustes_b3.parquet"                        # base longa (depuração)
PATH_PRECO      = "df_preco_de_ajuste_atual_completo.parquet"    # PREÇOS (wide)
PATH_VALOR      = "df_valor_ajuste_contrato.parquet"             # VALOR AJUSTE (DAP em R$; demais = pontos)
PATH_JSON       = "df_preco_de_ajuste_atual_completo.json"       # JSON pt-BR de PREÇOS
PATH_PRECO_CSV  = "df_preco_de_ajuste_atual_completo.csv"        # CSV pt-BR texto (opcional)
PATH_VALOR_CSV  = "df_valor_ajuste_contrato.csv"                 # CSV pt-BR texto (opcional)
PATH_RUN_LOG    = "atualizacao_b3_log.txt"                       # LOG plano das atualizações/duplicações

# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}
PAYLOAD_BASE = {"Name": "IPCACoupon","Date":"2025-11-10","FinalDate":"2025-11-10","ClientId":"","Filters":{}}

# Mercadorias/nomes para buscar
NAMES = ["IPCACoupon","DI1Day","BusinessDollar","WDOMiniFuture","USTNOTEFuture"]

# ==========================
# Helpers gerais + JSON pt-BR
# ==========================

try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None

def _append_log(msg: str):
    ts = dt.datetime.now(_TZ).strftime("%Y-%m-%d %H:%M:%S") if _TZ else dt.datetime.now().isoformat(sep=" ", timespec="seconds")
    line = f"[{ts}] {msg}"
    print(line)
    try:
        with open(PATH_RUN_LOG, "a", encoding="utf-8") as f:
            f.write(line + "\n")
    except Exception:
        pass

def _fmt_ptbr_2dec(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return ""
    try:
        v = float(x)
    except Exception:
        # já é texto -> mantém
        return str(x)
    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    """
    Exporta o wide como uma lista de registros JSON, sempre em texto pt-BR (com vírgula decimal),
    garantindo consistência inclusive nas colunas recém-duplicadas.
    """
    if wide_df is None or wide_df.empty:
        return "[]"

    # Padroniza nomes de colunas para 'YYYY-MM-DD'
    cols_norm = []
    for c in wide_df.columns:
        try:
            cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols_norm.append(str(c))
    df = wide_df.copy()
    df.columns = cols_norm

    # Converte TUDO para string pt-BR
    df_str = df.applymap(_fmt_ptbr_2dec)

    # Garante coluna 'Assets' como primeira coluna
    df_str = df_str.copy()
    df_str.index.name = "Assets"
    out = df_str.reset_index().rename(columns={df_str.reset_index().columns[0]: "Assets"})

    # Agora todo valor já é string -> JSON sempre com aspas
    return out.to_json(orient="records", force_ascii=False)

# ==========================
# 2) Parsing — genérico + especial IPCACoupon
# ==========================

def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Name"], p["Date"], p["FinalDate"] = name, s, s
    return p

def _ensure_debug_dir():
    if DEBUG_DUMP_DIR:
        Path(DEBUG_DUMP_DIR).mkdir(parents=True, exist_ok=True)

def _dump_csv(name: str, data: dt.date, raw_bytes: bytes):
    if not DEBUG_DUMP_DIR:
        return
    _ensure_debug_dir()
    fn = Path(DEBUG_DUMP_DIR) / f"{name}_{data.strftime('%Y-%m-%d')}.csv"
    try:
        fn.write_bytes(raw_bytes)
        print(f"    [dump] CSV bruto salvo em: {fn}")
    except Exception as e:
        print(f"    [dump:fail] {e}")

def _print_snippet(tag: str, text: str, max_lines: int = DEBUG_MAX_LINES):
    lines = text.splitlines()
    header = f"----[ {tag} | primeiras {min(len(lines), max_lines)} de {len(lines)} linhas ]----"
    print(header)
    for ln in lines[:max_lines]:
        print(ln)
    print("-" * len(header))

def baixar_csv_bdi(name: str, data: dt.date) -> str:
    r = requests.post(URL, headers=HEADERS, json=montar_payload(name, data))
    clen = r.headers.get("Content-Length", "?")
    _append_log(f"[HTTP] {name} {data} -> status={r.status_code} content-length={clen}")
    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    # salvar bruto (debug) + hash
    _dump_csv(name, data, r.content)
    md5 = hashlib.md5(r.content).hexdigest()  # noqa: S324
    print(f"        md5={md5} bytes={len(r.content)}")

    # tentar decodificar
    for enc in ("utf-8", "latin-1"):
        try:
            txt = r.content.decode(enc)
            break
        except UnicodeDecodeError:
            txt = None
    if txt is None:
        raise RuntimeError(f"{data} / {name}: falha ao decodificar (utf-8/latin-1)")

    # mostrar snippet bruto
    _print_snippet(f"RAW {name} {data}", txt)

    return txt

def _strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')

def _looks_like_header(line: str) -> bool:
    l = _strip_accents(line).lower()
    # Cabeçalhos típicos do bloco de Mercado Futuro
    return ("vencimento" in l) and (("preco" in l) or ("ajuste" in l)) and (("vari" in l) and ("ponto" in l))

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()
    start = None
    for i, line in enumerate(lines):
        if _looks_like_header(line):
            start = i
            break
    if start is None:
        return csv_text
    block = [lines[start]]
    for line in lines[start+1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block.append(line)
    return "\n".join(block)

def ptbr_to_float(s):
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s in {"", "-"}:
        return None
    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def parse_ajustes(csv_text: str, data_ref: dt.date, name: str) -> pd.DataFrame:
    """
    Parser genérico (cabeçalhos normalizados) — retorna colunas:
      Vencimento | Pontos (variação) | PrecoAjusteAtual | Data_Referencia | Name
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)
    target_text = bloco if ";" in bloco else csv_text

    # Log de cabeçalho detectado
    if bloco is csv_text:
        print(f"[parse] {name} {data_ref}: cabeçalho NÃO localizado — tentando CSV inteiro")
    else:
        first = bloco.splitlines()[0] if bloco else "<vazio>"
        print(f"[parse] {name} {data_ref}: cabeçalho localizado -> {first}")

    # Mostrar snippet do bloco que vamos realmente ler
    _print_snippet(f"PARSER_TARGET {name} {data_ref}", target_text)

    try:
        df_raw = pd.read_csv(io.StringIO(target_text), sep=";", dtype=str, engine="python", on_bad_lines="skip")
    except Exception:
        _print_snippet(f"PARSER_FAIL_{name}_{data_ref}", target_text)
        raise

    if df_raw.empty or df_raw.shape[1] < 3:
        _print_snippet(f"PARSER_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("CSV sem estrutura reconhecível para 'Ajustes do Pregão'.")

    # normalização
    colmap = {c: _strip_accents(c).lower() for c in df_raw.columns}
    print(f"[parse] colunas originais: {list(df_raw.columns)}")
    print(f"[parse] colunas normalizadas: {list(colmap.values())}")

    def _find_col(*must_have):
        for orig, norm in colmap.items():
            if all(x in norm for x in must_have):
                return orig
        return None

    c_venc  = _find_col("vencimento")
    # aceitar tanto "variacao em pontos" quanto "variacao pontos"
    c_var   = _find_col("vari", "ponto")
    # alguns instrumentos trazem "ajuste", outros "preco de ajuste atual"
    c_preco = _find_col("ajuste") or _find_col("preco", "ajuste", "atual")

    print(f"[parse] mapeadas -> Venc:{c_venc}  VarPts:{c_var}  Preco:{c_preco}")

    if c_venc is None or c_var is None or c_preco is None:
        _print_snippet(f"PARSER_HDR_MISSING_{name}_{data_ref}", "\n".join(df_raw.columns.astype(str)))
        raise ValueError(f"Colunas não encontradas. Cabeçalhos vistos: {list(df_raw.columns)}")

    df = df_raw[[c_venc, c_var, c_preco]].copy()
    df.columns = ["Vencimento", "Pontos", "PrecoAjusteAtual"]
    df["Pontos"] = df["Pontos"].apply(ptbr_to_float)
    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].apply(ptbr_to_float)

    print("[parse] amostra parseada (até 10 linhas):")
    print(df.head(10).to_string(index=False))

    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        _print_snippet(f"PARSER_CLEAN_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("Após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df.reset_index(drop=True, inplace=True)
    return df

# ---------- PARSER ESPECIAL IPCACoupon ----------
_PT_BR_NUM = re.compile(r'^-?\d{1,3}(\.\d{3})*(,\d+)?$')

def _ipc_to_float_ptbr(s: str):
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s in {"-", "–", "—"}:
        return None
    s = s.replace("↑", "").replace("↓", "")
    if _PT_BR_NUM.match(s):
        s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None

def _ipc_clean_text(t: str) -> str:
    return (t or "").replace("\ufeff", "").replace("\xa0", " ").replace("\r\n", "\n").replace("\r", "\n")

def _ipc_slice_table_block(text: str) -> tuple[str, str, float | None]:
    """
    Retorna (bloco_csv, linha_resumo, valor_indice_ipca_pro_rata)
    - bloco_csv inclui cabeçalho que inicia com 'Vencimento;'
    - linha_resumo é algo como 'F26=98.168,92 F27=...'
    - valor_indice_ipca_pro_rata vem da linha 'Valor Índice Ipca pro Rata Tempore: 7.367,7', se existir
    """
    txt = _ipc_clean_text(text)

    # 1) Encontrar cabeçalho da tabela
    m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação em Pontos;.*$', txt)
    if not m_head:
        # aceitar cabeçalhos mais curtos (quando a B3 corta colunas de oferta/demanda)
        m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação.*$', txt)
    if not m_head:
        raise ValueError("Cabeçalho 'Vencimento;' não encontrado no CSV da B3 (IPCACoupon).")

    start = m_head.start()

    # 2) Final do bloco normalmente antes de '* Preços de Ajustes Corrigidos ...'
    m_end = re.search(r'(?m)^\* Preços.*$', txt)
    end = m_end.start() if m_end else len(txt)
    bloco = txt[start:end].strip()

    # 3) Linha-resumo (F26=..., etc.)
    m_resumo = re.search(r'(?mi)^(?:[FGHJKMNQUVXZ]\d{2}=\d{1,3}\.\d{3},\d{2}\s*)+', txt)
    linha_resumo = m_resumo.group(0).strip() if m_resumo else ""

    # 4) Valor do Índice IPCA pro rata do próprio arquivo (prioritário p/ valor por ponto)
    m_ind = re.search(r'Valor\s+Índice\s+Ipca\s+pro\s+Rata\s+Tempore:\s*([0-9\.\,]+)', txt, re.IGNORECASE)
    valor_indice_ipca = _ipc_to_float_ptbr(m_ind.group(1)) if m_ind else None

    return bloco, linha_resumo, valor_indice_ipca

def parse_ipcacoupon_special(csv_text: str, data_ref: dt.date, name: str = "IPCACoupon") -> pd.DataFrame:
    """
    Parser dedicado para IPCACoupon. Retorna dataframe com colunas:
      Vencimento | Pontos | PrecoAjusteAtual | Data_Referencia | Name | ValorIndiceDia
    - PrecoAjusteAtual vem de 'Ajuste' (ou fallback da linha-resumo).
    - Pontos pode vir vazio/NaN (não é essencial para DAP em R$).
    - ValorIndiceDia é extraído da linha 'Valor Índice Ipca pro Rata Tempore: ...' (se houver).
    """
    bloco, linha_resumo, valor_indice_ipca = _ipc_slice_table_block(csv_text)
    linhas = [ln for ln in bloco.split("\n") if ln.strip()]
    reader = csv.reader(linhas, delimiter=';')
    header = next(reader)
    cols = [c.strip() for c in header]
    try:
        idx_venc    = cols.index("Vencimento")
        idx_ult     = cols.index("Último Preço")
        idx_aj      = cols.index("Ajuste")
        # "Variação em Pontos" pode ser truncado em alguns exports; usar 'Variação' e 'Pontos'
        idx_var = None
        for i, c in enumerate(cols):
            cc = c.replace(" ", "").lower()
            if ("vari" in cc) and ("ponto" in cc):
                idx_var = i
                break
    except ValueError as e:
        raise ValueError(f"Colunas esperadas (IPCACoupon) não encontradas no header {cols!r}: {e}")

    rows = []
    cod_venc_pat = re.compile(r'^[FGHJKMNQUVXZ]\d{2}$')

    for r in reader:
        if not r or len(r) <= idx_aj:
            continue
        venc = r[idx_venc].strip()
        if not cod_venc_pat.match(venc):
            continue

        ultimo_preco = _ipc_to_float_ptbr(r[idx_ult])
        ajuste = _ipc_to_float_ptbr(r[idx_aj])
        pontos = _ipc_to_float_ptbr(r[idx_var]) if idx_var is not None and idx_var < len(r) else None

        rows.append({
            "Vencimento": venc,
            "Pontos": pontos,
            "PrecoAjusteAtual": ajuste,   # 'Ajuste' = preço de ajuste do dia
            "UltimoPreco": ultimo_preco,
        })

    df = pd.DataFrame(rows).sort_values("Vencimento", ignore_index=True)

    # Fallback/validação com a linha-resumo -> preencher PrecoAjusteAtual faltante
    if linha_resumo:
        ajustes_resumo = {}
        for token in linha_resumo.split():
            if "=" in token:
                k, v = token.split("=", 1)
                k, v = k.strip(), v.strip().rstrip(";")
                fv = _ipc_to_float_ptbr(v)
                if cod_venc_pat.match(k) and fv is not None:
                    ajustes_resumo[k] = fv
        if not df.empty and ajustes_resumo:
            df["PrecoAjusteAtual"] = df.apply(
                lambda x: x["PrecoAjusteAtual"] if pd.notnull(x["PrecoAjusteAtual"]) else ajustes_resumo.get(x["Vencimento"]),
                axis=1
            )

    # Filtra linhas válidas
    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        raise ValueError("IPCACoupon: após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df["ValorIndiceDia"] = valor_indice_ipca  # pode ser None; wide trata fallback
    df.reset_index(drop=True, inplace=True)
    return df[["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"]]

# ==========================
# 3) IPCA helpers (IPEA) — fallback quando arquivo não trouxer o Índice do dia
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{IPCA_SERIE_CODIGO}')"
    resp = requests.get(url); resp.raise_for_status()
    df = pd.DataFrame(resp.json()["value"])
    df["VALDATA"] = pd.to_datetime(df["VALDATA"].astype(str).str[:10], errors="coerce")
    df = df[["VALDATA","VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)
    return df.rename(columns={"VALDATA":"Data","VALVALOR":"IPCA_Indice"})

def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])

def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:
        d += dt.timedelta(days=1)
    return d

def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev = dt.date(data_ref.year, data_ref.month, 15)
        nxt_m = 1 if data_ref.month == 12 else data_ref.month + 1
        nxt_y = data_ref.year + 1 if data_ref.month == 12 else data_ref.year
        nxt = dt.date(nxt_y, nxt_m, 15)
    else:
        pm = 12 if data_ref.month == 1 else data_ref.month - 1
        py = data_ref.year - 1 if data_ref.month == 1 else data_ref.year
        prev = dt.date(py, pm, 15)
        nxt  = dt.date(data_ref.year, data_ref.month, 15)
    return prev, nxt

def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float = IPCA_PREVISTO,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    """
    Fallback: valor por ponto (R$) = 0,00025 * Índice(IPCA) pro rata do dia.
    """
    data_ref = pd.to_datetime(data_ref).date()
    prev_15, next_15 = datas_ipca_referencia(data_ref)
    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)
    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")
    ni_ref = obter_ipca_ref(df_ipca, prev_adj)
    ipca_pro_rata = ni_ref * (1 + ipca_previsto/100) ** (du_desde/du_entre)
    return ipca_pro_rata * reais_por_ponto

# ==========================
# 4) Base longa (append dedup)
# ==========================

def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        return pd.read_parquet(p)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                            chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb

# ==========================
# 5) Wides (preço e valor) — leitura/escrita
# ==========================

def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")
    df = pd.read_parquet(path)
    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"
    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass
    df = df[~df.index.duplicated(keep="last")]
    return df

def salvar_wide(df: pd.DataFrame, path_parquet: str, path_csv: str, csv_ptbr_text: bool = True):
    """
    Salva o wide em Parquet e CSV. Se csv_ptbr_text=True, todas as células numéricas
    do CSV são convertidas para TEXTO pt-BR (vírgula decimal).
    """
    df2 = df.copy()
    df2.index.name = "Assets"
    out = df2.reset_index().rename(columns={df2.reset_index().columns[0]:"Assets"})
    out.to_parquet(path_parquet, index=False)

    if csv_ptbr_text:
        out_txt = out.copy()
        # normaliza nomes das colunas de data
        cols_norm = []
        for c in out_txt.columns:
            if c == "Assets":
                cols_norm.append(c)
                continue
            try:
                cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
            except Exception:
                cols_norm.append(str(c))
        out_txt.columns = cols_norm

        for c in out_txt.columns:
            if c == "Assets":
                continue
            out_txt[c] = out_txt[c].map(_fmt_ptbr_2dec)
        out_txt.to_csv(path_csv, index=False, encoding="utf-8")
    else:
        out.to_csv(path_csv, index=False, encoding="utf-8")

def mapear_asset(name: str, venc: str) -> str | None:
    suf = (venc or "").strip()[-2:]
    if name == "IPCACoupon":     return f"DAP{suf}"
    if name == "DI1Day":         return f"DI_{suf}"
    if name in ("BusinessDollar","WDOMiniFuture"): return "WDO1"
    if name == "USTNOTEFuture":  return "TREASURY"
    return None

def _valor_por_ponto_dap(df_ipca: pd.DataFrame, data_ref: dt.date, df_long_dia: pd.DataFrame) -> float:
    """
    Calcula o valor por ponto (R$) para DAP:
    - 1º prioridade: 'ValorIndiceDia' do arquivo da B3 (linha "Valor Índice Ipca pro Rata Tempore").
      ValorPorPonto = 0.00025 * ValorIndiceDia
    - Fallback: cálculo via IPEA / projeção (método anterior).
    """
    vi = None
    try:
        vi = df_long_dia.loc[df_long_dia["Name"]=="IPCACoupon","ValorIndiceDia"].dropna().iloc[0]
    except Exception:
        vi = None
    if vi is not None:
        vpp = REAIS_POR_PONTO * float(vi)
        print(f"[DAP] Valor por ponto via arquivo B3: Índice={vi:.4f} -> R$ {vpp:.6f}")
        return vpp
    # fallback
    vpp = calcular_valor_ponto_dap_para_data(df_ipca, data_ref)
    print(f"[DAP] Valor por ponto via fallback IPEA: R$ {vpp:.6f}")
    return vpp

def construir_colunas_wide_duplas(
    df_long_dia: pd.DataFrame,
    data_ref: dt.date,
    df_ipca: pd.DataFrame
) -> tuple[pd.Series, pd.Series]:
    """
    Retorna:
      s_preco -> série por Asset com PrecoAjusteAtual
      s_valor -> série por Asset com Valor de Ajuste (DAP em R$; demais = pontos)
    """
    df = df_long_dia.copy()
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()]

    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()
    s_valor = df.groupby("Asset")["Pontos"].first()

    # Converter DAP (Pontos -> R$) usando o valor por ponto do próprio arquivo (ou fallback)
    if not df[df["Name"] == "IPCACoupon"].empty:
        valor_ponto_dap = _valor_por_ponto_dap(df_ipca, data_ref, df_long_dia)
        for k in list(s_valor.index):
            if str(k).startswith("DAP") and pd.notna(s_valor.loc[k]):
                try:
                    s_valor.loc[k] = float(s_valor.loc[k]) * valor_ponto_dap
                except Exception:
                    pass

    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name
    return s_preco, s_valor

# ==========================
# 6) Calendário B3 + backoff
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")

def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]

def ultimo_dia_util_antes_hoje() -> dt.date:
    today = dt.date.today()
    val = b3_calendar().valid_days(today - dt.timedelta(days=15), today)
    return val[-1].date()

def buscar_um_dia(name: str, d: dt.date) -> pd.DataFrame | None:
    csv_text = baixar_csv_bdi(name, d)
    # Preferir parser especial para IPCACoupon
    if name == "IPCACoupon":
        try:
            return parse_ipcacoupon_special(csv_text, d, name)
        except Exception as e_special:
            _append_log(f"[warn] IPCACoupon parser especial falhou ({e_special}); tentando genérico…")
    return parse_ajustes(csv_text, d, name)

def buscar_dia_com_backoff(target_d: dt.date, df_ipca: pd.DataFrame) -> pd.DataFrame:
    dfs = []
    validos_back = b3_valid_days(target_d - dt.timedelta(days=60), target_d)[::-1]
    tentativas_max = max(BACKOFF_LIM, 15)

    for name in NAMES:
        ok = False
        tentativa = 0
        for prev_d in [target_d] + validos_back:
            try:
                csv_text = baixar_csv_bdi(name, prev_d)
                # Tenta parser especial se IPCACoupon; senão, parser genérico
                if name == "IPCACoupon":
                    try:
                        df_n = parse_ipcacoupon_special(csv_text, prev_d, name)
                    except Exception as e_special:
                        _append_log(f"[warn] IPCACoupon parser especial falhou ({e_special}); usando genérico.")
                        df_n = parse_ajustes(csv_text, prev_d, name)
                else:
                    df_n = parse_ajustes(csv_text, prev_d, name)

                if df_n is not None and not df_n.empty:
                    # Normaliza Data_Referencia para o target (backfill)
                    df_n["Data_Referencia"] = pd.to_datetime(target_d)
                    dfs.append(df_n)
                    ok = True
                    if prev_d != target_d:
                        _append_log(f"• {name}: {target_d} vazio → usando {prev_d} (backfill)")
                    break
            except Exception as e:
                msg = str(e)
                _append_log(f"! {name} @ {prev_d}: falhou parse ({msg[:120]})")
            tentativa += 1
            if tentativa > tentativas_max:
                break
        if not ok:
            _append_log(f"! {name}: sem dados até {tentativas_max} DUs atrás para {target_d}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

# ==========================
# 7) Pipeline (preço + valor) + duplicação 1x
# ==========================

def _dup_if_needed(wp: pd.DataFrame, wv: pd.DataFrame, end_dt: dt.date):
    """
    Regras:
    - Descobre o último dia (por colunas) de cada base.
    - Calcula o PRÓXIMO dia útil da B3.
    - Só duplica se (a) esse próximo DU for <= end_dt e (b) a coluna ainda não existir.
    - Nunca cria cadeia de duplicações (no máximo 1 coluna nova).
    """
    if wp is None or wv is None or wp.shape[1] < 1 or wv.shape[1] < 1:
        return wp, wv, False

    cols_p = sorted(pd.to_datetime(wp.columns))
    cols_v = sorted(pd.to_datetime(wv.columns))
    last_col_p = cols_p[-1].strftime("%Y-%m-%d")
    last_col_v = cols_v[-1].strftime("%Y-%m-%d")
    last_dt_p = cols_p[-1].date()
    last_dt_v = cols_v[-1].date()

    # Usa o máximo dos dois como "último dia efetivo"
    last_dt = max(last_dt_p, last_dt_v)

    prox = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
    if len(prox) < 2:
        _append_log("Não foi possível calcular próximo dia útil para duplicação.")
        return wp, wv, False

    prox_dt = prox[1]
    prox_col = prox_dt.strftime("%Y-%m-%d")

    # Só duplica se: (i) ainda não existir a coluna e (ii) o dia estiver dentro do horizonte alvo
    if prox_dt > end_dt:
        _append_log(f"Próximo DU {prox_col} é posterior ao end_dt -> não duplica.")
        return wp, wv, False

    already_p = prox_col in wp.columns
    already_v = prox_col in wv.columns
    if already_p and already_v:
        _append_log(f"Coluna {prox_col} já existe em ambas as bases -> não duplica.")
        return wp, wv, False

    # Duplica em cada base apenas se a coluna não existir nela
    if not already_p:
        wp[prox_col] = wp[last_col_p]
    if not already_v:
        wv[prox_col] = wv[last_col_v]

    _append_log(f"Duplicado 1x para o próximo dia útil: {prox_col}")
    return wp, wv, True

def main():
    # 1) carregar IPCA e bases existentes
    df_ipca = carregar_ipca_ipeadata()
    wide_preco = ler_wide(PATH_PRECO)
    wide_valor = ler_wide(PATH_VALOR)

    # 2) definir range de datas
    def _last_col_date(df):
        if df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)
    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)

    end_dt = ultimo_dia_util_antes_hoje()
    dias_util = b3_valid_days(start_dt, end_dt)

    _append_log(f"Atualizando de {start_dt} até {end_dt} (dias úteis B3: {len(dias_util)})")

    # 3) iterar dias úteis
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref, df_ipca)

        if df_dia_all.empty:
            # Sem dados do dia => NÃO atualiza planilhas (mantém como está)
            _append_log(f"{dref}: nenhum dado disponível — mantendo planilhas (sem atualização).")
            continue

        # Houve dados => atualiza base longa e wides
        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)
        _append_log(f"{dref}: base longa atualizada para {len(df_long)} linhas totais.")

        s_preco, s_valor = construir_colunas_wide_duplas(df_dia_all, dref, df_ipca)

        if wide_preco.empty:
            wide_preco = pd.DataFrame(s_preco)
        else:
            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_preco[s_preco.name] = s_preco

        if wide_valor.empty:
            wide_valor = pd.DataFrame(s_valor)
        else:
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_valor[s_valor.name] = s_valor

        _append_log(f"{dref}: preços/valores atualizados.")

    # 4) DUPLICAÇÃO — no máximo 1x, por data
    wide_preco, wide_valor, duplicou = _dup_if_needed(wide_preco, wide_valor, end_dt)

    # 5) ordenar colunas e salvar
    def _order(df):
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO, PATH_PRECO_CSV, csv_ptbr_text=True)
    salvar_wide(wide_valor, PATH_VALOR, PATH_VALOR_CSV, csv_ptbr_text=True)
    _append_log(f"Salvos: {PATH_PRECO} {wide_preco.shape} | {PATH_VALOR} {wide_valor.shape}")

    # 6) JSON pt-BR dos PREÇOS (texto)
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        _append_log(f"Salvo JSON pt-BR de preços (texto): {PATH_JSON}")
    except Exception as e:
        _append_log(f"[warn] Falha ao gerar JSON pt-BR de preços: {e}")

if __name__ == "__main__":
    main()

[2025-11-12 13:57:38] Atualizando de 2025-11-12 até 2025-11-12 (dias úteis B3: 1)
[2025-11-12 13:57:38] [HTTP] IPCACoupon 2025-11-12 -> status=200 content-length=352
    [dump] CSV bruto salvo em: debug_b3_csv\IPCACoupon_2025-11-12.csv
        md5=b0b9e4eddc299be1fb61ab4ec4ff6587 bytes=507
----[ RAW IPCACoupon 2025-11-12 | primeiras 9 de 9 linhas ]----
﻿Mercado Futuro

Vencimento;Contratos em Aberto;Negócios Realizados;Contratos Negociados;Volume;Ajuste Anterior;Preço de Abertura;Preço Mínimo;Preço Máximo;Preço Médio;Último Preço;Ajuste;Variação em Pontos;Última Oferta de Compra;Última Oferta de Venda
Nenhum resultado

* Preços de Ajustes Corrigidos Referentes a 13/11/2025
Taxa DI Over para 12/11/2025: -%
Valor Índice Ipca pro Rata Tempore: 7.364,82
A taxa e os preços são apenas prévias, não devendo ser considerados definitivos.
---------------------------------------------------------------
[2025-11-12 13:57:38] [warn] IPCACoupon parser especial falhou ('Vencimento'); usando genérico.

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_22356\1286074264.py:114: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_str = df.applymap(_fmt_ptbr_2dec)


In [19]:
# -*- coding: utf-8 -*-
"""
Pipeline B3 (BDI) — IPCACoupon/DI/Dólar/WDO/Treasury
- Parsing robusto (inclui parser especial p/ IPCACoupon).
- Bases longas e wide (Preço de Ajuste Atual e Valor de Ajuste em R$ p/ DAP).
- Atualiza de onde parou até o último dia útil **anterior a hoje**.
- Se não houver dados do dia, NÃO atualiza; ao final pode duplicar 1x (dentro de end_dt).
- Exporta JSON/CSV com números em TEXTO pt-BR (vírgula decimal), sem “escapar” número.
"""

import datetime as dt
import io, re, os, json, unicodedata, hashlib, csv
from copy import deepcopy
from pathlib import Path

import pandas as pd
import requests
import pandas_market_calendars as mcal

# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO   = "PRECOS12_IPCA12"
IPCA_PREVISTO       = 0.15        # % ANBIMA do mês (fallback)
REAIS_POR_PONTO     = 0.00025     # R$ por ponto (coeficiente do contrato)
BACKOFF_LIM         = 15          # janela de backoff (dias ÚTEIS)

# Debug / inspeção
DEBUG_MAX_LINES = 40
DEBUG_DUMP_DIR  = "debug_b3_csv"  # use None para desativar

# arquivos (bases)
PATH_LONG       = "df_ajustes_b3.parquet"
PATH_PRECO      = "df_preco_de_ajuste_atual_completo.parquet"
PATH_VALOR      = "df_valor_ajuste_contrato.parquet"
PATH_JSON       = "df_preco_de_ajuste_atual_completo.json"   # JSON pt-BR (texto)
PATH_PRECO_CSV  = "df_preco_de_ajuste_atual_completo.csv"    # CSV pt-BR (texto)
PATH_VALOR_CSV  = "df_valor_ajuste_contrato.csv"             # CSV pt-BR (texto)
PATH_RUN_LOG    = "atualizacao_b3_log.txt"

# ==========================
# 1) Download do CSV na B3
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}
PAYLOAD_BASE = {"Name": "IPCACoupon","Date":"2025-11-10","FinalDate":"2025-11-10","ClientId":"","Filters":{}}

NAMES = ["IPCACoupon","DI1Day","BusinessDollar","WDOMiniFuture","USTNOTEFuture"]

# ==========================
# Helpers gerais + JSON pt-BR
# ==========================

try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None

def _append_log(msg: str):
    ts = dt.datetime.now(_TZ).strftime("%Y-%m-%d %H:%M:%S") if _TZ else dt.datetime.now().isoformat(sep=" ", timespec="seconds")
    line = f"[{ts}] {msg}"
    print(line)
    try:
        with open(PATH_RUN_LOG, "a", encoding="utf-8") as f:
            f.write(line + "\n")
    except Exception:
        pass

def _fmt_ptbr_2dec(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return ""
    try:
        v = float(x)
    except Exception:
        return str(x)
    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    """
    Exporta wide como JSON com TODAS as células em string pt-BR.
    Montagem manual para evitar re-inferência de tipos pelo pandas.to_json.
    """
    if wide_df is None or wide_df.empty:
        return "[]"

    # normaliza colunas -> 'YYYY-MM-DD'
    cols_norm = []
    for c in wide_df.columns:
        try:
            cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols_norm.append(str(c))

    df = wide_df.copy()
    df.columns = cols_norm
    df.index.name = "Assets"

    # Converte tudo para string pt-BR
    df_txt = df.applymap(_fmt_ptbr_2dec)

    # Monta registros manualmente (garante tipo string)
    records = []
    for asset, row in df_txt.iterrows():
        rec = {"Assets": str(asset)}
        for col in df_txt.columns:
            val = row[col]
            rec[str(col)] = "" if val is None else str(val)
        records.append(rec)

    return json.dumps(records, ensure_ascii=False)

# ==========================
# 2) Parsing — genérico + especial IPCACoupon
# ==========================

def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Name"], p["Date"], p["FinalDate"] = name, s, s
    return p

def _ensure_debug_dir():
    if DEBUG_DUMP_DIR:
        Path(DEBUG_DUMP_DIR).mkdir(parents=True, exist_ok=True)

def _dump_csv(name: str, data: dt.date, raw_bytes: bytes):
    if not DEBUG_DUMP_DIR:
        return
    _ensure_debug_dir()
    fn = Path(DEBUG_DUMP_DIR) / f"{name}_{data.strftime('%Y-%m-%d')}.csv"
    try:
        fn.write_bytes(raw_bytes)
        print(f"    [dump] CSV bruto salvo em: {fn}")
    except Exception as e:
        print(f"    [dump:fail] {e}")

def _print_snippet(tag: str, text: str, max_lines: int = DEBUG_MAX_LINES):
    lines = text.splitlines()
    header = f"----[ {tag} | primeiras {min(len(lines), max_lines)} de {len(lines)} linhas ]----"
    print(header)
    for ln in lines[:max_lines]:
        print(ln)
    print("-" * len(header))

def baixar_csv_bdi(name: str, data: dt.date) -> str:
    r = requests.post(URL, headers=HEADERS, json=montar_payload(name, data))
    clen = r.headers.get("Content-Length", "?")
    _append_log(f"[HTTP] {name} {data} -> status={r.status_code} content-length={clen}")
    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    _dump_csv(name, data, r.content)
    md5 = hashlib.md5(r.content).hexdigest()
    print(f"        md5={md5} bytes={len(r.content)}")

    for enc in ("utf-8", "latin-1"):
        try:
            txt = r.content.decode(enc)
            break
        except UnicodeDecodeError:
            txt = None
    if txt is None:
        raise RuntimeError(f"{data} / {name}: falha ao decodificar")

    _print_snippet(f"RAW {name} {data}", txt)
    return txt

def _strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')

def _looks_like_header(line: str) -> bool:
    l = _strip_accents(line).lower()
    return ("vencimento" in l) and (("preco" in l) or ("ajuste" in l)) and (("vari" in l) and ("ponto" in l))

def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()
    start = None
    for i, line in enumerate(lines):
        if _looks_like_header(line):
            start = i
            break
    if start is None:
        return csv_text
    block = [lines[start]]
    for line in lines[start+1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block.append(line)
    return "\n".join(block)

def ptbr_to_float(s):
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s in {"", "-"}:
        return None
    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def parse_ajustes(csv_text: str, data_ref: dt.date, name: str) -> pd.DataFrame:
    bloco = extrair_bloco_mercado_futuro(csv_text)
    target_text = bloco if ";" in bloco else csv_text

    if bloco is csv_text:
        print(f"[parse] {name} {data_ref}: cabeçalho NÃO localizado — tentando CSV inteiro")
    else:
        first = bloco.splitlines()[0] if bloco else "<vazio>"
        print(f"[parse] {name} {data_ref}: cabeçalho localizado -> {first}")

    _print_snippet(f"PARSER_TARGET {name} {data_ref}", target_text)

    try:
        df_raw = pd.read_csv(io.StringIO(target_text), sep=";", dtype=str, engine="python", on_bad_lines="skip")
    except Exception:
        _print_snippet(f"PARSER_FAIL_{name}_{data_ref}", target_text)
        raise

    if df_raw.empty or df_raw.shape[1] < 3:
        _print_snippet(f"PARSER_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("CSV sem estrutura reconhecível para 'Ajustes do Pregão'.")

    colmap = {c: _strip_accents(c).lower() for c in df_raw.columns}
    print(f"[parse] colunas originais: {list(df_raw.columns)}")
    print(f"[parse] colunas normalizadas: {list(colmap.values())}")

    def _find_col(*must_have):
        for orig, norm in colmap.items():
            if all(x in norm for x in must_have):
                return orig
        return None

    c_venc  = _find_col("vencimento")
    c_var   = _find_col("vari", "ponto")
    c_preco = _find_col("ajuste") or _find_col("preco", "ajuste", "atual")

    print(f"[parse] mapeadas -> Venc:{c_venc}  VarPts:{c_var}  Preco:{c_preco}")

    if c_venc is None or c_var is None or c_preco is None:
        _print_snippet(f"PARSER_HDR_MISSING_{name}_{data_ref}", "\n".join(df_raw.columns.astype(str)))
        raise ValueError(f"Colunas não encontradas. Cabeçalhos vistos: {list(df_raw.columns)}")

    df = df_raw[[c_venc, c_var, c_preco]].copy()
    df.columns = ["Vencimento", "Pontos", "PrecoAjusteAtual"]
    df["Pontos"] = df["Pontos"].apply(ptbr_to_float)
    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].apply(ptbr_to_float)

    print("[parse] amostra parseada (até 10 linhas):")
    print(df.head(10).to_string(index=False))

    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        _print_snippet(f"PARSER_CLEAN_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("Após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df.reset_index(drop=True, inplace=True)
    return df

# ---------- PARSER ESPECIAL IPCACoupon ----------
_PT_BR_NUM = re.compile(r'^-?\d{1,3}(\.\d{3})*(,\d+)?$')

def _ipc_to_float_ptbr(s: str):
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s in {"-", "–", "—"}:
        return None
    s = s.replace("↑", "").replace("↓", "")
    if _PT_BR_NUM.match(s):
        s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None

def _ipc_clean_text(t: str) -> str:
    return (t or "").replace("\ufeff", "").replace("\xa0", " ").replace("\r\n", "\n").replace("\r", "\n")

def _ipc_slice_table_block(text: str) -> tuple[str, str, float | None]:
    txt = _ipc_clean_text(text)
    m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação em Pontos;.*$', txt)
    if not m_head:
        m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação.*$', txt)
    if not m_head:
        raise ValueError("Cabeçalho 'Vencimento;' não encontrado no CSV da B3 (IPCACoupon).")
    start = m_head.start()
    m_end = re.search(r'(?m)^\* Preços.*$', txt)
    end = m_end.start() if m_end else len(txt)
    bloco = txt[start:end].strip()
    m_resumo = re.search(r'(?mi)^(?:[FGHJKMNQUVXZ]\d{2}=\d{1,3}\.\d{3},\d{2}\s*)+', txt)
    linha_resumo = m_resumo.group(0).strip() if m_resumo else ""
    m_ind = re.search(r'Valor\s+Índice\s+Ipca\s+pro\s+Rata\s+Tempore:\s*([0-9\.\,]+)', txt, re.IGNORECASE)
    valor_indice_ipca = _ipc_to_float_ptbr(m_ind.group(1)) if m_ind else None
    return bloco, linha_resumo, valor_indice_ipca

def parse_ipcacoupon_special(csv_text: str, data_ref: dt.date, name: str = "IPCACoupon") -> pd.DataFrame:
    bloco, linha_resumo, valor_indice_ipca = _ipc_slice_table_block(csv_text)
    linhas = [ln for ln in bloco.split("\n") if ln.strip()]
    reader = csv.reader(linhas, delimiter=';')
    header = next(reader)
    cols = [c.strip() for c in header]
    try:
        idx_venc    = cols.index("Vencimento")
        idx_ult     = cols.index("Último Preço")
        idx_aj      = cols.index("Ajuste")
        idx_var = None
        for i, c in enumerate(cols):
            cc = c.replace(" ", "").lower()
            if ("vari" in cc) and ("ponto" in cc):
                idx_var = i
                break
    except ValueError as e:
        raise ValueError(f"Colunas esperadas (IPCACoupon) não encontradas no header {cols!r}: {e}")

    rows = []
    cod_venc_pat = re.compile(r'^[FGHJKMNQUVXZ]\d{2}$')
    for r in reader:
        if not r or len(r) <= idx_aj:
            continue
        venc = r[idx_venc].strip()
        if not cod_venc_pat.match(venc):
            continue
        ultimo_preco = _ipc_to_float_ptbr(r[idx_ult])
        ajuste = _ipc_to_float_ptbr(r[idx_aj])
        pontos = _ipc_to_float_ptbr(r[idx_var]) if idx_var is not None and idx_var < len(r) else None
        rows.append({
            "Vencimento": venc,
            "Pontos": pontos,
            "PrecoAjusteAtual": ajuste,
            "UltimoPreco": ultimo_preco,
        })

    df = pd.DataFrame(rows).sort_values("Vencimento", ignore_index=True)

    if linha_resumo:
        ajustes_resumo = {}
        for token in linha_resumo.split():
            if "=" in token:
                k, v = token.split("=", 1)
                k, v = k.strip(), v.strip().rstrip(";")
                fv = _ipc_to_float_ptbr(v)
                if cod_venc_pat.match(k) and fv is not None:
                    ajustes_resumo[k] = fv
        if not df.empty and ajustes_resumo:
            df["PrecoAjusteAtual"] = df.apply(
                lambda x: x["PrecoAjusteAtual"] if pd.notnull(x["PrecoAjusteAtual"]) else ajustes_resumo.get(x["Vencimento"]),
                axis=1
            )

    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        raise ValueError("IPCACoupon: após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df["ValorIndiceDia"] = valor_indice_ipca
    df.reset_index(drop=True, inplace=True)
    return df[["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"]]

# ==========================
# 3) IPCA helpers (IPEA)
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{IPCA_SERIE_CODIGO}')"
    resp = requests.get(url); resp.raise_for_status()
    df = pd.DataFrame(resp.json()["value"])
    df["VALDATA"] = pd.to_datetime(df["VALDATA"].astype(str).str[:10], errors="coerce")
    df = df[["VALDATA","VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)
    return df.rename(columns={"VALDATA":"Data","VALVALOR":"IPCA_Indice"})

def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])

def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:
        d += dt.timedelta(days=1)
    return d

def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev = dt.date(data_ref.year, data_ref.month, 15)
        nxt_m = 1 if data_ref.month == 12 else data_ref.month + 1
        nxt_y = data_ref.year + 1 if data_ref.month == 12 else data_ref.year
        nxt = dt.date(nxt_y, nxt_m, 15)
    else:
        pm = 12 if data_ref.month == 1 else data_ref.month - 1
        py = data_ref.year - 1 if data_ref.month == 1 else data_ref.year
        prev = dt.date(py, pm, 15)
        nxt  = dt.date(data_ref.year, data_ref.month, 15)
    return prev, nxt

def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float = IPCA_PREVISTO,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    data_ref = pd.to_datetime(data_ref).date()
    prev_15, next_15 = datas_ipca_referencia(data_ref)
    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)
    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")
    ni_ref = obter_ipca_ref(df_ipca, prev_adj)
    ipca_pro_rata = ni_ref * (1 + ipca_previsto/100) ** (du_desde/du_entre)
    return ipca_pro_rata * reais_por_ponto

# ==========================
# 4) Base longa
# ==========================

def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        return pd.read_parquet(p)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                            chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb

# ==========================
# 5) Wides (preço e valor)
# ==========================

def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")
    df = pd.read_parquet(path)
    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"
    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass
    df = df[~df.index.duplicated(keep="last")]
    return df

def salvar_wide(df: pd.DataFrame, path_parquet: str, path_csv: str, csv_ptbr_text: bool = True):
    df2 = df.copy()
    df2.index.name = "Assets"
    out = df2.reset_index().rename(columns={df2.reset_index().columns[0]:"Assets"})
    out.to_parquet(path_parquet, index=False)

    if csv_ptbr_text:
        out_txt = out.copy()
        # normaliza nomes das colunas de data
        cols_norm = []
        for c in out_txt.columns:
            if c == "Assets":
                cols_norm.append(c)
            else:
                try:
                    cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
                except Exception:
                    cols_norm.append(str(c))
        out_txt.columns = cols_norm

        for c in out_txt.columns:
            if c == "Assets":
                continue
            out_txt[c] = out_txt[c].map(_fmt_ptbr_2dec)
        out_txt.to_csv(path_csv, index=False, encoding="utf-8")
    else:
        out.to_csv(path_csv, index=False, encoding="utf-8")

def mapear_asset(name: str, venc: str) -> str | None:
    suf = (venc or "").strip()[-2:]
    if name == "IPCACoupon":     return f"DAP{suf}"
    if name == "DI1Day":         return f"DI_{suf}"
    if name in ("BusinessDollar","WDOMiniFuture"): return "WDO1"
    if name == "USTNOTEFuture":  return "TREASURY"
    return None

def _valor_por_ponto_dap(df_ipca: pd.DataFrame, data_ref: dt.date, df_long_dia: pd.DataFrame) -> float:
    vi = None
    try:
        vi = df_long_dia.loc[df_long_dia["Name"]=="IPCACoupon","ValorIndiceDia"].dropna().iloc[0]
    except Exception:
        vi = None
    if vi is not None:
        vpp = REAIS_POR_PONTO * float(vi)
        print(f"[DAP] Valor por ponto via arquivo B3: Índice={vi:.4f} -> R$ {vpp:.6f}")
        return vpp
    vpp = calcular_valor_ponto_dap_para_data(df_ipca, data_ref)
    print(f"[DAP] Valor por ponto via fallback IPEA: R$ {vpp:.6f}")
    return vpp

def construir_colunas_wide_duplas(
    df_long_dia: pd.DataFrame,
    data_ref: dt.date,
    df_ipca: pd.DataFrame
) -> tuple[pd.Series, pd.Series]:
    df = df_long_dia.copy()
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()]

    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()
    s_valor = df.groupby("Asset")["Pontos"].first()

    if not df[df["Name"] == "IPCACoupon"].empty:
        valor_ponto_dap = _valor_por_ponto_dap(df_ipca, data_ref, df_long_dia)
        for k in list(s_valor.index):
            if str(k).startswith("DAP") and pd.notna(s_valor.loc[k]):
                try:
                    s_valor.loc[k] = float(s_valor.loc[k]) * valor_ponto_dap
                except Exception:
                    pass

    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name
    return s_preco, s_valor

# ==========================
# 6) Calendário B3 + backoff
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")

def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]

def ultimo_dia_util_ANTES_de_hoje() -> dt.date:
    """
    Retorna o último dia útil **estritamente anterior** a hoje.
    Isso evita incluir o dia corrente (mesmo se hoje for DU).
    """
    today = dt.date.today()
    # janela de busca: últimos ~20 dias corridos
    v = b3_calendar().valid_days(today - dt.timedelta(days=20), today - dt.timedelta(days=1))
    return v[-1].date()

def buscar_dia_com_backoff(target_d: dt.date, df_ipca: pd.DataFrame) -> pd.DataFrame:
    dfs = []
    validos_back = b3_valid_days(target_d - dt.timedelta(days=60), target_d)[::-1]
    tentativas_max = max(BACKOFF_LIM, 15)

    for name in NAMES:
        ok = False
        tentativa = 0
        for prev_d in [target_d] + validos_back:
            try:
                csv_text = baixar_csv_bdi(name, prev_d)
                if name == "IPCACoupon":
                    try:
                        df_n = parse_ipcacoupon_special(csv_text, prev_d, name)
                    except Exception as e_special:
                        _append_log(f"[warn] IPCACoupon parser especial falhou ({e_special}); usando genérico.")
                        df_n = parse_ajustes(csv_text, prev_d, name)
                else:
                    df_n = parse_ajustes(csv_text, prev_d, name)

                if df_n is not None and not df_n.empty:
                    df_n["Data_Referencia"] = pd.to_datetime(target_d)
                    dfs.append(df_n)
                    ok = True
                    if prev_d != target_d:
                        _append_log(f"• {name}: {target_d} vazio → usando {prev_d} (backfill)")
                    break
            except Exception as e:
                msg = str(e)
                _append_log(f"! {name} @ {prev_d}: falhou parse ({msg[:120]})")
            tentativa += 1
            if tentativa > tentativas_max:
                break
        if not ok:
            _append_log(f"! {name}: sem dados até {tentativas_max} DUs atrás para {target_d}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

# ==========================
# 7) Pipeline (preço + valor) + duplicação 1x
# ==========================

def _dup_if_needed(wp: pd.DataFrame, wv: pd.DataFrame, end_dt: dt.date):
    """
    Duplica **no máximo 1x** para o próximo DU, somente se:
      - o próximo DU <= end_dt
      - a coluna ainda não existir
    """
    if wp is None or wv is None or wp.shape[1] < 1 or wv.shape[1] < 1:
        return wp, wv, False

    cols_p = sorted(pd.to_datetime(wp.columns))
    cols_v = sorted(pd.to_datetime(wv.columns))
    last_col_p = cols_p[-1].strftime("%Y-%m-%d")
    last_col_v = cols_v[-1].strftime("%Y-%m-%d")
    last_dt = max(cols_p[-1].date(), cols_v[-1].date())

    prox = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
    if len(prox) < 2:
        _append_log("Não foi possível calcular próximo dia útil para duplicação.")
        return wp, wv, False

    prox_dt = prox[1]
    prox_col = prox_dt.strftime("%Y-%m-%d")

    if prox_dt > end_dt:
        _append_log(f"Próximo DU {prox_col} > end_dt {end_dt} → não duplica.")
        return wp, wv, False

    already_p = prox_col in wp.columns
    already_v = prox_col in wv.columns
    if already_p and already_v:
        _append_log(f"Coluna {prox_col} já existe → não duplica.")
        return wp, wv, False

    if not already_p:
        wp[prox_col] = wp[last_col_p]
    if not already_v:
        wv[prox_col] = wv[last_col_v]

    _append_log(f"Duplicado 1x para o próximo dia útil: {prox_col}")
    return wp, wv, True

def main():
    # 1) carregar IPCA e bases existentes
    df_ipca = carregar_ipca_ipeadata()
    wide_preco = ler_wide(PATH_PRECO)
    wide_valor = ler_wide(PATH_VALOR)

    # 2) definir range de datas
    def _last_col_date(df):
        if df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)
    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)

    # >>>>>> ALTERAÇÃO: “último dia útil ANTES de hoje”
    end_dt = ultimo_dia_util_ANTES_de_hoje()

    dias_util = b3_valid_days(start_dt, end_dt)
    _append_log(f"Atualizando de {start_dt} até {end_dt} (DU B3: {len(dias_util)})")

    # 3) iterar dias úteis
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref, df_ipca)

        if df_dia_all.empty:
            _append_log(f"{dref}: nenhum dado disponível — mantendo planilhas (sem atualização).")
            continue

        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)
        _append_log(f"{dref}: base longa atualizada; total linhas = {len(df_long)}.")

        s_preco, s_valor = construir_colunas_wide_duplas(df_dia_all, dref, df_ipca)

        if wide_preco.empty:
            wide_preco = pd.DataFrame(s_preco)
        else:
            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_preco[s_preco.name] = s_preco

        if wide_valor.empty:
            wide_valor = pd.DataFrame(s_valor)
        else:
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_valor[s_valor.name] = s_valor

        _append_log(f"{dref}: preços/valores atualizados.")

    # 4) DUPLICAÇÃO — no máximo 1x, respeitando end_dt (ontem)
    wide_preco, wide_valor, _ = _dup_if_needed(wide_preco, wide_valor, end_dt)

    # 5) ordenar colunas e salvar
    def _order(df):
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO, PATH_PRECO_CSV, csv_ptbr_text=True)
    salvar_wide(wide_valor, PATH_VALOR, PATH_VALOR_CSV, csv_ptbr_text=True)
    _append_log(f"Salvos: {PATH_PRECO} {wide_preco.shape} | {PATH_VALOR} {wide_valor.shape}")

    # 6) JSON pt-BR (texto garantido)
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        _append_log(f"Salvo JSON pt-BR de preços (texto): {PATH_JSON}")
    except Exception as e:
        _append_log(f"[warn] Falha ao gerar JSON pt-BR: {e}")

if __name__ == "__main__":
    main()


[2025-11-12 14:24:26] Atualizando de 2025-11-12 até 2025-11-11 (DU B3: 0)
[2025-11-12 14:24:26] Próximo DU 2025-11-13 > end_dt 2025-11-11 → não duplica.
[2025-11-12 14:24:28] Salvos: df_preco_de_ajuste_atual_completo.parquet (43, 219) | df_valor_ajuste_contrato.parquet (32, 219)
[2025-11-12 14:24:28] Salvo JSON pt-BR de preços (texto): df_preco_de_ajuste_atual_completo.json


C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_22356\73072723.py:108: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_txt = df.applymap(_fmt_ptbr_2dec)


In [ ]:
# -*- coding: utf-8 -*-
"""
Pipeline B3 (BDI) — IPCACoupon/DI/Dólar/WDO/Treasury
- Parsing robusto (inclui parser especial p/ IPCACoupon).
- Bases longas e wide (Preço de Ajuste Atual e Valor de Ajuste em R$ p/ DAP).
- Atualiza de onde parou até o último dia útil **anterior a hoje**.
- Tenta "hoje" à parte: se houver dados para hoje, adiciona; se não houver, DUPLICA o último dia útil.
- Exporta JSON/CSV com números em TEXTO pt-BR (vírgula decimal), sem “escapar” número.
- HTTP com timeout+retries e coleta paralela para “hoje”.
- Remove do output final os assets: DI_25, DAP_25 e DAP25.
- Substitui strings vazias "" por missing (pd.NA) no processamento e no parquet.
"""

import datetime as dt
import io, re, os, json, unicodedata, hashlib, csv, time
from copy import deepcopy
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests
import pandas_market_calendars as mcal
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO   = "PRECOS12_IPCA12"
REAIS_POR_PONTO     = 0.00025     # R$ por ponto (coeficiente do contrato)
BACKOFF_LIM         = 15          # janela de backoff (dias ÚTEIS)

IPCA_PREVISTA_XLSX  = "TaxasInflacaoDiariaPrevisao.xlsx"

# Debug / inspeção
DEBUG_MAX_LINES = 40
DEBUG_DUMP_DIR  = "debug_b3_csv"  # use None para desativar

# arquivos (bases)
PATH_LONG       = "df_ajustes_b3.parquet"
PATH_PRECO      = "df_preco_de_ajuste_atual_completo.parquet"
PATH_VALOR      = "df_valor_ajuste_contrato.parquet"
PATH_JSON       = "df_preco_de_ajuste_atual_completo.json"   # JSON pt-BR (texto)
PATH_PRECO_CSV  = "df_preco_de_ajuste_atual_completo.csv"    # CSV pt-BR (texto)
PATH_VALOR_CSV  = "df_valor_ajuste_contrato.csv"             # CSV pt-BR (texto)
PATH_RUN_LOG    = "atualizacao_b3_log.txt"

# Assets a EXCLUIR do output final (parquet/csv/json)
ASSETS_EXCLUIR = ["DI_25", "DAP_25", "DAP25", "DI25"]

# ==========================
# HTTP — sessão com retries/timeouts
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}
PAYLOAD_BASE = {"Name": "IPCACoupon","Date":"2025-11-10","FinalDate":"2025-11-10","ClientId":"","Filters":{}}

NAMES = ["IPCACoupon","DI1Day","BusinessDollar","WDOMiniFuture","USTNOTEFuture"]

HTTP_CONNECT_TIMEOUT = 3.0   # seg
HTTP_READ_TIMEOUT    = 15.0  # seg (por tentativa)
HTTP_TOTAL_BUDGET    = 90.0  # seg (orçamento total para o dia “exato”)

RETRY_CFG = Retry(
    total=2,                      # 2 tentativas extras (3 no total)
    backoff_factor=0.6,           # 0.6s, 1.2s...
    status_forcelist=(500, 502, 503, 504),
    allowed_methods=frozenset(["POST"])
)

_session = requests.Session()
_adapter = HTTPAdapter(max_retries=RETRY_CFG, pool_connections=10, pool_maxsize=10)
_session.mount("https://", _adapter)
_session.mount("http://", _adapter)

# ==========================
# Helpers gerais + JSON pt-BR
# ==========================

try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None


def _append_log(msg: str):
    ts = dt.datetime.now(_TZ).strftime("%Y-%m-%d %H:%M:%S") if _TZ else dt.datetime.now().isoformat(sep=" ", timespec="seconds")
    line = f"[{ts}] {msg}"
    print(line)
    try:
        with open(PATH_RUN_LOG, "a", encoding="utf-8") as f:
            f.write(line + "\n")
    except Exception:
        pass


def _normalize_missing_values_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Troca strings vazias/whitespace/"nan"/"none"/"null" por pd.NA (missing values).
    Útil para evitar ValueError: could not convert string to float: '' em downstream.
    """
    if df is None or df.empty:
        return df

    # empty/whitespace -> NA
    df2 = df.replace(r"^\s*$", pd.NA, regex=True)

    # tokens comuns -> NA
    df2 = df2.replace({"nan": pd.NA, "NaN": pd.NA, "none": pd.NA, "None": pd.NA, "null": pd.NA, "NULL": pd.NA})

    return df2


def remover_assets_indesejados(w: pd.DataFrame) -> pd.DataFrame:
    if w is None or w.empty:
        return w
    return w.drop(index=ASSETS_EXCLUIR, errors="ignore")


def _fmt_ptbr_2dec(x):
    """
    Converte número para string "pt-BR", ex:
    98252.84 -> "98.252,84"

    IMPORTANTE:
    - Missing (None/NaN/pd.NA/"") -> pd.NA (não "" no parquet).
    - Se já veio como string pt-BR ("98.252,84"), retorna como está.
    """
    if x is None:
        return pd.NA
    if pd.isna(x):
        return pd.NA
    if isinstance(x, str) and x.strip() == "":
        return pd.NA

    try:
        v = float(x)
    except Exception:
        # Se já veio como string pt-BR, devolve como está (desde que não seja vazia)
        s = str(x)
        return pd.NA if s.strip() == "" else s

    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")


def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    """
    Exporta wide como JSON com TODAS as células em string pt-BR.
    Missing (pd.NA/None/NaN) vira "" no JSON.
    """
    if wide_df is None or wide_df.empty:
        return "[]"

    # normaliza colunas -> 'YYYY-MM-DD'
    cols_norm = []
    for c in wide_df.columns:
        try:
            cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols_norm.append(str(c))

    df = wide_df.copy()
    df.columns = cols_norm
    df.index.name = "Assets"

    # Converte tudo para string pt-BR
    df_txt = df.copy()
    for c in df_txt.columns:
        df_txt[c] = df_txt[c].map(_fmt_ptbr_2dec)

    # Monta registros manualmente (garante tipo string)
    records = []
    for asset, row in df_txt.iterrows():
        rec = {"Assets": str(asset)}
        for col in df_txt.columns:
            val = row[col]
            rec[str(col)] = "" if (val is None or pd.isna(val)) else str(val)
        records.append(rec)

    return json.dumps(records, ensure_ascii=False)

# ==========================
#  IPCA Previsao Helpers
# ==========================

def carregar_ipca_prevista(path_xlsx: str = IPCA_PREVISTA_XLSX) -> pd.DataFrame:
    """
    Lê a planilha TaxasInflacaoDiariaPrevisao.xlsx (aba 'Teste')
    e devolve um DataFrame com colunas:
        - Data (date)
        - IPCA_Previsto (float, % ao mês – ex: 0.16 = 0,16%)
    Se o arquivo não existir, encerra o programa.
    """
    p = Path(path_xlsx)
    if not p.exists():
        msg = f"Arquivo de inflação prevista não encontrado: {path_xlsx}"
        _append_log(msg)
        raise SystemExit(msg)

    try:
        df = pd.read_excel(p, sheet_name="Teste")
    except Exception as e:
        msg = f"Falha ao ler a aba 'Teste' em {path_xlsx}: {e}"
        _append_log(msg)
        raise SystemExit(msg)

    # Esperado: colunas 'Datas' e 'Projecao'
    if "Datas" not in df.columns or "Projecao" not in df.columns:
        msg = f"Planilha {path_xlsx} (aba 'Teste') não possui colunas 'Datas' e 'Projecao'."
        _append_log(msg)
        raise SystemExit(msg)

    df = df[["Datas", "Projecao"]].copy()
    df.rename(columns={"Datas": "Data", "Projecao": "IPCA_Previsto"}, inplace=True)

    df["Data"] = pd.to_datetime(df["Data"], errors="coerce").dt.date
    df["IPCA_Previsto"] = pd.to_numeric(df["IPCA_Previsto"], errors="coerce")

    df = df.dropna(subset=["Data", "IPCA_Previsto"]).reset_index(drop=True)

    if df.empty:
        msg = f"Planilha {path_xlsx} não contém linhas válidas de Data/IPCA_Previsto."
        _append_log(msg)
        raise SystemExit(msg)

    _append_log(f"[IPCA_PREV] Carregado {len(df)} linhas de inflação prevista de {path_xlsx}")
    return df


def obter_ipca_prevista_para_data(df_prevista: pd.DataFrame, data_ref: dt.date) -> float:
    """
    Retorna o IPCA previsto (% ao mês) para a data_ref com base na planilha.
    Se não houver valor para a data, encerra o programa com mensagem.
    """
    data_ref = pd.to_datetime(data_ref).date()
    linha = df_prevista.loc[df_prevista["Data"] == data_ref]

    if linha.empty:
        msg = f"Não há inflação prevista na planilha para a data {data_ref}."
        _append_log(msg)
        raise SystemExit(msg)

    ipca_prev = float(linha.iloc[0]["IPCA_Previsto"])
    _append_log(f"[IPCA_PREV] {data_ref}: IPCA previsto = {ipca_prev:.4f}%")
    return ipca_prev

# ==========================
# 1) Download do CSV na B3
# ==========================

def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Name"], p["Date"], p["FinalDate"] = name, s, s
    return p


def _ensure_debug_dir():
    if DEBUG_DUMP_DIR:
        Path(DEBUG_DUMP_DIR).mkdir(parents=True, exist_ok=True)


def _dump_csv(name: str, data: dt.date, raw_bytes: bytes):
    if not DEBUG_DUMP_DIR:
        return
    _ensure_debug_dir()
    fn = Path(DEBUG_DUMP_DIR) / f"{name}_{data.strftime('%Y-%m-%d')}.csv"
    try:
        fn.write_bytes(raw_bytes)
        print(f"    [dump] CSV bruto salvo em: {fn}")
    except Exception as e:
        print(f"    [dump:fail] {e}")


def _print_snippet(tag: str, text: str, max_lines: int = DEBUG_MAX_LINES):
    lines = text.splitlines()
    header = f"----[ {tag} | primeiras {min(len(lines), max_lines)} de {len(lines)} linhas ]----"
    print(header)
    for ln in lines[:max_lines]:
        print(ln)
    print("-" * len(header))


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    start = time.monotonic()
    r = _session.post(
        URL,
        headers=HEADERS,
        json=montar_payload(name, data),
        timeout=(HTTP_CONNECT_TIMEOUT, HTTP_READ_TIMEOUT),
    )
    elapsed = time.monotonic() - start
    clen = r.headers.get("Content-Length", "?")
    _append_log(f"[HTTP] {name} {data} -> status={r.status_code} content-length={clen} t={elapsed:.2f}s")
    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    _dump_csv(name, data, r.content)
    md5 = hashlib.md5(r.content).hexdigest()
    _append_log(f"[HTTP] md5={md5} bytes={len(r.content)} ({name} {data})")

    for enc in ("utf-8", "latin-1"):
        try:
            txt = r.content.decode(enc)
            break
        except UnicodeDecodeError:
            txt = None
    if txt is None:
        raise RuntimeError(f"{data} / {name}: falha ao decodificar")

    _print_snippet(f"RAW {name} {data}", txt)
    return txt

# ==========================
# 2) Parsing — genérico + especial IPCACoupon
# ==========================

def _strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')


def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    lines = csv_text.splitlines()
    start = None
    for i, line in enumerate(lines):
        l = _strip_accents(line).lower()
        if ("vencimento" in l) and (("preco" in l) or ("ajuste" in l)) and (("vari" in l) and ("ponto" in l)):
            start = i
            break
    if start is None:
        return csv_text
    block = [lines[start]]
    for line in lines[start+1:]:
        if not line.strip():
            break
        if ";" not in line and "=" in line:
            break
        block.append(line)
    return "\n".join(block)


def ptbr_to_float(s):
    """
    Converte string pt-BR (com milhar '.' e decimal ',') para float.
    Retorna None (missing) para "", "-", None, NaN e strings que virem vazias após limpeza.
    """
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s in {"", "-"}:
        return None

    s = re.sub(r"[^0-9\-,\.]", "", s)  # remove lixo
    s = s.strip()
    if s in {"", "-"}:
        return None

    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(csv_text: str, data_ref: dt.date, name: str) -> pd.DataFrame:
    bloco = extrair_bloco_mercado_futuro(csv_text)
    target_text = bloco if ";" in bloco else csv_text

    if bloco is csv_text:
        _append_log(f"[parse] {name} {data_ref}: cabeçalho NÃO localizado — tentando CSV inteiro")
    else:
        first = bloco.splitlines()[0] if bloco else "<vazio>"
        _append_log(f"[parse] {name} {data_ref}: cabeçalho localizado -> {first}")

    _print_snippet(f"PARSER_TARGET {name} {data_ref}", target_text)

    df_raw = pd.read_csv(
        io.StringIO(target_text),
        sep=";",
        dtype=str,
        engine="python",
        on_bad_lines="skip"
    )
    if df_raw.empty or df_raw.shape[1] < 3:
        _print_snippet(f"PARSER_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("CSV sem estrutura reconhecível para 'Ajustes do Pregão'.")

    colmap = {c: _strip_accents(c).lower() for c in df_raw.columns}
    _append_log(f"[parse] colunas normalizadas: {list(colmap.values())}")

    def _find_col(*must_have):
        for orig, norm in colmap.items():
            if all(x in norm for x in must_have):
                return orig
        return None

    c_venc = _find_col("vencimento")
    c_var  = _find_col("vari", "ponto")

    # ---------- SELETOR ESPERTO PARA "PREÇO DE AJUSTE ATUAL" ----------
    def _find_preco_ajuste_col():
        for orig, norm in colmap.items():
            if "preco" in norm and "ajuste" in norm and ("atual" in norm or "corrigid" in norm):
                return orig
        for orig, norm in colmap.items():
            if "ajuste" in norm and "anterior" not in norm:
                return orig
        for orig, norm in colmap.items():
            if "preco" in norm and "ajuste" in norm:
                return orig
        return None

    c_preco = _find_preco_ajuste_col()

    _append_log(f"[parse] mapeadas -> Venc:{c_venc}  VarPts:{c_var}  Preco:{c_preco}")

    if c_venc is None or c_var is None or c_preco is None:
        _print_snippet(
            f"PARSER_HDR_MISSING_{name}_{data_ref}",
            "\n".join(df_raw.columns.astype(str))
        )
        raise ValueError(f"Colunas não encontradas. Cabeçalhos vistos: {list(df_raw.columns)}")

    df = df_raw[[c_venc, c_var, c_preco]].copy()
    df.columns = ["Vencimento", "Pontos", "PrecoAjusteAtual"]

    df["Pontos"] = df["Pontos"].apply(ptbr_to_float)
    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].apply(ptbr_to_float)

    _append_log(f"[parse] {name} {data_ref} — linhas válidas antes do filtro: {len(df)}")
    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    _append_log(f"[parse] {name} {data_ref} — linhas após filtro: {len(df)}")
    if df.empty:
        raise ValueError("Após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df.reset_index(drop=True, inplace=True)
    return df

# ---------- PARSER ESPECIAL IPCACoupon ----------
_PT_BR_NUM = re.compile(r'^-?\d{1,3}(\.\d{3})*(,\d+)?$')


def _ipc_to_float_ptbr(s: str):
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s in {"-", "–", "—"}:
        return None
    s = s.replace("↑", "").replace("↓", "").strip()
    if s == "":
        return None

    if _PT_BR_NUM.match(s):
        s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None


def _ipc_clean_text(t: str) -> str:
    return (t or "").replace("\ufeff", "").replace("\xa0", " ").replace("\r\n", "\n").replace("\r", "\n")


def _ipc_slice_table_block(text: str) -> tuple[str, str, float | None]:
    txt = _ipc_clean_text(text)
    m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação em Pontos;.*$', txt)
    if not m_head:
        m_head = re.search(r'(?mi)^Vencimento;.*;Último Preço;Ajuste;Variação.*$', txt)
    if not m_head:
        raise ValueError("Cabeçalho 'Vencimento;' não encontrado no CSV da B3 (IPCACoupon).")
    start = m_head.start()
    m_end = re.search(r'(?m)^\* Preços.*$', txt)
    end = m_end.start() if m_end else len(txt)
    bloco = txt[start:end].strip()
    m_resumo = re.search(r'(?mi)^(?:[FGHJKMNQUVXZ]\d{2}=\d{1,3}\.\d{3},\d{2}\s*)+', txt)
    linha_resumo = m_resumo.group(0).strip() if m_resumo else ""
    m_ind = re.search(r'Valor\s+Índice\s+Ipca\s+pro\s+Rata\s+Tempore:\s*([0-9\.\,]+)', txt, re.IGNORECASE)
    valor_indice_ipca = _ipc_to_float_ptbr(m_ind.group(1)) if m_ind else None
    return bloco, linha_resumo, valor_indice_ipca


def parse_ipcacoupon_special(csv_text: str, data_ref: dt.date, name: str = "IPCACoupon") -> pd.DataFrame:
    bloco, linha_resumo, valor_indice_ipca = _ipc_slice_table_block(csv_text)
    linhas = [ln for ln in bloco.split("\n") if ln.strip()]
    reader = csv.reader(linhas, delimiter=';')
    header = next(reader)
    cols = [c.strip() for c in header]
    try:
        idx_venc    = cols.index("Vencimento")
        idx_ult     = cols.index("Último Preço")
        idx_aj      = cols.index("Ajuste")
        idx_var = None
        for i, c in enumerate(cols):
            cc = c.replace(" ", "").lower()
            if ("vari" in cc) and ("ponto" in cc):
                idx_var = i
                break
    except ValueError as e:
        raise ValueError(f"Colunas esperadas (IPCACoupon) não encontradas no header {cols!r}: {e}")

    rows = []
    cod_venc_pat = re.compile(r'^[FGHJKMNQUVXZ]\d{2}$')
    for r in reader:
        if not r or len(r) <= idx_aj:
            continue
        venc = (r[idx_venc] or "").strip()
        if not cod_venc_pat.match(venc):
            continue
        ultimo_preco = _ipc_to_float_ptbr(r[idx_ult] if idx_ult < len(r) else None)
        ajuste = _ipc_to_float_ptbr(r[idx_aj] if idx_aj < len(r) else None)
        pontos = _ipc_to_float_ptbr(r[idx_var]) if idx_var is not None and idx_var < len(r) else None
        rows.append({
            "Vencimento": venc,
            "Pontos": pontos,
            "PrecoAjusteAtual": ajuste,
            "UltimoPreco": ultimo_preco,
        })

    df = pd.DataFrame(rows).sort_values("Vencimento", ignore_index=True)

    if linha_resumo:
        ajustes_resumo = {}
        for token in linha_resumo.split():
            if "=" in token:
                k, v = token.split("=", 1)
                k, v = k.strip(), v.strip().rstrip(";")
                fv = _ipc_to_float_ptbr(v)
                if cod_venc_pat.match(k) and fv is not None:
                    ajustes_resumo[k] = fv
        if not df.empty and ajustes_resumo:
            df["PrecoAjusteAtual"] = df.apply(
                lambda x: x["PrecoAjusteAtual"] if pd.notnull(x["PrecoAjusteAtual"]) else ajustes_resumo.get(x["Vencimento"]),
                axis=1
            )

    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        raise ValueError("IPCACoupon: após limpeza, não há linhas com Pontos ou Preço.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df["ValorIndiceDia"] = valor_indice_ipca
    df.reset_index(drop=True, inplace=True)
    return df[["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"]]

# ==========================
# 3) IPCA helpers (IPEA)
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{IPCA_SERIE_CODIGO}')"
    resp = requests.get(url, timeout=(HTTP_CONNECT_TIMEOUT, HTTP_READ_TIMEOUT))
    resp.raise_for_status()
    df = pd.DataFrame(resp.json()["value"])
    df["VALDATA"] = pd.to_datetime(df["VALDATA"].astype(str).str[:10], errors="coerce")
    df = df[["VALDATA","VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)
    return df.rename(columns={"VALDATA":"Data","VALVALOR":"IPCA_Indice"})


def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])


def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:
        d += dt.timedelta(days=1)
    return d


def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev = dt.date(data_ref.year, data_ref.month, 15)
        nxt_m = 1 if data_ref.month == 12 else data_ref.month + 1
        nxt_y = data_ref.year + 1 if data_ref.month == 12 else data_ref.year
        nxt = dt.date(nxt_y, nxt_m, 15)
    else:
        pm = 12 if data_ref.month == 1 else data_ref.month - 1
        py = data_ref.year - 1 if data_ref.month == 1 else data_ref.year
        prev = dt.date(py, pm, 15)
        nxt  = dt.date(data_ref.year, data_ref.month, 15)
    return prev, nxt


def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    data_ref = pd.to_datetime(data_ref).date()
    prev_15, next_15 = datas_ipca_referencia(data_ref)
    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)
    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")
    ni_ref = obter_ipca_ref(df_ipca, prev_adj)
    ipca_pro_rata = ni_ref * (1 + ipca_previsto/100) ** (du_desde/du_entre)
    return ipca_pro_rata * reais_por_ponto

# ==========================
# 4) Base longa
# ==========================

def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        return pd.read_parquet(p)
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])


def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                            chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb

# ==========================
# 5) Wides (preço e valor)
# ==========================

def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")

    df = pd.read_parquet(path)

    # Garante índice "Assets"
    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"

    # Normaliza ordem das colunas por data (nomes 'YYYY-MM-DD')
    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass

    df = df[~df.index.duplicated(keep="last")]

    # PONTO-CHAVE: SEMPRE que ler o wide, remove eventual coluna duplicada no final
    df = drop_tail_duplicate(df)

    # Troca "" por missing
    df = _normalize_missing_values_df(df)

    # Remove assets indesejados já na leitura (se existirem em arquivos antigos)
    df = remover_assets_indesejados(df)

    return df


def adicionar_coluna_duplicada_final(
    wp: pd.DataFrame,
    wv: pd.DataFrame,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    No export final: garante que exista UMA coluna futura (próximo dia útil B3)
    como cópia da última coluna 'real'.
    """
    col_names = []
    if wp is not None and wp.shape[1] > 0:
        col_names.extend(list(wp.columns))
    if wv is not None and wv.shape[1] > 0:
        col_names.extend(list(wv.columns))

    if not col_names:
        _append_log("[dup-final] wp/wv vazios → nada para duplicar.")
        return wp, wv

    try:
        cols_dt = sorted(pd.to_datetime(col_names))
    except Exception:
        _append_log("[dup-final] Não consegui interpretar colunas como datas → não duplica.")
        return wp, wv

    last_dt = cols_dt[-1].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > last_dt]
    if not prox_list:
        _append_log(f"[dup-final] Não há próximo dia útil após {last_dt} → não duplica.")
        return wp, wv

    prox_dt = prox_list[0]
    prox_col = prox_dt.strftime("%Y-%m-%d")

    already_p = wp is not None and wp.shape[1] > 0 and prox_col in wp.columns
    already_v = wv is not None and wv.shape[1] > 0 and prox_col in wv.columns

    if already_p and already_v:
        _append_log(f"[dup-final] Coluna {prox_col} já existe em preços e valores → nenhuma duplicação feita.")
        return wp, wv

    if wp is not None and wp.shape[1] > 0 and not already_p and last_col in wp.columns:
        wp[prox_col] = wp[last_col]
        _append_log(f"[dup-final] (preço) Duplicado {last_col} -> {prox_col}")

    if wv is not None and wv.shape[1] > 0 and not already_v and last_col in wv.columns:
        wv[prox_col] = wv[last_col]
        _append_log(f"[dup-final] (valor) Duplicado {last_col} -> {prox_col}")

    return wp, wv


def salvar_wide(df: pd.DataFrame, path_parquet: str, path_csv: str, csv_ptbr_text: bool = True):
    """
    Salva o wide em parquet e CSV.
    Quando csv_ptbr_text=True, valores são salvos como TEXTO pt-BR.
    Missing fica como NULL no parquet e vazio no CSV.
    """
    df2 = df.copy()
    df2.index.name = "Assets"
    base = df2.reset_index().rename(columns={df2.reset_index().columns[0]: "Assets"})

    if csv_ptbr_text:
        out_txt = base.copy()
        # normaliza nomes das colunas de data
        cols_norm = []
        for c in out_txt.columns:
            if c == "Assets":
                cols_norm.append(c)
            else:
                try:
                    cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
                except Exception:
                    cols_norm.append(str(c))
        out_txt.columns = cols_norm

        # converte TODOS os valores (exceto Assets) para string pt-BR (missing -> pd.NA)
        for c in out_txt.columns:
            if c == "Assets":
                continue
            out_txt[c] = out_txt[c].map(_fmt_ptbr_2dec)

        out_txt.to_parquet(path_parquet, index=False)
        out_txt.to_csv(path_csv, index=False, encoding="utf-8")
    else:
        base.to_parquet(path_parquet, index=False)
        base.to_csv(path_csv, index=False, encoding="utf-8")


def mapear_asset(name: str, venc: str) -> str | None:
    """
    Mapeia os ativos de acordo com as regras definidas:
    - DAP (IPCACoupon):
        * ano par  -> pegar só vértice 'Q'
        * ano ímpar -> pegar só vértice 'K'
        * demais vértices do mesmo ano são ignorados
    - DI: mantém DI de 26+ (DI25 é ignorado)
    - WDO / UST: regras fixas
    """
    if not venc:
        return None

    venc = venc.strip().upper()

    # ---------- DAP (IPCACoupon) ----------
    if name == "IPCACoupon":
        if len(venc) < 3:
            return None

        letra = venc[0]
        ano_str = venc[-2:]
        try:
            ano = int(ano_str)
        except ValueError:
            return None

        if ano % 2 == 0:
            if letra != "Q":
                return None
        else:
            if letra != "K":
                return None

        return f"DAP{ano_str}"

    # ---------- DI (filtrar anos permitidos) ----------
    if name == "DI1Day":
        sufixo = venc[-2:]
        try:
            ano = int(sufixo)
        except ValueError:
            return None

        if not (26 <= ano):
            _append_log(f"[mapear_asset] Ignorando {name} venc={venc} (ano {ano} fora da faixa 26–37)")
            return None

        return f"DI_{sufixo}"

    # ---------- Dólar / WDO ----------
    if name in ("BusinessDollar", "WDOMiniFuture"):
        return "WDO1"

    # ---------- Treasury ----------
    if name == "USTNOTEFuture":
        return "TREASURY"

    return None


def _valor_por_ponto_dap(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    df_long_dia: pd.DataFrame,
    df_ipca_prevista: pd.DataFrame,
) -> float:
    """
    Calcula o valor por ponto do DAP:
      1) Tenta usar ValorIndiceDia vindo do arquivo da B3 (IPCACoupon).
      2) Se não houver, usa fallback via IPEA + inflação prevista da planilha.
         Se não houver inflação prevista para a data, encerra o programa.
    """
    vi = None
    try:
        vi = df_long_dia.loc[df_long_dia["Name"] == "IPCACoupon", "ValorIndiceDia"].dropna().iloc[0]
    except Exception:
        vi = None

    if vi is not None:
        vpp = REAIS_POR_PONTO * float(vi)
        _append_log(f"[DAP] Valor por ponto via arquivo B3: Índice={vi:.4f} -> R$ {vpp:.6f}")
        return vpp

    ipca_previsto = obter_ipca_prevista_para_data(df_ipca_prevista, data_ref)
    vpp = calcular_valor_ponto_dap_para_data(
        df_ipca=df_ipca,
        data_ref=data_ref,
        ipca_previsto=ipca_previsto,
        reais_por_ponto=REAIS_POR_PONTO,
    )
    _append_log(f"[DAP] Valor por ponto via fallback IPEA + planilha: R$ {vpp:.6f}")
    return vpp


def construir_colunas_wide_duplas(
    df_long_dia: pd.DataFrame,
    data_ref: dt.date,
    df_ipca: pd.DataFrame,
    df_ipca_prevista: pd.DataFrame,
) -> tuple[pd.Series, pd.Series]:
    df = df_long_dia.copy()
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()]

    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()
    s_valor = df.groupby("Asset")["Pontos"].first()

    if not df[df["Name"] == "IPCACoupon"].empty:
        valor_ponto_dap = _valor_por_ponto_dap(
            df_ipca=df_ipca,
            data_ref=data_ref,
            df_long_dia=df_long_dia,
            df_ipca_prevista=df_ipca_prevista,
        )
        for k in list(s_valor.index):
            if str(k).startswith("DAP"):
                val = pd.to_numeric(s_valor.loc[k], errors="coerce")
                if pd.notna(val):
                    s_valor.loc[k] = float(val) * valor_ponto_dap

    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name
    return s_preco, s_valor

# ==========================
# 6) Calendário B3 + backoff
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")


def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]


def ultimo_dia_util_ANTES_de_hoje() -> dt.date:
    today = dt.date.today()
    v = b3_calendar().valid_days(today - dt.timedelta(days=20), today - dt.timedelta(days=1))
    return v[-1].date()

# ==========================
# 7) Fetch genérico com backoff (para dias históricos)
# ==========================

def buscar_dia_com_backoff(target_d: dt.date, df_ipca: pd.DataFrame) -> pd.DataFrame:
    dfs = []
    validos_back = b3_valid_days(target_d - dt.timedelta(days=60), target_d)[::-1]
    tentativas_max = max(BACKOFF_LIM, 15)

    for name in NAMES:
        ok = False
        tentativa = 0
        for prev_d in [target_d] + validos_back:
            try:
                csv_text = baixar_csv_bdi(name, prev_d)
                if name == "IPCACoupon":
                    try:
                        df_n = parse_ipcacoupon_special(csv_text, prev_d, name)
                    except Exception as e_special:
                        _append_log(f"[warn] IPCACoupon parser especial falhou ({e_special}); usando genérico.")
                        df_n = parse_ajustes(csv_text, prev_d, name)
                else:
                    df_n = parse_ajustes(csv_text, prev_d, name)

                if df_n is not None and not df_n.empty:
                    df_n["Data_Referencia"] = pd.to_datetime(target_d)
                    dfs.append(df_n)
                    ok = True
                    if prev_d != target_d:
                        _append_log(f"• {name}: {target_d} vazio → usando {prev_d} (backfill)")
                    break
            except Exception as e:
                _append_log(f"! {name} @ {prev_d}: falhou parse ({str(e)[:120]})")
            tentativa += 1
            if tentativa > tentativas_max:
                break
        if not ok:
            _append_log(f"! {name}: sem dados até {tentativas_max} DUs atrás para {target_d}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)

    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

# ==========================
# 8) “Hoje” — coleta paralela com orçamento total (sem backfill)
# ==========================

def _try_parse_for_name(name: str, d: dt.date) -> pd.DataFrame:
    csv_text = baixar_csv_bdi(name, d)
    if name == "IPCACoupon":
        try:
            df_n = parse_ipcacoupon_special(csv_text, d, name)
        except Exception as e_special:
            _append_log(f"[warn] IPCACoupon especial falhou ({e_special}); usando genérico.")
            df_n = parse_ajustes(csv_text, d, name)
    else:
        df_n = parse_ajustes(csv_text, d, name)
    return df_n


def _fetch_one_name_exact(name: str, d: dt.date) -> pd.DataFrame | None:
    try:
        return _try_parse_for_name(name, d)
    except Exception as e:
        _append_log(f"[today-empty] {name} {d}: {e}")
        return None


def buscar_dia_EXATO_sem_backfill(target_d: dt.date) -> pd.DataFrame:
    _append_log(f"[today-check] Coleta EXATA do dia {target_d} (sem backfill).")
    dfs = []
    deadline = time.monotonic() + HTTP_TOTAL_BUDGET
    with ThreadPoolExecutor(max_workers=min(len(NAMES), 5)) as ex:
        futs = {ex.submit(_fetch_one_name_exact, n, target_d): n for n in NAMES}
        for fut in as_completed(futs):
            if time.monotonic() > deadline:
                _append_log("[today-timeout] Estourou orçamento total de tempo; seguindo sem esperar o restante.")
                break
            df_n = fut.result()
            if df_n is not None and not df_n.empty:
                dfs.append(df_n)

    if dfs:
        df_all = pd.concat(dfs, ignore_index=True)
        df_all["Data_Referencia"] = pd.to_datetime(target_d)
        _append_log(f"[today-ok] Dados encontrados para {target_d}: {len(df_all)} linhas.")
        return df_all

    _append_log(f"[today-nodata] NENHUM instrumento com dados em {target_d}.")
    return pd.DataFrame(columns=["Vencimento","Pontos","PrecoAjusteAtual","Data_Referencia","Name","ValorIndiceDia"])

# ==========================
# 9) ffill em DAP25 e NTNB
# ==========================

def preencher_vazios_dap25_e_ntnb(wide_preco: pd.DataFrame) -> pd.DataFrame:
    """
    Faz forward fill (ffill) na linha para:
      - DAP25 (exato)
      - todos os ativos cujo nome contenha 'NTNB' (case-insensitive)

    Preenche missing (pd.NA/None/NaN/"") com o último valor conhecido na própria linha.
    """
    if wide_preco is None or wide_preco.empty:
        return wide_preco

    # garante que "" já virou missing
    wide_preco = _normalize_missing_values_df(wide_preco)

    idx = wide_preco.index.astype(str)

    targets = []
    if "DAP25" in wide_preco.index:
        targets.append("DAP25")

    ntb_mask = idx.str.contains("NTNB", case=False, na=False)
    targets.extend(list(wide_preco.index[ntb_mask]))

    # remove duplicados preservando ordem
    seen = set()
    targets = [a for a in targets if not (a in seen or seen.add(a))]

    for asset in targets:
        if asset not in wide_preco.index:
            continue
        s = wide_preco.loc[asset].copy()
        s = _normalize_missing_values_df(pd.DataFrame(s).T).iloc[0].ffill()
        wide_preco.loc[asset] = s

    return wide_preco


def drop_tail_duplicate(df: pd.DataFrame) -> pd.DataFrame:
    """
    Se a última coluna for apenas uma cópia da penúltima E representar
    exatamente o próximo dia útil B3, remove essa coluna.
    """
    if df is None or df.empty or df.shape[1] < 2:
        return df

    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
    except Exception:
        return df

    last_dt = cols_dt[-1].date()
    prev_dt = cols_dt[-2].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")
    prev_col = cols_dt[-2].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(prev_dt, prev_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > prev_dt]
    if not prox_list:
        return df

    prox_dt = prox_list[0]
    if prox_dt != last_dt:
        return df

    try:
        if df[last_col].equals(df[prev_col]):
            _append_log(f"[drop-dup] Removendo coluna duplicada {last_col} (cópia de {prev_col})")
            return df.drop(columns=[last_col])
    except Exception:
        return df

    return df

# ==========================
# 10) Pipeline principal
# ==========================

def main():
    # 1) carregar IPCA (histórico), IPCA previsto (planilha) e bases existentes
    df_ipca = carregar_ipca_ipeadata()
    df_ipca_prevista = carregar_ipca_prevista()

    wide_preco = ler_wide(PATH_PRECO)
    wide_valor = ler_wide(PATH_VALOR)

    # segurança extra: remove assets indesejados e normaliza missing
    wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
    wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

    # 2) definir range de datas históricas (até ontem)
    def _last_col_date(df):
        if df is None or df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)

    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)

    end_dt_hist = ultimo_dia_util_ANTES_de_hoje()

    dias_util = b3_valid_days(start_dt, end_dt_hist)
    _append_log(f"Atualizando de {start_dt} até {end_dt_hist} (DU B3: {len(dias_util)})")

    # 3) dias históricos (com backfill)
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref, df_ipca)

        if df_dia_all.empty:
            _append_log(f"{dref}: nenhum dado disponível — mantendo planilhas (sem atualização).")
            continue

        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)
        _append_log(f"{dref}: base longa atualizada; total linhas = {len(df_long)}.")

        s_preco, s_valor = construir_colunas_wide_duplas(
            df_long_dia=df_dia_all,
            data_ref=dref,
            df_ipca=df_ipca,
            df_ipca_prevista=df_ipca_prevista,
        )

        if wide_preco.empty:
            wide_preco = pd.DataFrame(s_preco)
        else:
            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_preco[s_preco.name] = s_preco

        if wide_valor.empty:
            wide_valor = pd.DataFrame(s_valor)
        else:
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_valor[s_valor.name] = s_valor

        # normalize missing e remove assets indesejados a cada passo (evita “reviver”)
        wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
        wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

        _append_log(f"{dref}: preços/valores atualizados.")

    # 4) tentar "hoje" (exato): se houver dados, adiciona; se não houver, deixa para a duplicação final
    today = dt.date.today() - dt.timedelta(days=1)
    is_today_du = today in set(b3_valid_days(today, today))

    if is_today_du:
        df_today = buscar_dia_EXATO_sem_backfill(today)
        if not df_today.empty:
            df_long = incrementar_base_ajuste(PATH_LONG, df_today)
            _append_log(f"{today}: base longa atualizada (hoje); total linhas = {len(df_long)}.")

            s_preco, s_valor = construir_colunas_wide_duplas(
                df_long_dia=df_today,
                data_ref=today,
                df_ipca=df_ipca,
                df_ipca_prevista=df_ipca_prevista,
            )

            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_preco[s_preco.name] = s_preco
            wide_valor[s_valor.name] = s_valor

            wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
            wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

            _append_log(f"{today}: preços/valores de HOJE adicionados.")
        else:
            _append_log(f"{today}: sem dados de HOJE — segue sem coluna de hoje (duplicação será feita no export final).")
    else:
        _append_log(f"{today}: não é dia útil B3 — não tenta hoje.")

    # 6) ordenar colunas, adicionar coluna duplicada final e salvar
    def _order(df):
        if df is None or df.empty:
            return df
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    # cria a coluna do próximo DU como cópia da última real, se ainda não existir
    wide_preco, wide_valor = adicionar_coluna_duplicada_final(wide_preco, wide_valor)

    # normaliza missing ("" -> pd.NA) e ffill em DAP25/NTNB
    wide_preco = _normalize_missing_values_df(wide_preco)
    wide_preco = preencher_vazios_dap25_e_ntnb(wide_preco)

    # reordena (agora incluindo a nova coluna futura)
    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    # remove DI_25, DAP_25 e DAP25 do output FINAL
    wide_preco = remover_assets_indesejados(wide_preco)
    wide_valor = remover_assets_indesejados(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO, PATH_PRECO_CSV, csv_ptbr_text=True)
    salvar_wide(wide_valor, PATH_VALOR, PATH_VALOR_CSV, csv_ptbr_text=True)
    _append_log(f"Salvos: {PATH_PRECO} {wide_preco.shape} | {PATH_VALOR} {wide_valor.shape}")

    # 7) JSON pt-BR (texto garantido) — missing vira ""
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        _append_log(f"Salvo JSON pt-BR de preços (texto): {PATH_JSON}")
    except Exception as e:
        _append_log(f"[warn] Falha ao gerar JSON pt-BR: {e}")

if __name__ == "__main__":
    main()

[2025-12-15 14:39:47] [IPCA_PREV] Carregado 182 linhas de inflação prevista de TaxasInflacaoDiariaPrevisao.xlsx
[2025-12-15 14:39:48] [drop-dup] Removendo coluna duplicada 2025-12-12 (cópia de 2025-12-11)
[2025-12-15 14:39:48] [drop-dup] Removendo coluna duplicada 2025-12-12 (cópia de 2025-12-11)
[2025-12-15 14:39:48] Atualizando de 2025-12-11 até 2025-12-12 (DU B3: 2)
[2025-12-15 14:39:48] [HTTP] IPCACoupon 2025-12-11 -> status=200 content-length=1142 t=0.33s
    [dump] CSV bruto salvo em: debug_b3_csv\IPCACoupon_2025-12-11.csv
[2025-12-15 14:39:48] [HTTP] md5=64cde4d5df4cb183d1ff3477b208c519 bytes=2128 (IPCACoupon 2025-12-11)
----[ RAW IPCACoupon 2025-12-11 | primeiras 22 de 22 linhas ]----
﻿Vencimento;Contratos em aberto;Negócios realizados;Contratos negociados;Volume;Ajuste anterior;Preço de abertura;Preço mínimo;Preço máximo;Preço médio;Último preço;Ajuste;Variação em pontos;Última oferta de compra;Última oferta de venda
F26;63.017;9;327;59.775.888;99.103,08;10,27;10,27;10,27;10,2

SystemExit: Não há inflação prevista na planilha para a data 2025-12-11.

c:\Users\emanuel.cabral\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# -*- coding: utf-8 -*-
"""
Pipeline B3 (BDI) — via ConsolidatedTradesDerivatives
- Baixa UM endpoint: ConsolidatedTradesDerivatives
- Filtra DI1 / DAP / WDO (e DOL como fallback) e transforma em:
    * wide_preco: "Preço de Ajuste Atual" (coluna "Ajuste")
    * wide_valor: "Variação em pontos" (coluna "Variação")
      - EXCEÇÃO DAP: usa "Valor do ajuste por contrato (R$)" (cash) no lugar de IPCA
- Backfill (se o dia vier vazio)
- Exporta parquet + CSV (pt-BR em texto) + JSON (pt-BR em texto)
- Remove do output final: DI_25, DAP_25, DAP25, DI25
- Substitui "" / "-" por missing (pd.NA) no processamento e no parquet
"""

from __future__ import annotations

import csv
import datetime as dt
import hashlib
import io
import json
import os
import re
import time
import unicodedata
from copy import deepcopy
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests
import pandas_market_calendars as mcal
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


# ==========================
# Arquivos (bases)
# ==========================

PATH_LONG       = "df_ajustes_b3.parquet"                  # base longa
PATH_PRECO      = "df_preco_de_ajuste_atual_completo.parquet"
PATH_VALOR      = "df_valor_ajuste_contrato.parquet"
PATH_JSON       = "df_preco_de_ajuste_atual_completo.json" # JSON pt-BR (texto)
PATH_PRECO_CSV  = "df_preco_de_ajuste_atual_completo.csv"  # CSV pt-BR (texto)
PATH_VALOR_CSV  = "df_valor_ajuste_contrato.csv"           # CSV pt-BR (texto)
PATH_RUN_LOG    = "atualizacao_b3_log.txt"

# Assets a EXCLUIR do output final (parquet/csv/json)
ASSETS_EXCLUIR = ["DI_25", "DAP_25", "DAP25", "DI25"]


# ==========================
# HTTP — sessão com retries/timeouts
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

# Nome ÚNICO do endpoint
B3_NAME = "ConsolidatedTradesDerivatives"

PAYLOAD_BASE = {
    "Name": B3_NAME,
    "Date": "2025-01-02",
    "FinalDate": "2025-01-02",
    "ClientId": "",
    "Filters": {},
}

HTTP_CONNECT_TIMEOUT = 3.0
HTTP_READ_TIMEOUT    = 20.0
HTTP_TOTAL_BUDGET    = 90.0

RETRY_CFG = Retry(
    total=2,
    backoff_factor=0.6,
    status_forcelist=(500, 502, 503, 504),
    allowed_methods=frozenset(["POST"])
)

_session = requests.Session()
_adapter = HTTPAdapter(max_retries=RETRY_CFG, pool_connections=10, pool_maxsize=10)
_session.mount("https://", _adapter)
_session.mount("http://", _adapter)


# ==========================
# Debug / inspeção
# ==========================

DEBUG_MAX_LINES = 40
DEBUG_DUMP_DIR  = "debug_b3_csv"  # None para desativar
BACKOFF_LIM     = 15              # janela backoff (dias ÚTEIS)


try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None


def _append_log(msg: str):
    ts = dt.datetime.now(_TZ).strftime("%Y-%m-%d %H:%M:%S") if _TZ else dt.datetime.now().isoformat(sep=" ", timespec="seconds")
    line = f"[{ts}] {msg}"
    print(line)
    try:
        with open(PATH_RUN_LOG, "a", encoding="utf-8") as f:
            f.write(line + "\n")
    except Exception:
        pass


def _ensure_debug_dir():
    if DEBUG_DUMP_DIR:
        Path(DEBUG_DUMP_DIR).mkdir(parents=True, exist_ok=True)


def _dump_csv(data: dt.date, raw_bytes: bytes):
    if not DEBUG_DUMP_DIR:
        return
    _ensure_debug_dir()
    fn = Path(DEBUG_DUMP_DIR) / f"{B3_NAME}_{data.strftime('%Y-%m-%d')}.csv"
    try:
        fn.write_bytes(raw_bytes)
        print(f"    [dump] CSV bruto salvo em: {fn}")
    except Exception as e:
        print(f"    [dump:fail] {e}")


def _print_snippet(tag: str, text: str, max_lines: int = DEBUG_MAX_LINES):
    lines = (text or "").splitlines()
    header = f"----[ {tag} | primeiras {min(len(lines), max_lines)} de {len(lines)} linhas ]----"
    print(header)
    for ln in lines[:max_lines]:
        print(ln)
    print("-" * len(header))


# ==========================
# Helpers de parsing / missing
# ==========================

def _strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')


def _normalize_missing_values_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Troca strings vazias/whitespace/"-" por pd.NA.
    """
    if df is None or df.empty:
        return df
    df2 = df.replace(r"^\s*$", pd.NA, regex=True)
    df2 = df2.replace({"-": pd.NA, "–": pd.NA, "—": pd.NA})
    df2 = df2.replace({"nan": pd.NA, "NaN": pd.NA, "none": pd.NA, "None": pd.NA, "null": pd.NA, "NULL": pd.NA})
    return df2


def ptbr_to_float(s):
    """
    Converte string pt-BR para float.
    Retorna None para "", "-", None, NaN etc.
    """
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)

    s = str(s).strip()
    if s in {"", "-", "–", "—"}:
        return None

    s = re.sub(r"[^0-9\-,\.]", "", s).strip()
    if s in {"", "-"}:
        return None

    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None


def remover_assets_indesejados(w: pd.DataFrame) -> pd.DataFrame:
    if w is None or w.empty:
        return w
    return w.drop(index=ASSETS_EXCLUIR, errors="ignore")


def _fmt_ptbr_2dec(x):
    """
    98252.84 -> "98.252,84"
    Missing -> pd.NA
    """
    if x is None or pd.isna(x):
        return pd.NA
    if isinstance(x, str) and x.strip() == "":
        return pd.NA

    try:
        v = float(x)
    except Exception:
        s = str(x)
        return pd.NA if s.strip() == "" else s

    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")


def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    if wide_df is None or wide_df.empty:
        return "[]"

    cols_norm = []
    for c in wide_df.columns:
        try:
            cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols_norm.append(str(c))

    df = wide_df.copy()
    df.columns = cols_norm
    df.index.name = "Assets"

    df_txt = df.copy()
    for c in df_txt.columns:
        df_txt[c] = df_txt[c].map(_fmt_ptbr_2dec)

    records = []
    for asset, row in df_txt.iterrows():
        rec = {"Assets": str(asset)}
        for col in df_txt.columns:
            val = row[col]
            rec[str(col)] = "" if (val is None or pd.isna(val)) else str(val)
        records.append(rec)

    return json.dumps(records, ensure_ascii=False)


# ==========================
# Payload / download do CSV
# ==========================

def montar_payload(data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Date"] = s
    p["FinalDate"] = s
    return p


def baixar_csv_b3(data: dt.date) -> str:
    start = time.monotonic()
    r = _session.post(
        URL,
        headers=HEADERS,
        json=montar_payload(data),
        timeout=(HTTP_CONNECT_TIMEOUT, HTTP_READ_TIMEOUT),
    )
    elapsed = time.monotonic() - start
    clen = r.headers.get("Content-Length", "?")
    _append_log(f"[HTTP] {B3_NAME} {data} -> status={r.status_code} content-length={clen} t={elapsed:.2f}s")

    if r.status_code != 200:
        raise RuntimeError(f"{data}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data}: CSV vazio")

    _dump_csv(data, r.content)
    md5 = hashlib.md5(r.content).hexdigest()
    _append_log(f"[HTTP] md5={md5} bytes={len(r.content)} ({B3_NAME} {data})")

    # decode
    txt = None
    for enc in ("utf-8-sig", "utf-8", "latin-1"):
        try:
            txt = r.content.decode(enc)
            break
        except UnicodeDecodeError:
            pass
    if txt is None:
        txt = r.content.decode("utf-8", errors="replace")

    _print_snippet(f"RAW {B3_NAME} {data}", txt)
    return txt


# ==========================
# Mapeamento de ativos (mesma lógica do seu código)
# ==========================

MONTH_CODE = {
    "F": 1, "G": 2, "H": 3, "J": 4, "K": 5, "M": 6,
    "N": 7, "Q": 8, "U": 9, "V": 10, "X": 11, "Z": 12
}


def maturity_date_from_venc(venc: str) -> dt.date | None:
    if not venc or len(venc) < 3:
        return None
    letra = venc[0].upper()
    yy = venc[-2:]
    if letra not in MONTH_CODE or not yy.isdigit():
        return None
    return dt.date(2000 + int(yy), MONTH_CODE[letra], 1)


def mapear_asset(name: str, venc: str) -> str | None:
    """
    Mesma regra que você já usa:
    - DAP (IPCACoupon):
        * ano par  -> manter só Q
        * ano ímpar -> manter só K
        -> Asset "DAPyy"
    - DI (DI1Day): manter ano >= 26
        -> Asset "DI_yy"
    - WDO/DOL -> "WDO1"
    """
    if not venc:
        return None

    venc = venc.strip().upper()

    if name == "IPCACoupon":
        if len(venc) < 3:
            return None
        letra = venc[0]
        ano_str = venc[-2:]
        try:
            ano = int(ano_str)
        except ValueError:
            return None

        if ano % 2 == 0:
            if letra != "Q":
                return None
        else:
            if letra != "K":
                return None

        return f"DAP{ano_str}"

    if name == "DI1Day":
        sufixo = venc[-2:]
        try:
            ano = int(sufixo)
        except ValueError:
            return None
        if not (26 <= ano):
            return None
        return f"DI_{sufixo}"

    if name in ("BusinessDollar", "WDOMiniFuture"):
        return "WDO1"

    if name == "USTNOTEFuture":
        return "TREASURY"

    return None


# ==========================
# Parsing do ConsolidatedTradesDerivatives
# ==========================

def parse_consolidated_trades(csv_text: str, data_ref: dt.date) -> pd.DataFrame:
    """
    Lê o CSV do ConsolidatedTradesDerivatives e devolve DF “long” já enxuto
    para instrumentos futuros (código com 6 chars) DI1/DAP/WDO/DOL.

    Colunas finais:
      - Instrumento
      - Vencimento (ex.: F26)
      - Name (DI1Day / IPCACoupon / WDOMiniFuture)
      - PrecoAjusteAtual (float)  <- coluna "Ajuste"
      - Pontos (float)            <- coluna "Variação"
      - ValorAjusteR$ (float)     <- coluna "Valor do ajuste por contrato (R$)"
      - Data_Referencia (datetime)
      - ValorIndiceDia (NA)       <- mantido só para compatibilidade com sua base antiga
    """
    df_raw = pd.read_csv(
        io.StringIO(csv_text),
        sep=";",
        skiprows=2,         # padrão desse arquivo: linha 0 título, linha 1 vazia, linha 2 header
        dtype=str,
        engine="python",
        on_bad_lines="skip",
    )
    if df_raw is None or df_raw.empty:
        return pd.DataFrame(columns=[
            "Instrumento","Vencimento","Name",
            "PrecoAjusteAtual","Pontos","ValorAjusteR$",
            "Data_Referencia","ValorIndiceDia"
        ])

    df_raw = _normalize_missing_values_df(df_raw)

    colmap = {c: _strip_accents(c).lower() for c in df_raw.columns}

    def _find_col(*must_have):
        for orig, norm in colmap.items():
            if all(x in norm for x in must_have):
                return orig
        return None

    c_inst = _find_col("instrumento", "financeiro") or _find_col("instrumento")
    c_ajuste = _find_col("ajuste")
    c_var = _find_col("variacao", "ponto") or _find_col("variacao")
    c_val_adj = _find_col("valor", "ajuste", "contrato")

    if not all([c_inst, c_ajuste, c_var, c_val_adj]):
        _append_log(f"[parse] Colunas esperadas não encontradas. Vistas: {list(df_raw.columns)}")
        return pd.DataFrame(columns=[
            "Instrumento","Vencimento","Name",
            "PrecoAjusteAtual","Pontos","ValorAjusteR$",
            "Data_Referencia","ValorIndiceDia"
        ])

    df = df_raw[[c_inst, c_ajuste, c_var, c_val_adj]].copy()
    df.columns = ["Instrumento", "PrecoAjusteAtual", "Pontos", "ValorAjusteR$"]

    # Mantém apenas tickers de futuro “curtos” (6 chars) e prefixos relevantes
    s = df["Instrumento"].astype(str)
    df = df[s.str.len().eq(6) & s.str.match(r"^(DI1|DAP|WDO|DOL)", na=False)].copy()

    if df.empty:
        return pd.DataFrame(columns=[
            "Instrumento","Vencimento","Name",
            "PrecoAjusteAtual","Pontos","ValorAjusteR$",
            "Data_Referencia","ValorIndiceDia"
        ])

    # Vencimento = parte após o prefixo (3 chars): ex DI1F26 -> F26
    df["Vencimento"] = df["Instrumento"].astype(str).str[3:]

    # Name: compatível com seu mapeamento
    def _name_from_inst(x: str) -> str:
        if x.startswith("DI1"):
            return "DI1Day"
        if x.startswith("DAP"):
            return "IPCACoupon"
        if x.startswith("WDO"):
            return "WDOMiniFuture"
        if x.startswith("DOL"):
            return "BusinessDollar"
        return "Other"

    df["Name"] = df["Instrumento"].astype(str).map(_name_from_inst)

    # Converte numéricos
    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].map(ptbr_to_float)
    df["Pontos"] = df["Pontos"].map(ptbr_to_float)
    df["ValorAjusteR$"] = df["ValorAjusteR$"].map(ptbr_to_float)

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["ValorIndiceDia"] = pd.NA

    # Drop linhas totalmente vazias em preço/var
    df = df[(pd.notna(df["PrecoAjusteAtual"])) | (pd.notna(df["Pontos"])) | (pd.notna(df["ValorAjusteR$"]))].copy()

    return df[[
        "Instrumento","Vencimento","Name",
        "PrecoAjusteAtual","Pontos","ValorAjusteR$",
        "Data_Referencia","ValorIndiceDia"
    ]].reset_index(drop=True)


def selecionar_vertices(df_day: pd.DataFrame, data_ref: dt.date) -> pd.DataFrame:
    """
    Resolve o “excesso” de maturidades no consolidado para ficar igual à ideia
    do seu pipeline:

    - DI: mantém só DI1Fyy (Jan) por ano -> DI_yy
    - DAP: sua regra (ano par Q / ímpar K) -> DAPyy
    - WDO1: escolhe 1 contrato (prefere WDO, senão DOL) com vencimento mais próximo
            (>= mês de data_ref; se não houver, pega o menor vencimento disponível)
    """
    if df_day is None or df_day.empty:
        return df_day

    df = df_day.copy()

    # DI: keep apenas mês "F" (Jan) para não colapsar várias maturidades no mesmo DI_yy
    di_mask = df["Name"].eq("DI1Day")
    df = df[~di_mask | df["Vencimento"].astype(str).str.startswith("F", na=False)].copy()

    # Mapeia Asset (aplica regra DAP par/ímpar etc.)
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()].copy()

    # Seleção do WDO1
    w = df[df["Asset"].eq("WDO1")].copy()
    if not w.empty:
        w["Prefix"] = w["Instrumento"].astype(str).str[:3]
        w["MatDate"] = w["Vencimento"].map(maturity_date_from_venc)
        ref_month = dt.date(data_ref.year, data_ref.month, 1)

        # prefere WDO (mini)
        w_pref = w[w["Prefix"].eq("WDO")].copy()
        if w_pref.empty:
            w_pref = w.copy()

        w_pref["MatDate2"] = w_pref["MatDate"].fillna(dt.date(2099, 1, 1))
        after = w_pref[w_pref["MatDate2"] >= ref_month]
        use = after if not after.empty else w_pref

        best = use.sort_values("MatDate2").head(1)

        df = pd.concat([df[df["Asset"].ne("WDO1")], best], ignore_index=True)

    return df.reset_index(drop=True)


# ==========================
# Base longa
# ==========================

LONG_COLS = [
    "Instrumento","Vencimento","Name",
    "PrecoAjusteAtual","Pontos","ValorAjusteR$",
    "Data_Referencia","ValorIndiceDia"
]


def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        base = pd.read_parquet(p)
        # normaliza colunas
        for c in LONG_COLS:
            if c not in base.columns:
                base[c] = pd.NA
        base = base[LONG_COLS].copy()
        return base
    return pd.DataFrame(columns=LONG_COLS)


def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                           chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_novo2 = df_novo.copy()
    for c in LONG_COLS:
        if c not in df_novo2.columns:
            df_novo2[c] = pd.NA
    df_novo2 = df_novo2[LONG_COLS].copy()

    df_comb = pd.concat([df_base, df_novo2], ignore_index=True) if not df_base.empty else df_novo2.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# Wides (preço e valor)
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")


def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]


def drop_tail_duplicate(df: pd.DataFrame) -> pd.DataFrame:
    """
    Se a última coluna for apenas uma cópia da penúltima e representar
    exatamente o próximo dia útil B3, remove essa coluna.
    """
    if df is None or df.empty or df.shape[1] < 2:
        return df

    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
    except Exception:
        return df

    last_dt = cols_dt[-1].date()
    prev_dt = cols_dt[-2].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")
    prev_col = cols_dt[-2].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(prev_dt, prev_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > prev_dt]
    if not prox_list:
        return df

    prox_dt = prox_list[0]
    if prox_dt != last_dt:
        return df

    try:
        if df[last_col].equals(df[prev_col]):
            _append_log(f"[drop-dup] Removendo coluna duplicada {last_col} (cópia de {prev_col})")
            return df.drop(columns=[last_col])
    except Exception:
        return df

    return df


def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")

    df = pd.read_parquet(path)

    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"

    # ordena colunas por data
    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass

    df = df[~df.index.duplicated(keep="last")]
    df = drop_tail_duplicate(df)
    df = _normalize_missing_values_df(df)
    df = remover_assets_indesejados(df)
    return df


def salvar_wide(df: pd.DataFrame, path_parquet: str, path_csv: str, csv_ptbr_text: bool = True):
    df2 = df.copy()
    df2.index.name = "Assets"
    base = df2.reset_index().rename(columns={df2.reset_index().columns[0]: "Assets"})

    if csv_ptbr_text:
        out_txt = base.copy()

        cols_norm = []
        for c in out_txt.columns:
            if c == "Assets":
                cols_norm.append(c)
            else:
                try:
                    cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
                except Exception:
                    cols_norm.append(str(c))
        out_txt.columns = cols_norm

        for c in out_txt.columns:
            if c == "Assets":
                continue
            out_txt[c] = out_txt[c].map(_fmt_ptbr_2dec)

        out_txt.to_parquet(path_parquet, index=False)
        out_txt.to_csv(path_csv, index=False, encoding="utf-8")
    else:
        base.to_parquet(path_parquet, index=False)
        base.to_csv(path_csv, index=False, encoding="utf-8")


def adicionar_coluna_duplicada_final(wp: pd.DataFrame, wv: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    col_names = []
    if wp is not None and wp.shape[1] > 0:
        col_names.extend(list(wp.columns))
    if wv is not None and wv.shape[1] > 0:
        col_names.extend(list(wv.columns))

    if not col_names:
        _append_log("[dup-final] wp/wv vazios → nada para duplicar.")
        return wp, wv

    try:
        cols_dt = sorted(pd.to_datetime(col_names))
    except Exception:
        _append_log("[dup-final] Não consegui interpretar colunas como datas → não duplica.")
        return wp, wv

    last_dt = cols_dt[-1].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > last_dt]
    if not prox_list:
        _append_log(f"[dup-final] Não há próximo dia útil após {last_dt} → não duplica.")
        return wp, wv

    prox_dt = prox_list[0]
    prox_col = prox_dt.strftime("%Y-%m-%d")

    already_p = wp is not None and wp.shape[1] > 0 and prox_col in wp.columns
    already_v = wv is not None and wv.shape[1] > 0 and prox_col in wv.columns

    if already_p and already_v:
        _append_log(f"[dup-final] Coluna {prox_col} já existe em preços e valores → nenhuma duplicação feita.")
        return wp, wv

    if wp is not None and wp.shape[1] > 0 and not already_p and last_col in wp.columns:
        wp[prox_col] = wp[last_col]
        _append_log(f"[dup-final] (preço) Duplicado {last_col} -> {prox_col}")

    if wv is not None and wv.shape[1] > 0 and not already_v and last_col in wv.columns:
        wv[prox_col] = wv[last_col]
        _append_log(f"[dup-final] (valor) Duplicado {last_col} -> {prox_col}")

    return wp, wv


def construir_colunas_wide_duplas(df_long_dia: pd.DataFrame, data_ref: dt.date) -> tuple[pd.Series, pd.Series]:
    """
    - s_preco: PrecoAjusteAtual (Ajuste)
    - s_valor: Pontos (Variação) para DI/WDO
      - DAP: usa ValorAjusteR$ (Valor do ajuste por contrato (R$))
    """
    df = selecionar_vertices(df_long_dia, data_ref)
    if df is None or df.empty:
        col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
        s_preco = pd.Series(dtype="float64", name=col_name)
        s_valor = pd.Series(dtype="float64", name=col_name)
        return s_preco, s_valor

    # aqui a coluna Asset já existe após selecionar_vertices
    if "Asset" not in df.columns:
        df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
        df = df[df["Asset"].notna()].copy()

    # ordena para "first()" ser estável
    df["MatDate"] = df["Vencimento"].map(maturity_date_from_venc)
    df = df.sort_values(["Asset", "MatDate", "Instrumento"], na_position="last").copy()

    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()
    s_valor = df.groupby("Asset")["Pontos"].first()

    # Override DAP com cash do arquivo (Valor do ajuste por contrato)
    dap_mask = df["Asset"].astype(str).str.startswith("DAP", na=False)
    if dap_mask.any():
        s_cash = df.loc[dap_mask].groupby("Asset")["ValorAjusteR$"].first()
        for a, v in s_cash.items():
            if pd.notna(v):
                s_valor.loc[a] = v

    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name
    return s_preco, s_valor


# ==========================
# Fetch com backoff (histórico) + “hoje” exato
# ==========================

def buscar_dia_com_backoff(target_d: dt.date) -> pd.DataFrame:
    validos_back = b3_valid_days(target_d - dt.timedelta(days=60), target_d)[::-1]
    tentativas_max = max(BACKOFF_LIM, 15)

    tentativa = 0
    for prev_d in [target_d] + validos_back:
        try:
            csv_text = baixar_csv_b3(prev_d)
            df_n = parse_consolidated_trades(csv_text, prev_d)
            if df_n is not None and not df_n.empty:
                df_n["Data_Referencia"] = pd.to_datetime(target_d)
                if prev_d != target_d:
                    _append_log(f"• {B3_NAME}: {target_d} vazio → usando {prev_d} (backfill)")
                return df_n
        except Exception as e:
            _append_log(f"! {B3_NAME} @ {prev_d}: falhou parse ({str(e)[:120]})")

        tentativa += 1
        if tentativa > tentativas_max:
            break

    _append_log(f"! {B3_NAME}: sem dados até {tentativas_max} DUs atrás para {target_d}")
    return pd.DataFrame(columns=LONG_COLS)


def buscar_dia_EXATO_sem_backfill(target_d: dt.date) -> pd.DataFrame:
    _append_log(f"[today-check] Coleta EXATA do dia {target_d} (sem backfill).")
    try:
        csv_text = baixar_csv_b3(target_d)
        df_n = parse_consolidated_trades(csv_text, target_d)
        if df_n is not None and not df_n.empty:
            df_n["Data_Referencia"] = pd.to_datetime(target_d)
            _append_log(f"[today-ok] Dados encontrados para {target_d}: {len(df_n)} linhas.")
            return df_n
    except Exception as e:
        _append_log(f"[today-empty] {target_d}: {e}")

    _append_log(f"[today-nodata] NENHUM dado em {target_d}.")
    return pd.DataFrame(columns=LONG_COLS)


# ==========================
# Calendário / range
# ==========================

def ultimo_dia_util_ANTES_de_hoje() -> dt.date:
    today = dt.date.today()
    v = b3_calendar().valid_days(today - dt.timedelta(days=20), today - dt.timedelta(days=1))
    return v[-1].date()


# ==========================
# Pipeline principal
# ==========================

def main():
    # 1) carregar bases existentes
    wide_preco = ler_wide(PATH_PRECO)
    wide_valor = ler_wide(PATH_VALOR)

    wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
    wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

    def _last_col_date(df):
        if df is None or df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)

    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)

    end_dt_hist = ultimo_dia_util_ANTES_de_hoje()
    dias_util = b3_valid_days(start_dt, end_dt_hist)

    _append_log(f"Atualizando de {start_dt} até {end_dt_hist} (DU B3: {len(dias_util)})")

    # 2) histórico (com backfill)
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref)
        if df_dia_all.empty:
            _append_log(f"{dref}: nenhum dado disponível — mantendo (sem atualização).")
            continue

        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)
        _append_log(f"{dref}: base longa atualizada; total linhas = {len(df_long)}.")

        s_preco, s_valor = construir_colunas_wide_duplas(df_dia_all, dref)

        if wide_preco.empty:
            wide_preco = pd.DataFrame(s_preco)
        else:
            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_preco[s_preco.name] = s_preco

        if wide_valor.empty:
            wide_valor = pd.DataFrame(s_valor)
        else:
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_valor[s_valor.name] = s_valor

        wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
        wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

        _append_log(f"{dref}: preços/valores atualizados.")

    # 3) tentar “hoje” (mantive sua lógica: pega ontem)
    today = dt.date.today() - dt.timedelta(days=1)
    is_today_du = today in set(b3_valid_days(today, today))

    if is_today_du:
        df_today = buscar_dia_EXATO_sem_backfill(today)
        if not df_today.empty:
            df_long = incrementar_base_ajuste(PATH_LONG, df_today)
            _append_log(f"{today}: base longa atualizada (hoje); total linhas = {len(df_long)}.")

            s_preco, s_valor = construir_colunas_wide_duplas(df_today, today)

            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_preco[s_preco.name] = s_preco
            wide_valor[s_valor.name] = s_valor

            wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
            wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

            _append_log(f"{today}: preços/valores de HOJE adicionados.")
        else:
            _append_log(f"{today}: sem dados de HOJE — segue sem coluna de hoje (duplicação será feita no export final).")
    else:
        _append_log(f"{today}: não é dia útil B3 — não tenta hoje.")

    # 4) ordenar colunas
    def _order(df):
        if df is None or df.empty:
            return df
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    # 5) cria a coluna do próximo DU como cópia da última real, se ainda não existir
    wide_preco, wide_valor = adicionar_coluna_duplicada_final(wide_preco, wide_valor)

    # 6) normalize missing e remove assets indesejados no output FINAL
    wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
    wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO, PATH_PRECO_CSV, csv_ptbr_text=True)
    salvar_wide(wide_valor, PATH_VALOR, PATH_VALOR_CSV, csv_ptbr_text=True)
    _append_log(f"Salvos: {PATH_PRECO} {wide_preco.shape} | {PATH_VALOR} {wide_valor.shape}")

    # 7) JSON pt-BR (texto garantido) — missing vira ""
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        _append_log(f"Salvo JSON pt-BR de preços (texto): {PATH_JSON}")
    except Exception as e:
        _append_log(f"[warn] Falha ao gerar JSON pt-BR: {e}")


if __name__ == "__main__":
    main()

[2025-12-15 16:30:58] [drop-dup] Removendo coluna duplicada 2025-12-12 (cópia de 2025-12-11)
[2025-12-15 16:30:58] [drop-dup] Removendo coluna duplicada 2025-12-12 (cópia de 2025-12-11)
[2025-12-15 16:30:58] Atualizando de 2025-12-11 até 2025-12-12 (DU B3: 2)
[2025-12-15 16:30:59] [HTTP] ConsolidatedTradesDerivatives 2025-12-11 -> status=200 content-length=241 t=0.53s
    [dump] CSV bruto salvo em: debug_b3_csv\ConsolidatedTradesDerivatives_2025-12-11.csv
[2025-12-15 16:30:59] [HTTP] md5=10964c1186cfc185b08379622188377e bytes=371 (ConsolidatedTradesDerivatives 2025-12-11)
----[ RAW ConsolidatedTradesDerivatives 2025-12-11 | primeiras 2 de 2 linhas ]----
Instrumento financeiro;Código ISIN;Segmento;Preço de abertura;Preço mínimo;Preço máximo;Preço médio;Preço de fechamento;Oscilação;Ajuste;Ajuste de referência;Preço de referência;Variação;Valor do ajuste por contrato (R$);Última oferta de compra;Última oferta de venda;Quantidade de negócios;Quantidade de contratos;Volume financeiro
Nenhu

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_21364\800569709.py:556: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comb = pd.concat([df_base, df_novo2], ignore_index=True) if not df_base.empty else df_novo2.copy()


[2025-12-15 16:31:04] 2025-12-12: base longa atualizada; total linhas = 2367.
[2025-12-15 16:31:04] 2025-12-12: preços/valores atualizados.
[2025-12-15 16:31:04] 2025-12-14: não é dia útil B3 — não tenta hoje.
[2025-12-15 16:31:04] [dup-final] (preço) Duplicado 2025-12-12 -> 2025-12-15
[2025-12-15 16:31:04] [dup-final] (valor) Duplicado 2025-12-12 -> 2025-12-15
[2025-12-15 16:31:05] Salvos: df_preco_de_ajuste_atual_completo.parquet (41, 241) | df_valor_ajuste_contrato.parquet (30, 241)
[2025-12-15 16:31:05] Salvo JSON pt-BR de preços (texto): df_preco_de_ajuste_atual_completo.json


Código certo abaixo

In [1]:
# -*- coding: utf-8 -*-
"""
Pipeline B3 (BDI) — via ConsolidatedTradesDerivatives
- Baixa UM endpoint: ConsolidatedTradesDerivatives
- Filtra DI1 / DAP / WDO (e DOL como fallback) e T10 (Treasury) e transforma em:
    * wide_preco: "Preço de Ajuste Atual" (coluna "Ajuste")
    * wide_valor: "Variação em pontos" (coluna "Variação")
      - EXCEÇÃO DAP: usa "Valor do ajuste por contrato (R$)" (cash) no lugar de IPCA
- Backfill (se o dia vier vazio)
- Exporta parquet + CSV (pt-BR em texto) + JSON (pt-BR em texto)
- Remove do output final: DI_25, DAP_25, DAP25, DI25
- Substitui "" / "-" por missing (pd.NA) no processamento e no parquet
- TREASURY: pega sempre o contrato "mais próximo" (>= mês de referência), como o WDO
"""

from __future__ import annotations

import csv
import datetime as dt
import hashlib
import io
import json
import os
import re
import time
import unicodedata
from copy import deepcopy
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests
import pandas_market_calendars as mcal
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


# ==========================
# Arquivos (bases)
# ==========================

PATH_LONG       = "df_ajustes_b3.parquet"                  # base longa
PATH_PRECO      = "df_preco_de_ajuste_atual_completo.parquet"
PATH_VALOR      = "df_valor_ajuste_contrato.parquet"
PATH_JSON       = "df_preco_de_ajuste_atual_completo.json" # JSON pt-BR (texto)
PATH_PRECO_CSV  = "df_preco_de_ajuste_atual_completo.csv"  # CSV pt-BR (texto)
PATH_VALOR_CSV  = "df_valor_ajuste_contrato.csv"           # CSV pt-BR (texto)
PATH_RUN_LOG    = "atualizacao_b3_log.txt"

# Assets a EXCLUIR do output final (parquet/csv/json)
ASSETS_EXCLUIR = ["DI_25", "DAP_25", "DAP25", "DI25"]


# ==========================
# HTTP — sessão com retries/timeouts
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

# Nome ÚNICO do endpoint
B3_NAME = "ConsolidatedTradesDerivatives"

PAYLOAD_BASE = {
    "Name": B3_NAME,
    "Date": "2025-01-02",
    "FinalDate": "2025-01-02",
    "ClientId": "",
    "Filters": {},
}

HTTP_CONNECT_TIMEOUT = 3.0
HTTP_READ_TIMEOUT    = 20.0
HTTP_TOTAL_BUDGET    = 90.0

RETRY_CFG = Retry(
    total=2,
    backoff_factor=0.6,
    status_forcelist=(500, 502, 503, 504),
    allowed_methods=frozenset(["POST"])
)

_session = requests.Session()
_adapter = HTTPAdapter(max_retries=RETRY_CFG, pool_connections=10, pool_maxsize=10)
_session.mount("https://", _adapter)
_session.mount("http://", _adapter)


# ==========================
# Debug / inspeção
# ==========================

DEBUG_MAX_LINES = 40
DEBUG_DUMP_DIR  = "debug_b3_csv"  # None para desativar
BACKOFF_LIM     = 15              # janela backoff (dias ÚTEIS)


try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None


def _append_log(msg: str):
    ts = dt.datetime.now(_TZ).strftime("%Y-%m-%d %H:%M:%S") if _TZ else dt.datetime.now().isoformat(sep=" ", timespec="seconds")
    line = f"[{ts}] {msg}"
    print(line)
    try:
        with open(PATH_RUN_LOG, "a", encoding="utf-8") as f:
            f.write(line + "\n")
    except Exception:
        pass


def _ensure_debug_dir():
    if DEBUG_DUMP_DIR:
        Path(DEBUG_DUMP_DIR).mkdir(parents=True, exist_ok=True)


def _dump_csv(data: dt.date, raw_bytes: bytes):
    if not DEBUG_DUMP_DIR:
        return
    _ensure_debug_dir()
    fn = Path(DEBUG_DUMP_DIR) / f"{B3_NAME}_{data.strftime('%Y-%m-%d')}.csv"
    try:
        fn.write_bytes(raw_bytes)
        print(f"    [dump] CSV bruto salvo em: {fn}")
    except Exception as e:
        print(f"    [dump:fail] {e}")


def _print_snippet(tag: str, text: str, max_lines: int = DEBUG_MAX_LINES):
    lines = (text or "").splitlines()
    header = f"----[ {tag} | primeiras {min(len(lines), max_lines)} de {len(lines)} linhas ]----"
    print(header)
    for ln in lines[:max_lines]:
        print(ln)
    print("-" * len(header))


# ==========================
# Helpers de parsing / missing
# ==========================

def _strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')


def _normalize_missing_values_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Troca strings vazias/whitespace/"-" por pd.NA.
    """
    if df is None or df.empty:
        return df
    df2 = df.replace(r"^\s*$", pd.NA, regex=True)
    df2 = df2.replace({"-": pd.NA, "–": pd.NA, "—": pd.NA})
    df2 = df2.replace({"nan": pd.NA, "NaN": pd.NA, "none": pd.NA, "None": pd.NA, "null": pd.NA, "NULL": pd.NA})
    return df2


def ptbr_to_float(s):
    """
    Converte string pt-BR para float.
    Retorna None para "", "-", None, NaN etc.
    """
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)

    s = str(s).strip()
    if s in {"", "-", "–", "—"}:
        return None

    s = re.sub(r"[^0-9\-,\.]", "", s).strip()
    if s in {"", "-"}:
        return None

    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None


def remover_assets_indesejados(w: pd.DataFrame) -> pd.DataFrame:
    if w is None or w.empty:
        return w
    return w.drop(index=ASSETS_EXCLUIR, errors="ignore")


def _fmt_ptbr_2dec(x):
    """
    98252.84 -> "98.252,84"
    Missing -> pd.NA
    """
    if x is None or pd.isna(x):
        return pd.NA
    if isinstance(x, str) and x.strip() == "":
        return pd.NA

    try:
        v = float(x)
    except Exception:
        s = str(x)
        return pd.NA if s.strip() == "" else s

    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")


def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    if wide_df is None or wide_df.empty:
        return "[]"

    cols_norm = []
    for c in wide_df.columns:
        try:
            cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols_norm.append(str(c))

    df = wide_df.copy()
    df.columns = cols_norm
    df.index.name = "Assets"

    df_txt = df.copy()
    for c in df_txt.columns:
        df_txt[c] = df_txt[c].map(_fmt_ptbr_2dec)

    records = []
    for asset, row in df_txt.iterrows():
        rec = {"Assets": str(asset)}
        for col in df_txt.columns:
            val = row[col]
            rec[str(col)] = "" if (val is None or pd.isna(val)) else str(val)
        records.append(rec)

    return json.dumps(records, ensure_ascii=False)


# ==========================
# Payload / download do CSV
# ==========================

def montar_payload(data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Date"] = s
    p["FinalDate"] = s
    return p


def baixar_csv_b3(data: dt.date) -> str:
    start = time.monotonic()
    r = _session.post(
        URL,
        headers=HEADERS,
        json=montar_payload(data),
        timeout=(HTTP_CONNECT_TIMEOUT, HTTP_READ_TIMEOUT),
    )
    elapsed = time.monotonic() - start
    clen = r.headers.get("Content-Length", "?")
    _append_log(f"[HTTP] {B3_NAME} {data} -> status={r.status_code} content-length={clen} t={elapsed:.2f}s")

    if r.status_code != 200:
        raise RuntimeError(f"{data}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data}: CSV vazio")

    _dump_csv(data, r.content)
    md5 = hashlib.md5(r.content).hexdigest()
    _append_log(f"[HTTP] md5={md5} bytes={len(r.content)} ({B3_NAME} {data})")

    # decode
    txt = None
    for enc in ("utf-8-sig", "utf-8", "latin-1"):
        try:
            txt = r.content.decode(enc)
            break
        except UnicodeDecodeError:
            pass
    if txt is None:
        txt = r.content.decode("utf-8", errors="replace")

    _print_snippet(f"RAW {B3_NAME} {data}", txt)
    return txt


# ==========================
# Mapeamento de ativos
# ==========================

MONTH_CODE = {
    "F": 1, "G": 2, "H": 3, "J": 4, "K": 5, "M": 6,
    "N": 7, "Q": 8, "U": 9, "V": 10, "X": 11, "Z": 12
}


def maturity_date_from_venc(venc: str) -> dt.date | None:
    if not venc or len(venc) < 3:
        return None
    letra = venc[0].upper()
    yy = venc[-2:]
    if letra not in MONTH_CODE or not yy.isdigit():
        return None
    return dt.date(2000 + int(yy), MONTH_CODE[letra], 1)


def mapear_asset(name: str, venc: str) -> str | None:
    """
    - DAP (IPCACoupon):
        * ano par  -> manter só Q
        * ano ímpar -> manter só K
        -> Asset "DAPyy"
    - DI (DI1Day): manter ano >= 26
        -> Asset "DI_yy"
    - WDO/DOL -> "WDO1"
    - Treasury (T10 -> USTNOTEFuture) -> "TREASURY"
    """
    if not venc:
        return None

    venc = venc.strip().upper()

    if name == "IPCACoupon":
        if len(venc) < 3:
            return None
        letra = venc[0]
        ano_str = venc[-2:]
        try:
            ano = int(ano_str)
        except ValueError:
            return None

        if ano % 2 == 0:
            if letra != "Q":
                return None
        else:
            if letra != "K":
                return None

        return f"DAP{ano_str}"

    if name == "DI1Day":
        sufixo = venc[-2:]
        try:
            ano = int(sufixo)
        except ValueError:
            return None
        if not (26 <= ano):
            return None
        return f"DI_{sufixo}"

    if name in ("BusinessDollar", "WDOMiniFuture"):
        return "WDO1"

    if name == "USTNOTEFuture":
        return "TREASURY"

    return None


# ==========================
# Parsing do ConsolidatedTradesDerivatives
# ==========================

def parse_consolidated_trades(csv_text: str, data_ref: dt.date) -> pd.DataFrame:
    """
    Lê o CSV do ConsolidatedTradesDerivatives e devolve DF “long” já enxuto.

    Inclui: DI1 / DAP / WDO / DOL / T10 (Treasury)

    Colunas finais:
      - Instrumento
      - Vencimento (ex.: F26 / H26 / M26)
      - Name (DI1Day / IPCACoupon / WDOMiniFuture / BusinessDollar / USTNOTEFuture)
      - PrecoAjusteAtual (float)  <- coluna "Ajuste"
      - Pontos (float)            <- coluna "Variação"
      - ValorAjusteR$ (float)     <- coluna "Valor do ajuste por contrato (R$)"
      - Data_Referencia (datetime)
      - ValorIndiceDia (NA)
    """
    df_raw = pd.read_csv(
        io.StringIO(csv_text),
        sep=";",
        skiprows=2,
        dtype=str,
        engine="python",
        on_bad_lines="skip",
    )
    if df_raw is None or df_raw.empty:
        return pd.DataFrame(columns=[
            "Instrumento","Vencimento","Name",
            "PrecoAjusteAtual","Pontos","ValorAjusteR$",
            "Data_Referencia","ValorIndiceDia"
        ])

    df_raw = _normalize_missing_values_df(df_raw)

    colmap = {c: _strip_accents(c).lower() for c in df_raw.columns}

    def _find_col(*must_have):
        for orig, norm in colmap.items():
            if all(x in norm for x in must_have):
                return orig
        return None

    c_inst = _find_col("instrumento", "financeiro") or _find_col("instrumento")
    c_ajuste = _find_col("ajuste")
    c_var = _find_col("variacao", "ponto") or _find_col("variacao")
    c_val_adj = _find_col("valor", "ajuste", "contrato")

    if not all([c_inst, c_ajuste, c_var, c_val_adj]):
        _append_log(f"[parse] Colunas esperadas não encontradas. Vistas: {list(df_raw.columns)}")
        return pd.DataFrame(columns=[
            "Instrumento","Vencimento","Name",
            "PrecoAjusteAtual","Pontos","ValorAjusteR$",
            "Data_Referencia","ValorIndiceDia"
        ])

    df = df_raw[[c_inst, c_ajuste, c_var, c_val_adj]].copy()
    df.columns = ["Instrumento", "PrecoAjusteAtual", "Pontos", "ValorAjusteR$"]

    # Mantém apenas tickers “curtos” (6 chars) e prefixos relevantes (inclui T10)
    s = df["Instrumento"].astype(str)
    df = df[s.str.len().eq(6) & s.str.match(r"^(DI1|DAP|WDO|DOL|T10)", na=False)].copy()

    if df.empty:
        return pd.DataFrame(columns=[
            "Instrumento","Vencimento","Name",
            "PrecoAjusteAtual","Pontos","ValorAjusteR$",
            "Data_Referencia","ValorIndiceDia"
        ])

    # Vencimento = parte após o prefixo (3 chars): ex DI1F26 -> F26 / T10H26 -> H26
    df["Vencimento"] = df["Instrumento"].astype(str).str[3:]

    def _name_from_inst(x: str) -> str:
        if x.startswith("DI1"):
            return "DI1Day"
        if x.startswith("DAP"):
            return "IPCACoupon"
        if x.startswith("WDO"):
            return "WDOMiniFuture"
        if x.startswith("DOL"):
            return "BusinessDollar"
        if x.startswith("T10"):
            return "USTNOTEFuture"
        return "Other"

    df["Name"] = df["Instrumento"].astype(str).map(_name_from_inst)

    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].map(ptbr_to_float)
    df["Pontos"] = df["Pontos"].map(ptbr_to_float)
    df["ValorAjusteR$"] = df["ValorAjusteR$"].map(ptbr_to_float)

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["ValorIndiceDia"] = pd.NA

    df = df[(pd.notna(df["PrecoAjusteAtual"])) | (pd.notna(df["Pontos"])) | (pd.notna(df["ValorAjusteR$"]))].copy()

    return df[[
        "Instrumento","Vencimento","Name",
        "PrecoAjusteAtual","Pontos","ValorAjusteR$",
        "Data_Referencia","ValorIndiceDia"
    ]].reset_index(drop=True)


def selecionar_vertices(df_day: pd.DataFrame, data_ref: dt.date) -> pd.DataFrame:
    """
    - DI: mantém só DI1Fyy (Jan) por ano -> DI_yy
    - DAP: regra par/ímpar (Q/K) -> DAPyy
    - WDO1: escolhe 1 contrato (prefere WDO, senão DOL) com vencimento mais próximo (>= mês ref)
    - TREASURY: escolhe 1 contrato T10 com vencimento mais próximo (>= mês ref), igual WDO
    """
    if df_day is None or df_day.empty:
        return df_day

    df = df_day.copy()

    # DI: keep apenas mês "F" (Jan)
    di_mask = df["Name"].eq("DI1Day")
    df = df[~di_mask | df["Vencimento"].astype(str).str.startswith("F", na=False)].copy()

    # Mapeia Asset
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()].copy()

    ref_month = dt.date(data_ref.year, data_ref.month, 1)

    # Seleção do WDO1
    w = df[df["Asset"].eq("WDO1")].copy()
    if not w.empty:
        w["Prefix"] = w["Instrumento"].astype(str).str[:3]
        w["MatDate"] = w["Vencimento"].map(maturity_date_from_venc)

        w_pref = w[w["Prefix"].eq("WDO")].copy()
        if w_pref.empty:
            w_pref = w.copy()

        w_pref["MatDate2"] = w_pref["MatDate"].fillna(dt.date(2099, 1, 1))
        after = w_pref[w_pref["MatDate2"] >= ref_month]
        use = after if not after.empty else w_pref

        best = use.sort_values("MatDate2").head(1)
        df = pd.concat([df[df["Asset"].ne("WDO1")], best], ignore_index=True)

    # Seleção do TREASURY (T10)
    t = df[df["Asset"].eq("TREASURY")].copy()
    if not t.empty:
        t["MatDate"] = t["Vencimento"].map(maturity_date_from_venc)
        t["MatDate2"] = t["MatDate"].fillna(dt.date(2099, 1, 1))
        after = t[t["MatDate2"] >= ref_month]
        use = after if not after.empty else t

        best = use.sort_values("MatDate2").head(1)
        df = pd.concat([df[df["Asset"].ne("TREASURY")], best], ignore_index=True)

    return df.reset_index(drop=True)


# ==========================
# Base longa
# ==========================

LONG_COLS = [
    "Instrumento","Vencimento","Name",
    "PrecoAjusteAtual","Pontos","ValorAjusteR$",
    "Data_Referencia","ValorIndiceDia"
]


def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        base = pd.read_parquet(p)
        for c in LONG_COLS:
            if c not in base.columns:
                base[c] = pd.NA
        base = base[LONG_COLS].copy()
        return base
    return pd.DataFrame(columns=LONG_COLS)


def incrementar_base_ajuste(path_parquet: str, df_novo: pd.DataFrame,
                           chaves=("Data_Referencia","Name","Vencimento")) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)
    df_novo2 = df_novo.copy()
    for c in LONG_COLS:
        if c not in df_novo2.columns:
            df_novo2[c] = pd.NA
    df_novo2 = df_novo2[LONG_COLS].copy()

    df_comb = pd.concat([df_base, df_novo2], ignore_index=True) if not df_base.empty else df_novo2.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb


# ==========================
# Wides (preço e valor)
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")


def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]


def drop_tail_duplicate(df: pd.DataFrame) -> pd.DataFrame:
    """
    Se a última coluna for apenas uma cópia da penúltima e representar
    exatamente o próximo dia útil B3, remove essa coluna.
    """
    if df is None or df.empty or df.shape[1] < 2:
        return df

    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
    except Exception:
        return df

    last_dt = cols_dt[-1].date()
    prev_dt = cols_dt[-2].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")
    prev_col = cols_dt[-2].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(prev_dt, prev_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > prev_dt]
    if not prox_list:
        return df

    prox_dt = prox_list[0]
    if prox_dt != last_dt:
        return df

    try:
        if df[last_col].equals(df[prev_col]):
            _append_log(f"[drop-dup] Removendo coluna duplicada {last_col} (cópia de {prev_col})")
            return df.drop(columns=[last_col])
    except Exception:
        return df

    return df


def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")

    df = pd.read_parquet(path)

    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"

    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass

    df = df[~df.index.duplicated(keep="last")]
    df = drop_tail_duplicate(df)
    df = _normalize_missing_values_df(df)
    df = remover_assets_indesejados(df)
    return df


def salvar_wide(df: pd.DataFrame, path_parquet: str, path_csv: str, csv_ptbr_text: bool = True):
    df2 = df.copy()
    df2.index.name = "Assets"
    base = df2.reset_index().rename(columns={df2.reset_index().columns[0]: "Assets"})

    if csv_ptbr_text:
        out_txt = base.copy()

        cols_norm = []
        for c in out_txt.columns:
            if c == "Assets":
                cols_norm.append(c)
            else:
                try:
                    cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
                except Exception:
                    cols_norm.append(str(c))
        out_txt.columns = cols_norm

        for c in out_txt.columns:
            if c == "Assets":
                continue
            out_txt[c] = out_txt[c].map(_fmt_ptbr_2dec)

        out_txt.to_parquet(path_parquet, index=False)
        out_txt.to_csv(path_csv, index=False, encoding="utf-8")
    else:
        base.to_parquet(path_parquet, index=False)
        base.to_csv(path_csv, index=False, encoding="utf-8")


def adicionar_coluna_duplicada_final(wp: pd.DataFrame, wv: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    col_names = []
    if wp is not None and wp.shape[1] > 0:
        col_names.extend(list(wp.columns))
    if wv is not None and wv.shape[1] > 0:
        col_names.extend(list(wv.columns))

    if not col_names:
        _append_log("[dup-final] wp/wv vazios → nada para duplicar.")
        return wp, wv

    try:
        cols_dt = sorted(pd.to_datetime(col_names))
    except Exception:
        _append_log("[dup-final] Não consegui interpretar colunas como datas → não duplica.")
        return wp, wv

    last_dt = cols_dt[-1].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > last_dt]
    if not prox_list:
        _append_log(f"[dup-final] Não há próximo dia útil após {last_dt} → não duplica.")
        return wp, wv

    prox_dt = prox_list[0]
    prox_col = prox_dt.strftime("%Y-%m-%d")

    already_p = wp is not None and wp.shape[1] > 0 and prox_col in wp.columns
    already_v = wv is not None and wv.shape[1] > 0 and prox_col in wv.columns

    if already_p and already_v:
        _append_log(f"[dup-final] Coluna {prox_col} já existe em preços e valores → nenhuma duplicação feita.")
        return wp, wv

    if wp is not None and wp.shape[1] > 0 and not already_p and last_col in wp.columns:
        wp[prox_col] = wp[last_col]
        _append_log(f"[dup-final] (preço) Duplicado {last_col} -> {prox_col}")

    if wv is not None and wv.shape[1] > 0 and not already_v and last_col in wv.columns:
        wv[prox_col] = wv[last_col]
        _append_log(f"[dup-final] (valor) Duplicado {last_col} -> {prox_col}")

    return wp, wv


def construir_colunas_wide_duplas(df_long_dia: pd.DataFrame, data_ref: dt.date) -> tuple[pd.Series, pd.Series]:
    """
    - s_preco: PrecoAjusteAtual (Ajuste)
    - s_valor: Pontos (Variação) para DI/WDO/TREASURY
      - DAP: usa ValorAjusteR$ (Valor do ajuste por contrato (R$))
    """
    df = selecionar_vertices(df_long_dia, data_ref)
    if df is None or df.empty:
        col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
        s_preco = pd.Series(dtype="float64", name=col_name)
        s_valor = pd.Series(dtype="float64", name=col_name)
        return s_preco, s_valor

    if "Asset" not in df.columns:
        df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
        df = df[df["Asset"].notna()].copy()

    df["MatDate"] = df["Vencimento"].map(maturity_date_from_venc)
    df = df.sort_values(["Asset", "MatDate", "Instrumento"], na_position="last").copy()

    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()
    s_valor = df.groupby("Asset")["Pontos"].first()

    dap_mask = df["Asset"].astype(str).str.startswith("DAP", na=False)
    if dap_mask.any():
        s_cash = df.loc[dap_mask].groupby("Asset")["ValorAjusteR$"].first()
        for a, v in s_cash.items():
            if pd.notna(v):
                s_valor.loc[a] = v

    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name
    return s_preco, s_valor


# ==========================
# Fetch com backoff (histórico) + “hoje” exato
# ==========================

def buscar_dia_com_backoff(target_d: dt.date) -> pd.DataFrame:
    validos_back = b3_valid_days(target_d - dt.timedelta(days=60), target_d)[::-1]
    tentativas_max = max(BACKOFF_LIM, 15)

    tentativa = 0
    for prev_d in [target_d] + validos_back:
        try:
            csv_text = baixar_csv_b3(prev_d)
            df_n = parse_consolidated_trades(csv_text, prev_d)
            if df_n is not None and not df_n.empty:
                df_n["Data_Referencia"] = pd.to_datetime(target_d)
                if prev_d != target_d:
                    _append_log(f"• {B3_NAME}: {target_d} vazio → usando {prev_d} (backfill)")
                return df_n
        except Exception as e:
            _append_log(f"! {B3_NAME} @ {prev_d}: falhou parse ({str(e)[:120]})")

        tentativa += 1
        if tentativa > tentativas_max:
            break

    _append_log(f"! {B3_NAME}: sem dados até {tentativas_max} DUs atrás para {target_d}")
    return pd.DataFrame(columns=LONG_COLS)


def buscar_dia_EXATO_sem_backfill(target_d: dt.date) -> pd.DataFrame:
    _append_log(f"[today-check] Coleta EXATA do dia {target_d} (sem backfill).")
    try:
        csv_text = baixar_csv_b3(target_d)
        df_n = parse_consolidated_trades(csv_text, target_d)
        if df_n is not None and not df_n.empty:
            df_n["Data_Referencia"] = pd.to_datetime(target_d)
            _append_log(f"[today-ok] Dados encontrados para {target_d}: {len(df_n)} linhas.")
            return df_n
    except Exception as e:
        _append_log(f"[today-empty] {target_d}: {e}")

    _append_log(f"[today-nodata] NENHUM dado em {target_d}.")
    return pd.DataFrame(columns=LONG_COLS)


# ==========================
# Calendário / range
# ==========================

def ultimo_dia_util_ANTES_de_hoje() -> dt.date:
    today = dt.date.today()
    v = b3_calendar().valid_days(today - dt.timedelta(days=20), today - dt.timedelta(days=1))
    return v[-1].date()


# ==========================
# Pipeline principal
# ==========================

def main():
    wide_preco = ler_wide(PATH_PRECO)
    wide_valor = ler_wide(PATH_VALOR)

    wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
    wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

    def _last_col_date(df):
        if df is None or df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)

    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)

    end_dt_hist = ultimo_dia_util_ANTES_de_hoje()
    dias_util = b3_valid_days(start_dt, end_dt_hist)

    _append_log(f"Atualizando de {start_dt} até {end_dt_hist} (DU B3: {len(dias_util)})")

    # 2) histórico (com backfill)
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref)
        if df_dia_all.empty:
            _append_log(f"{dref}: nenhum dado disponível — mantendo (sem atualização).")
            continue

        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)
        _append_log(f"{dref}: base longa atualizada; total linhas = {len(df_long)}.")

        s_preco, s_valor = construir_colunas_wide_duplas(df_dia_all, dref)

        if wide_preco.empty:
            wide_preco = pd.DataFrame(s_preco)
        else:
            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_preco[s_preco.name] = s_preco

        if wide_valor.empty:
            wide_valor = pd.DataFrame(s_valor)
        else:
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_valor[s_valor.name] = s_valor

        wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
        wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

        _append_log(f"{dref}: preços/valores atualizados.")

    # 3) tentar “hoje” (mantive sua lógica: pega ontem)
    today = dt.date.today() - dt.timedelta(days=1)
    is_today_du = today in set(b3_valid_days(today, today))

    if is_today_du:
        df_today = buscar_dia_EXATO_sem_backfill(today)
        if not df_today.empty:
            df_long = incrementar_base_ajuste(PATH_LONG, df_today)
            _append_log(f"{today}: base longa atualizada (hoje); total linhas = {len(df_long)}.")

            s_preco, s_valor = construir_colunas_wide_duplas(df_today, today)

            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_preco[s_preco.name] = s_preco
            wide_valor[s_valor.name] = s_valor

            wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
            wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

            _append_log(f"{today}: preços/valores de HOJE adicionados.")
        else:
            _append_log(f"{today}: sem dados de HOJE — segue sem coluna de hoje (duplicação será feita no export final).")
    else:
        _append_log(f"{today}: não é dia útil B3 — não tenta hoje.")

    # 4) ordenar colunas
    def _order(df):
        if df is None or df.empty:
            return df
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    # 5) cria a coluna do próximo DU como cópia da última real, se ainda não existir
    wide_preco, wide_valor = adicionar_coluna_duplicada_final(wide_preco, wide_valor)

    # 6) normalize missing e remove assets indesejados no output FINAL
    wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
    wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO, PATH_PRECO_CSV, csv_ptbr_text=True)
    salvar_wide(wide_valor, PATH_VALOR, PATH_VALOR_CSV, csv_ptbr_text=True)
    _append_log(f"Salvos: {PATH_PRECO} {wide_preco.shape} | {PATH_VALOR} {wide_valor.shape}")

    # 7) JSON pt-BR (texto garantido) — missing vira ""
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        _append_log(f"Salvo JSON pt-BR de preços (texto): {PATH_JSON}")
    except Exception as e:
        _append_log(f"[warn] Falha ao gerar JSON pt-BR: {e}")


if __name__ == "__main__":
    main()

[2025-12-23 11:09:00] [drop-dup] Removendo coluna duplicada 2025-12-22 (cópia de 2025-12-19)
[2025-12-23 11:09:00] [drop-dup] Removendo coluna duplicada 2025-12-22 (cópia de 2025-12-19)
[2025-12-23 11:09:00] Atualizando de 2025-12-19 até 2025-12-22 (DU B3: 2)
[2025-12-23 11:09:01] [HTTP] ConsolidatedTradesDerivatives 2025-12-19 -> status=200 content-length=108296 t=0.47s
    [dump] CSV bruto salvo em: debug_b3_csv\ConsolidatedTradesDerivatives_2025-12-19.csv
[2025-12-23 11:09:01] [HTTP] md5=24862a32448d92a3d7165c40c1af169a bytes=526849 (ConsolidatedTradesDerivatives 2025-12-19)
----[ RAW ConsolidatedTradesDerivatives 2025-12-19 | primeiras 40 de 6879 linhas ]----
Resume as transações realizadas em derivativos durante um pregão, incluindo volumes e valores.

Instrumento financeiro;Código ISIN;Segmento;Preço de abertura;Preço mínimo;Preço máximo;Preço médio;Preço de fechamento;Oscilação;Ajuste;Ajuste de referência;Preço de referência;Variação;Valor do ajuste por contrato (R$);Última ofer

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_9708\3537849035.py:563: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comb = pd.concat([df_base, df_novo2], ignore_index=True) if not df_base.empty else df_novo2.copy()


[2025-12-23 11:09:01] 2025-12-19: base longa atualizada; total linhas = 2925.
[2025-12-23 11:09:01] 2025-12-19: preços/valores atualizados.
[2025-12-23 11:09:02] [HTTP] ConsolidatedTradesDerivatives 2025-12-22 -> status=200 content-length=105765 t=0.36s
    [dump] CSV bruto salvo em: debug_b3_csv\ConsolidatedTradesDerivatives_2025-12-22.csv
[2025-12-23 11:09:02] [HTTP] md5=7893ddf18b53609e743f91a0e8284179 bytes=517881 (ConsolidatedTradesDerivatives 2025-12-22)
----[ RAW ConsolidatedTradesDerivatives 2025-12-22 | primeiras 40 de 6765 linhas ]----
Resume as transações realizadas em derivativos durante um pregão, incluindo volumes e valores.

Instrumento financeiro;Código ISIN;Segmento;Preço de abertura;Preço mínimo;Preço máximo;Preço médio;Preço de fechamento;Oscilação;Ajuste;Ajuste de referência;Preço de referência;Variação;Valor do ajuste por contrato (R$);Última oferta de compra;Última oferta de venda;Quantidade de negócios;Quantidade de contratos;Volume financeiro
BGIZ25C029000;BRBME

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_9708\3537849035.py:563: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comb = pd.concat([df_base, df_novo2], ignore_index=True) if not df_base.empty else df_novo2.copy()


[2025-12-23 11:09:02] 2025-12-22: base longa atualizada; total linhas = 3036.
[2025-12-23 11:09:02] 2025-12-22: preços/valores atualizados.
[2025-12-23 11:09:02] [today-check] Coleta EXATA do dia 2025-12-22 (sem backfill).
[2025-12-23 11:09:03] [HTTP] ConsolidatedTradesDerivatives 2025-12-22 -> status=200 content-length=105765 t=0.50s
    [dump] CSV bruto salvo em: debug_b3_csv\ConsolidatedTradesDerivatives_2025-12-22.csv
[2025-12-23 11:09:03] [HTTP] md5=7893ddf18b53609e743f91a0e8284179 bytes=517881 (ConsolidatedTradesDerivatives 2025-12-22)
----[ RAW ConsolidatedTradesDerivatives 2025-12-22 | primeiras 40 de 6765 linhas ]----
Resume as transações realizadas em derivativos durante um pregão, incluindo volumes e valores.

Instrumento financeiro;Código ISIN;Segmento;Preço de abertura;Preço mínimo;Preço máximo;Preço médio;Preço de fechamento;Oscilação;Ajuste;Ajuste de referência;Preço de referência;Variação;Valor do ajuste por contrato (R$);Última oferta de compra;Última oferta de venda;Q

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_9708\3537849035.py:563: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comb = pd.concat([df_base, df_novo2], ignore_index=True) if not df_base.empty else df_novo2.copy()


[2025-12-23 11:09:03] [dup-final] (preço) Duplicado 2025-12-22 -> 2025-12-23
[2025-12-23 11:09:03] [dup-final] (valor) Duplicado 2025-12-22 -> 2025-12-23
[2025-12-23 11:09:05] Salvos: df_preco_de_ajuste_atual_completo.parquet (41, 247) | df_valor_ajuste_contrato.parquet (30, 247)
[2025-12-23 11:09:05] Salvo JSON pt-BR de preços (texto): df_preco_de_ajuste_atual_completo.json


In [7]:
# inspect_b3_consolidated_trades.py
import io
import datetime as dt
import requests
import pandas as pd


URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"

HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv,application/json;q=0.9,*/*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}


def fetch_csv(name: str, date: dt.date, final_date: dt.date | None = None) -> bytes:
    if final_date is None:
        final_date = date

    payload = {
        "Name": name,
        "Date": date.strftime("%Y-%m-%d"),
        "FinalDate": final_date.strftime("%Y-%m-%d"),
        "ClientId": "",
        "Filters": {},
    }

    r = requests.post(URL, headers=HEADERS, json=payload, timeout=(5, 30))
    print(f"HTTP {r.status_code}")
    print("Content-Type:", r.headers.get("Content-Type"))
    print("Content-Length:", r.headers.get("Content-Length"))

    if r.status_code != 200:
        # mostra um pedaço do erro
        print("\n--- BODY (erro, início) ---")
        print(r.text[:800])
        raise SystemExit("Falhou no request (não retornou 200).")

    return r.content


def decode_csv(raw: bytes) -> str:
    # B3 geralmente vem bem com utf-8-sig; fallback latin-1
    for enc in ("utf-8-sig", "utf-8", "latin-1"):
        try:
            return raw.decode(enc)
        except UnicodeDecodeError:
            pass
    return raw.decode("utf-8", errors="replace")


def print_head(text: str, n_lines: int = 30) -> None:
    lines = text.splitlines()
    print(f"\n--- PRIMEIRAS {min(n_lines, len(lines))} / {len(lines)} LINHAS ---")
    for ln in lines[:n_lines]:
        print(ln)


def to_dataframe(text: str) -> pd.DataFrame:
    # No “Negócios consolidados do pregão…”, normalmente:
    # linha 0: título
    # linha 1: vazia
    # linha 2: cabeçalho
    # então skiprows=2
    return pd.read_csv(io.StringIO(text), sep=";", skiprows=2, dtype=str, engine="python")


if __name__ == "__main__":
    nome = "ConsolidatedTradesDerivatives"
    data = dt.date(2025, 12, 12)

    raw = fetch_csv(nome, data)
    out_file = f"{nome}_{data:%Y-%m-%d}.csv"
    with open(out_file, "wb") as f:
        f.write(raw)
    print("\nSalvo em:", out_file)

    text = decode_csv(raw)
    print_head(text, n_lines=25)

    df = to_dataframe(text)
    print("\n--- COLUNAS ---")
    print(df.columns.tolist())

    print("\n--- PREVIEW (5 linhas) ---")
    print(df.head(5))

    print("\n--- SHAPE ---")
    print(df.shape)


HTTP 200
Content-Type: text/csv
Content-Length: 119067

Salvo em: ConsolidatedTradesDerivatives_2025-12-12.csv

--- PRIMEIRAS 25 / 6793 LINHAS ---
Resume as transações realizadas em derivativos durante um pregão, incluindo volumes e valores.

Instrumento financeiro;Código ISIN;Segmento;Preço de abertura;Preço mínimo;Preço máximo;Preço médio;Preço de fechamento;Oscilação;Ajuste;Ajuste de referência;Preço de referência;Variação;Valor do ajuste por contrato (R$);Última oferta de compra;Última oferta de venda;Quantidade de negócios;Quantidade de contratos;Volume financeiro
DIFF29F31;BRBMEFDIF0L8;FINANCIAL;13,71;13,71;13,75;13,727;13,75;-1,11;-;-;-;-;-;-;-;4;4.430;266946248
DIFF31F33;BRBMEFDIF0M6;FINANCIAL;13,79;13,79;13,79;13,79;13,79;-0,5;-;-;-;-;-;-;-;1;35;1628580,25
DIFN27F28;BRBMEFDIF1K8;FINANCIAL;12,355;12,355;12,355;12,355;12,355;-2,9;-;-;-;-;-;-;-;2;582;46737800,34
DIFF29F30;BRBMEFDIF144;FINANCIAL;13,71;13,71;13,71;13,71;13,71;-1,89;-;-;-;-;-;-;-;1;160;10355665,55
DIFF28F29;BRBMEFDI

In [ ]:

# -*- coding: utf-8 -*-
"""
Pipeline B3 (BDI) — IPCACoupon/DI/Dólar/WDO/Treasury

Atualizações IMPORTANTES (para o "novo CSV exportável" via endpoint /bdi/table/export/csv):
- Parser genérico agora captura (quando existir) as colunas:
    • Ajuste (ou Preço de Ajuste Atual)
    • Variação em Pontos
    • Valor do Ajuste / Valor de Ajuste (R$)  <-- se vier no novo CSV, passa a ser usado para a "planilha" de VALOR
    • Preço de Ajuste Anterior (se houver)
    • Último Preço (se houver)
- Construção do wide "VALOR" passa a priorizar "ValorAjusteContrato" quando existir.
  Caso não exista, mantém fallback para "Pontos" (com regra especial para DAP).
- Base longa (parquet) foi expandida com novas colunas, mantendo compatibilidade com arquivos antigos.

Mantém:
- Parser especial do IPCACoupon (mais robusto ao header com/sem acento)
- Backoff histórico
- Tentativa do "dia exato"
- Export parquet/csv/json pt-BR
- Remove DI_25, DAP_25 e DAP25 do output final
- Substitui strings vazias "" por pd.NA
"""

import datetime as dt
import io, re, os, json, unicodedata, hashlib, csv, time
from copy import deepcopy
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests
import pandas_market_calendars as mcal
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ==========================
# 0) Parâmetros do DAP / IPCA
# ==========================

IPCA_SERIE_CODIGO   = "PRECOS12_IPCA12"
REAIS_POR_PONTO     = 0.00025     # R$ por ponto (coeficiente do contrato)
BACKOFF_LIM         = 15          # janela de backoff (dias ÚTEIS)

IPCA_PREVISTA_XLSX  = "TaxasInflacaoDiariaPrevisao.xlsx"

# Debug / inspeção
DEBUG_MAX_LINES = 40
DEBUG_DUMP_DIR  = "debug_b3_csv"  # use None para desativar

# arquivos (bases)
PATH_LONG       = "df_ajustes_b3.parquet"
PATH_PRECO      = "df_preco_de_ajuste_atual_completo.parquet"
PATH_VALOR      = "df_valor_ajuste_contrato.parquet"
PATH_JSON       = "df_preco_de_ajuste_atual_completo.json"   # JSON pt-BR (texto)
PATH_PRECO_CSV  = "df_preco_de_ajuste_atual_completo.csv"    # CSV pt-BR (texto)
PATH_VALOR_CSV  = "df_valor_ajuste_contrato.csv"             # CSV pt-BR (texto)
PATH_RUN_LOG    = "atualizacao_b3_log.txt"

# Assets a EXCLUIR do output final (parquet/csv/json)
ASSETS_EXCLUIR = ["DI_25", "DAP_25", "DAP25", "DI25"]

# Se o novo CSV trouxer "Valor do Ajuste", priorizar no wide_valor:
PREFERIR_VALOR_AJUSTE_CONTRATO = True

# ==========================
# HTTP — sessão com retries/timeouts
# ==========================

URL = "https://arquivos.b3.com.br/bdi/table/export/csv?lang=pt-BR"
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "text/csv, application/json;q=0.9, */*;q=0.8",
    "Origin": "https://arquivos.b3.com.br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}
PAYLOAD_BASE = {"Name": "IPCACoupon", "Date": "2025-11-10", "FinalDate": "2025-11-10", "ClientId": "", "Filters": {}}

NAMES = ["IPCACoupon", "DI1Day", "BusinessDollar", "WDOMiniFuture", "USTNOTEFuture"]

HTTP_CONNECT_TIMEOUT = 3.0   # seg
HTTP_READ_TIMEOUT    = 15.0  # seg (por tentativa)
HTTP_TOTAL_BUDGET    = 90.0  # seg (orçamento total para o dia “exato”)

RETRY_CFG = Retry(
    total=2,                      # 2 tentativas extras (3 no total)
    backoff_factor=0.6,           # 0.6s, 1.2s...
    status_forcelist=(500, 502, 503, 504),
    allowed_methods=frozenset(["POST"])
)

_session = requests.Session()
_adapter = HTTPAdapter(max_retries=RETRY_CFG, pool_connections=10, pool_maxsize=10)
_session.mount("https://", _adapter)
_session.mount("http://", _adapter)

# ==========================
# Helpers gerais + JSON pt-BR
# ==========================

try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("America/Sao_Paulo")
except Exception:
    _TZ = None


def _append_log(msg: str):
    ts = dt.datetime.now(_TZ).strftime("%Y-%m-%d %H:%M:%S") if _TZ else dt.datetime.now().isoformat(sep=" ", timespec="seconds")
    line = f"[{ts}] {msg}"
    print(line)
    try:
        with open(PATH_RUN_LOG, "a", encoding="utf-8") as f:
            f.write(line + "\n")
    except Exception:
        pass


def _normalize_missing_values_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Troca strings vazias/whitespace/"nan"/"none"/"null" por pd.NA (missing values).
    """
    if df is None or df.empty:
        return df

    df2 = df.replace(r"^\s*$", pd.NA, regex=True)
    df2 = df2.replace({"nan": pd.NA, "NaN": pd.NA, "none": pd.NA, "None": pd.NA, "null": pd.NA, "NULL": pd.NA})
    return df2


def remover_assets_indesejados(w: pd.DataFrame) -> pd.DataFrame:
    if w is None or w.empty:
        return w
    return w.drop(index=ASSETS_EXCLUIR, errors="ignore")


def _fmt_ptbr_2dec(x):
    """
    Converte número para string "pt-BR", ex:
    98252.84 -> "98.252,84"

    Missing -> pd.NA.
    """
    if x is None:
        return pd.NA
    if pd.isna(x):
        return pd.NA
    if isinstance(x, str) and x.strip() == "":
        return pd.NA

    try:
        v = float(x)
    except Exception:
        s = str(x)
        return pd.NA if s.strip() == "" else s

    s = f"{v:,.2f}"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")


def wide_to_ptbr_json_text(wide_df: pd.DataFrame) -> str:
    """
    Exporta wide como JSON com TODAS as células em string pt-BR.
    Missing vira "" no JSON.
    """
    if wide_df is None or wide_df.empty:
        return "[]"

    cols_norm = []
    for c in wide_df.columns:
        try:
            cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
        except Exception:
            cols_norm.append(str(c))

    df = wide_df.copy()
    df.columns = cols_norm
    df.index.name = "Assets"

    df_txt = df.copy()
    for c in df_txt.columns:
        df_txt[c] = df_txt[c].map(_fmt_ptbr_2dec)

    records = []
    for asset, row in df_txt.iterrows():
        rec = {"Assets": str(asset)}
        for col in df_txt.columns:
            val = row[col]
            rec[str(col)] = "" if (val is None or pd.isna(val)) else str(val)
        records.append(rec)

    return json.dumps(records, ensure_ascii=False)

# ==========================
#  IPCA Previsao Helpers
# ==========================

def carregar_ipca_prevista(path_xlsx: str = IPCA_PREVISTA_XLSX) -> pd.DataFrame:
    p = Path(path_xlsx)
    if not p.exists():
        msg = f"Arquivo de inflação prevista não encontrado: {path_xlsx}"
        _append_log(msg)
        raise SystemExit(msg)

    try:
        df = pd.read_excel(p, sheet_name="Teste")
    except Exception as e:
        msg = f"Falha ao ler a aba 'Teste' em {path_xlsx}: {e}"
        _append_log(msg)
        raise SystemExit(msg)

    if "Datas" not in df.columns or "Projecao" not in df.columns:
        msg = f"Planilha {path_xlsx} (aba 'Teste') não possui colunas 'Datas' e 'Projecao'."
        _append_log(msg)
        raise SystemExit(msg)

    df = df[["Datas", "Projecao"]].copy()
    df.rename(columns={"Datas": "Data", "Projecao": "IPCA_Previsto"}, inplace=True)

    df["Data"] = pd.to_datetime(df["Data"], errors="coerce").dt.date
    df["IPCA_Previsto"] = pd.to_numeric(df["IPCA_Previsto"], errors="coerce")

    df = df.dropna(subset=["Data", "IPCA_Previsto"]).reset_index(drop=True)
    if df.empty:
        msg = f"Planilha {path_xlsx} não contém linhas válidas de Data/IPCA_Previsto."
        _append_log(msg)
        raise SystemExit(msg)

    _append_log(f"[IPCA_PREV] Carregado {len(df)} linhas de inflação prevista de {path_xlsx}")
    return df


def obter_ipca_prevista_para_data(df_prevista: pd.DataFrame, data_ref: dt.date) -> float:
    data_ref = pd.to_datetime(data_ref).date()
    linha = df_prevista.loc[df_prevista["Data"] == data_ref]
    if linha.empty:
        msg = f"Não há inflação prevista na planilha para a data {data_ref}."
        _append_log(msg)
        raise SystemExit(msg)

    ipca_prev = float(linha.iloc[0]["IPCA_Previsto"])
    _append_log(f"[IPCA_PREV] {data_ref}: IPCA previsto = {ipca_prev:.4f}%")
    return ipca_prev

# ==========================
# 1) Download do CSV na B3
# ==========================

def montar_payload(name: str, data: dt.date) -> dict:
    p = deepcopy(PAYLOAD_BASE)
    s = data.strftime("%Y-%m-%d")
    p["Name"], p["Date"], p["FinalDate"] = name, s, s
    return p


def _ensure_debug_dir():
    if DEBUG_DUMP_DIR:
        Path(DEBUG_DUMP_DIR).mkdir(parents=True, exist_ok=True)


def _dump_csv(name: str, data: dt.date, raw_bytes: bytes):
    if not DEBUG_DUMP_DIR:
        return
    _ensure_debug_dir()
    fn = Path(DEBUG_DUMP_DIR) / f"{name}_{data.strftime('%Y-%m-%d')}.csv"
    try:
        fn.write_bytes(raw_bytes)
        print(f"    [dump] CSV bruto salvo em: {fn}")
    except Exception as e:
        print(f"    [dump:fail] {e}")


def _print_snippet(tag: str, text: str, max_lines: int = DEBUG_MAX_LINES):
    lines = text.splitlines()
    header = f"----[ {tag} | primeiras {min(len(lines), max_lines)} de {len(lines)} linhas ]----"
    print(header)
    for ln in lines[:max_lines]:
        print(ln)
    print("-" * len(header))


def baixar_csv_bdi(name: str, data: dt.date) -> str:
    start = time.monotonic()
    r = _session.post(
        URL,
        headers=HEADERS,
        json=montar_payload(name, data),
        timeout=(HTTP_CONNECT_TIMEOUT, HTTP_READ_TIMEOUT),
    )
    elapsed = time.monotonic() - start
    clen = r.headers.get("Content-Length", "?")
    _append_log(f"[HTTP] {name} {data} -> status={r.status_code} content-length={clen} t={elapsed:.2f}s")
    if r.status_code != 200:
        raise RuntimeError(f"{data} / {name}: HTTP {r.status_code} - {r.text[:300]}")
    if not r.content:
        raise RuntimeError(f"{data} / {name}: CSV vazio")

    _dump_csv(name, data, r.content)
    md5 = hashlib.md5(r.content).hexdigest()
    _append_log(f"[HTTP] md5={md5} bytes={len(r.content)} ({name} {data})")

    txt = None
    for enc in ("utf-8-sig", "utf-8", "latin-1"):
        try:
            txt = r.content.decode(enc)
            break
        except UnicodeDecodeError:
            txt = None
    if txt is None:
        raise RuntimeError(f"{data} / {name}: falha ao decodificar")

    _print_snippet(f"RAW {name} {data}", txt)
    return txt

# ==========================
# 2) Parsing — genérico + especial IPCACoupon
# ==========================

def _strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')


def _norm_colname(c: str) -> str:
    return _strip_accents(str(c)).lower().strip()


def extrair_bloco_mercado_futuro(csv_text: str) -> str:
    """
    Tenta recortar o bloco da tabela principal, mais tolerante:
    basta achar um header que contenha 'vencimento' e 'ajuste' (ou 'preco'+'ajuste').
    """
    lines = csv_text.splitlines()
    start = None
    for i, line in enumerate(lines):
        l = _strip_accents(line).lower()
        if "vencimento" in l and ("ajuste" in l or ("preco" in l and "ajuste" in l)):
            # normalmente é o header da tabela
            start = i
            break
    if start is None:
        return csv_text

    block = [lines[start]]
    for line in lines[start+1:]:
        if not line.strip():
            break
        ll = line.strip()
        lln = _strip_accents(ll).lower()
        if lln.startswith("*") or lln.startswith("precos") or lln.startswith("preços"):
            break
        # corte defensivo: algumas seções posteriores começam sem ';'
        if ";" not in ll and "=" in ll:
            break
        block.append(line)
    return "\n".join(block)


def ptbr_to_float(s):
    """
    Converte string pt-BR para float.
    Retorna None para "", "-", None e strings vazias após limpeza.
    """
    if s is None:
        return None
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    if s in {"", "-"}:
        return None

    s = re.sub(r"[^0-9\-,\.]", "", s)
    s = s.strip()
    if s in {"", "-"}:
        return None

    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None


def parse_ajustes(csv_text: str, data_ref: dt.date, name: str) -> pd.DataFrame:
    """
    Parser genérico:
    - Vencimento
    - Ajuste (Preço de Ajuste Atual)
    - Variação em Pontos
    - (Opcional) Valor do Ajuste / Valor de Ajuste do Contrato (R$)
    - (Opcional) Preço de Ajuste Anterior
    - (Opcional) Último Preço
    """
    bloco = extrair_bloco_mercado_futuro(csv_text)
    target_text = bloco if ";" in bloco else csv_text

    if bloco is csv_text:
        _append_log(f"[parse] {name} {data_ref}: cabeçalho NÃO localizado — tentando CSV inteiro")
    else:
        first = bloco.splitlines()[0] if bloco else "<vazio>"
        _append_log(f"[parse] {name} {data_ref}: cabeçalho localizado -> {first}")

    _print_snippet(f"PARSER_TARGET {name} {data_ref}", target_text)

    df_raw = pd.read_csv(
        io.StringIO(target_text),
        sep=";",
        dtype=str,
        engine="python",
        on_bad_lines="skip"
    )
    if df_raw.empty or df_raw.shape[1] < 3:
        _print_snippet(f"PARSER_EMPTY_{name}_{data_ref}", target_text)
        raise ValueError("CSV sem estrutura reconhecível para 'Ajustes do Pregão'.")

    colmap = {c: _norm_colname(c) for c in df_raw.columns}
    _append_log(f"[parse] colunas normalizadas: {list(colmap.values())}")

    def _find_col(predicate):
        for orig, norm in colmap.items():
            if predicate(norm):
                return orig
        return None

    # Obrigatória: Vencimento
    c_venc = _find_col(lambda n: "venc" in n)

    # Variação em Pontos (pode não existir em alguns exports, então tratamos como opcional)
    c_var_pontos = _find_col(lambda n: ("vari" in n and "ponto" in n) or ("variacao" in n and "ponto" in n))

    # Ajuste / Preço de Ajuste Atual
    def _is_ajuste_atual(n: str) -> bool:
        # preferências: "preco ... ajuste ... atual" / "ajuste" simples
        if "ajuste" in n and "anterior" in n:
            return False
        if "ajuste" in n and "preco" in n and ("atual" in n or "corrig" in n):
            return True
        if n == "ajuste":
            return True
        # fallback: qualquer coluna contendo "ajuste" que não seja anterior
        if "ajuste" in n and "anterior" not in n:
            return True
        return False

    c_ajuste = _find_col(_is_ajuste_atual)

    # Preço de ajuste anterior (opcional)
    c_ajuste_ant = _find_col(lambda n: "ajuste" in n and "anterior" in n)

    # Último preço (opcional)
    c_ultimo_preco = _find_col(lambda n: "ultimo" in n and "preco" in n)

    # Valor do ajuste em R$ (opcional) — nomes comuns no export
    def _is_valor_ajuste(n: str) -> bool:
        if "valor" in n and "ajuste" in n:
            return True
        if "ajuste" in n and ("r$" in n or "reais" in n):
            return True
        if "valor" in n and ("vari" in n) and ("r$" in n or "reais" in n):
            return True
        return False

    c_valor_ajuste = _find_col(_is_valor_ajuste)

    _append_log(
        f"[parse] mapeadas -> Venc:{c_venc}  VarPts:{c_var_pontos}  Ajuste:{c_ajuste}  "
        f"ValAjuste:{c_valor_ajuste}  AjusteAnt:{c_ajuste_ant}  Ultimo:{c_ultimo_preco}"
    )

    if c_venc is None or c_ajuste is None:
        _print_snippet(f"PARSER_HDR_MISSING_{name}_{data_ref}", "\n".join(df_raw.columns.astype(str)))
        raise ValueError(f"Colunas mínimas não encontradas (Vencimento/Ajuste). Cabeçalhos vistos: {list(df_raw.columns)}")

    cols = [c_venc, c_ajuste]
    if c_var_pontos:
        cols.append(c_var_pontos)
    if c_valor_ajuste:
        cols.append(c_valor_ajuste)
    if c_ajuste_ant:
        cols.append(c_ajuste_ant)
    if c_ultimo_preco:
        cols.append(c_ultimo_preco)

    df = df_raw[cols].copy()

    rename = {c_venc: "Vencimento", c_ajuste: "PrecoAjusteAtual"}
    if c_var_pontos:
        rename[c_var_pontos] = "Pontos"
    if c_valor_ajuste:
        rename[c_valor_ajuste] = "ValorAjusteContrato"
    if c_ajuste_ant:
        rename[c_ajuste_ant] = "PrecoAjusteAnterior"
    if c_ultimo_preco:
        rename[c_ultimo_preco] = "UltimoPreco"

    df.rename(columns=rename, inplace=True)

    # tipos numéricos (quando col existe)
    if "Pontos" in df.columns:
        df["Pontos"] = df["Pontos"].apply(ptbr_to_float)
    else:
        df["Pontos"] = pd.NA

    df["PrecoAjusteAtual"] = df["PrecoAjusteAtual"].apply(ptbr_to_float)

    if "ValorAjusteContrato" in df.columns:
        df["ValorAjusteContrato"] = df["ValorAjusteContrato"].apply(ptbr_to_float)
    else:
        df["ValorAjusteContrato"] = pd.NA

    if "PrecoAjusteAnterior" in df.columns:
        df["PrecoAjusteAnterior"] = df["PrecoAjusteAnterior"].apply(ptbr_to_float)
    else:
        df["PrecoAjusteAnterior"] = pd.NA

    if "UltimoPreco" in df.columns:
        df["UltimoPreco"] = df["UltimoPreco"].apply(ptbr_to_float)
    else:
        df["UltimoPreco"] = pd.NA

    # garante ValorIndiceDia na base longa
    df["ValorIndiceDia"] = pd.NA

    _append_log(f"[parse] {name} {data_ref} — linhas antes do filtro: {len(df)}")
    # critério: precisa ter Ajuste OU (Pontos) OU (ValorAjusteContrato)
    mask_ok = df["PrecoAjusteAtual"].notna()
    if "Pontos" in df.columns:
        mask_ok = mask_ok | df["Pontos"].notna()
    if "ValorAjusteContrato" in df.columns:
        mask_ok = mask_ok | df["ValorAjusteContrato"].notna()

    df = df[mask_ok].copy()
    _append_log(f"[parse] {name} {data_ref} — linhas após filtro: {len(df)}")
    if df.empty:
        raise ValueError("Após limpeza, não há linhas com Ajuste/Pontos/Valor.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df.reset_index(drop=True, inplace=True)

    # ordem consistente
    out_cols = [
        "Vencimento",
        "Pontos",
        "PrecoAjusteAtual",
        "Data_Referencia",
        "Name",
        "ValorIndiceDia",
        "UltimoPreco",
        "PrecoAjusteAnterior",
        "ValorAjusteContrato",
    ]
    for c in out_cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df[out_cols]

# ---------- PARSER ESPECIAL IPCACoupon ----------
_PT_BR_NUM = re.compile(r'^-?\d{1,3}(\.\d{3})*(,\d+)?$')


def _ipc_to_float_ptbr(s: str):
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s in {"-", "–", "—"}:
        return None
    s = s.replace("↑", "").replace("↓", "").strip()
    if s == "":
        return None

    if _PT_BR_NUM.match(s):
        s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None


def _ipc_clean_text(t: str) -> str:
    return (t or "").replace("\ufeff", "").replace("\xa0", " ").replace("\r\n", "\n").replace("\r", "\n")


def _ipc_slice_table_block(text: str) -> tuple[str, str, float | None]:
    txt = _ipc_clean_text(text)

    # header tolerante
    m_head = re.search(r'(?mi)^Vencimento;.*;Ajuste;.*$', txt)
    if not m_head:
        m_head = re.search(r'(?mi)^Vencimento;.*;.*Ajuste.*$', txt)
    if not m_head:
        raise ValueError("Cabeçalho 'Vencimento;' não encontrado no CSV da B3 (IPCACoupon).")

    start = m_head.start()
    m_end = re.search(r'(?m)^\*.*$', txt)
    end = m_end.start() if m_end else len(txt)

    bloco = txt[start:end].strip()
    m_resumo = re.search(r'(?mi)^(?:[FGHJKMNQUVXZ]\d{2}=\d{1,3}\.\d{3},\d{2}\s*)+', txt)
    linha_resumo = m_resumo.group(0).strip() if m_resumo else ""

    m_ind = re.search(r'Valor\s+Índice\s+Ipca\s+pro\s+Rata\s+Tempore:\s*([0-9\.\,]+)', txt, re.IGNORECASE)
    valor_indice_ipca = _ipc_to_float_ptbr(m_ind.group(1)) if m_ind else None

    return bloco, linha_resumo, valor_indice_ipca


def parse_ipcacoupon_special(csv_text: str, data_ref: dt.date, name: str = "IPCACoupon") -> pd.DataFrame:
    bloco, linha_resumo, valor_indice_ipca = _ipc_slice_table_block(csv_text)
    linhas = [ln for ln in bloco.split("\n") if ln.strip()]
    reader = csv.reader(linhas, delimiter=';')
    header = next(reader)

    # header normalizado
    cols_raw = [c.strip() for c in header]
    cols_norm = [_norm_colname(c).replace(" ", "") for c in cols_raw]

    def _idx(predicate):
        for i, n in enumerate(cols_norm):
            if predicate(n):
                return i
        return None

    idx_venc = _idx(lambda n: "venc" in n)
    idx_ult  = _idx(lambda n: "ultimo" in n and "preco" in n)
    idx_aj   = _idx(lambda n: n == "ajuste" or ("preco" in n and "ajuste" in n))
    idx_var  = _idx(lambda n: "vari" in n and "ponto" in n)

    if idx_venc is None or idx_aj is None:
        raise ValueError(f"IPCACoupon: não achei colunas mínimas no header: {cols_raw!r}")

    rows = []
    cod_venc_pat = re.compile(r'^[FGHJKMNQUVXZ]\d{2}$')

    for r in reader:
        if not r or len(r) <= idx_aj:
            continue

        venc = (r[idx_venc] or "").strip()
        if not cod_venc_pat.match(venc):
            continue

        ultimo_preco = _ipc_to_float_ptbr(r[idx_ult]) if idx_ult is not None and idx_ult < len(r) else None
        ajuste       = _ipc_to_float_ptbr(r[idx_aj]) if idx_aj is not None and idx_aj < len(r) else None
        pontos       = _ipc_to_float_ptbr(r[idx_var]) if idx_var is not None and idx_var < len(r) else None

        rows.append({
            "Vencimento": venc,
            "Pontos": pontos,
            "PrecoAjusteAtual": ajuste,
            "UltimoPreco": ultimo_preco,
        })

    df = pd.DataFrame(rows).sort_values("Vencimento", ignore_index=True)

    # Linha-resumo (fallback de Ajuste)
    if linha_resumo:
        ajustes_resumo = {}
        for token in linha_resumo.split():
            if "=" in token:
                k, v = token.split("=", 1)
                k, v = k.strip(), v.strip().rstrip(";")
                fv = _ipc_to_float_ptbr(v)
                if cod_venc_pat.match(k) and fv is not None:
                    ajustes_resumo[k] = fv
        if not df.empty and ajustes_resumo:
            df["PrecoAjusteAtual"] = df.apply(
                lambda x: x["PrecoAjusteAtual"] if pd.notnull(x["PrecoAjusteAtual"]) else ajustes_resumo.get(x["Vencimento"]),
                axis=1
            )

    df = df[(df["Pontos"].notna()) | (df["PrecoAjusteAtual"].notna())].copy()
    if df.empty:
        raise ValueError("IPCACoupon: após limpeza, não há linhas com Pontos ou Ajuste.")

    df["Data_Referencia"] = pd.to_datetime(data_ref)
    df["Name"] = name
    df["ValorIndiceDia"] = valor_indice_ipca

    # campos extras (mantém compatibilidade com parser genérico)
    df["PrecoAjusteAnterior"] = pd.NA
    df["ValorAjusteContrato"] = pd.NA

    df.reset_index(drop=True, inplace=True)
    return df[
        [
            "Vencimento",
            "Pontos",
            "PrecoAjusteAtual",
            "Data_Referencia",
            "Name",
            "ValorIndiceDia",
            "UltimoPreco",
            "PrecoAjusteAnterior",
            "ValorAjusteContrato",
        ]
    ]

# ==========================
# 3) IPCA helpers (IPEA)
# ==========================

def carregar_ipca_ipeadata() -> pd.DataFrame:
    url = f"https://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{IPCA_SERIE_CODIGO}')"
    resp = requests.get(url, timeout=(HTTP_CONNECT_TIMEOUT, HTTP_READ_TIMEOUT))
    resp.raise_for_status()
    df = pd.DataFrame(resp.json()["value"])
    df["VALDATA"] = pd.to_datetime(df["VALDATA"].astype(str).str[:10], errors="coerce")
    df = df[["VALDATA", "VALVALOR"]].sort_values("VALDATA").reset_index(drop=True)
    return df.rename(columns={"VALDATA": "Data", "VALVALOR": "IPCA_Indice"})


def obter_ipca_ref(df_ipca: pd.DataFrame, data_ref: dt.date) -> float:
    serie = df_ipca[df_ipca["Data"] <= pd.to_datetime(data_ref)]
    if serie.empty:
        raise ValueError(f"Sem IPCA até {data_ref}")
    return float(serie.iloc[-1]["IPCA_Indice"])


def proximo_dia_util_simples(d: dt.date) -> dt.date:
    while d.weekday() >= 5:
        d += dt.timedelta(days=1)
    return d


def datas_ipca_referencia(data_ref: dt.date) -> tuple[dt.date, dt.date]:
    if data_ref.day >= 15:
        prev = dt.date(data_ref.year, data_ref.month, 15)
        nxt_m = 1 if data_ref.month == 12 else data_ref.month + 1
        nxt_y = data_ref.year + 1 if data_ref.month == 12 else data_ref.year
        nxt = dt.date(nxt_y, nxt_m, 15)
    else:
        pm = 12 if data_ref.month == 1 else data_ref.month - 1
        py = data_ref.year - 1 if data_ref.month == 1 else data_ref.year
        prev = dt.date(py, pm, 15)
        nxt = dt.date(data_ref.year, data_ref.month, 15)
    return prev, nxt


def calcular_valor_ponto_dap_para_data(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    ipca_previsto: float,
    reais_por_ponto: float = REAIS_POR_PONTO,
) -> float:
    data_ref = pd.to_datetime(data_ref).date()
    prev_15, next_15 = datas_ipca_referencia(data_ref)
    prev_adj = proximo_dia_util_simples(prev_15)
    next_adj = proximo_dia_util_simples(next_15)
    du_desde = len(pd.bdate_range(prev_adj, data_ref)) - 1
    du_entre = len(pd.bdate_range(prev_adj, next_adj)) - 1
    if du_entre <= 0:
        raise ValueError(f"DU_entre <= 0 entre {prev_adj} e {next_adj}")
    ni_ref = obter_ipca_ref(df_ipca, prev_adj)
    ipca_pro_rata = ni_ref * (1 + ipca_previsto / 100) ** (du_desde / du_entre)
    return ipca_pro_rata * reais_por_ponto

# ==========================
# 4) Base longa
# ==========================

LONG_COLS = [
    "Vencimento",
    "Pontos",
    "PrecoAjusteAtual",
    "Data_Referencia",
    "Name",
    "ValorIndiceDia",
    "UltimoPreco",
    "PrecoAjusteAnterior",
    "ValorAjusteContrato",
]


def carregar_base_parquet_long(path_parquet: str) -> pd.DataFrame:
    p = Path(path_parquet)
    if p.exists():
        df = pd.read_parquet(p)
        # compatibilidade: adiciona colunas novas se faltarem
        for c in LONG_COLS:
            if c not in df.columns:
                df[c] = pd.NA
        return df[LONG_COLS]
    return pd.DataFrame(columns=LONG_COLS)


def incrementar_base_ajuste(
    path_parquet: str,
    df_novo: pd.DataFrame,
    chaves=("Data_Referencia", "Name", "Vencimento"),
) -> pd.DataFrame:
    df_base = carregar_base_parquet_long(path_parquet)

    # garante colunas padrão
    for c in LONG_COLS:
        if c not in df_novo.columns:
            df_novo[c] = pd.NA
    df_novo = df_novo[LONG_COLS].copy()

    df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()
    df_comb = df_comb.drop_duplicates(subset=list(chaves), keep="last")
    df_comb.to_parquet(path_parquet, index=False)
    return df_comb

# ==========================
# 5) Wides (preço e valor)
# ==========================

def ler_wide(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame().rename_axis("Assets")

    df = pd.read_parquet(path)

    if "Assets" in df.columns:
        df = df.set_index("Assets")
    df.index.name = "Assets"

    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
        df = df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
    except Exception:
        pass

    df = df[~df.index.duplicated(keep="last")]
    df = drop_tail_duplicate(df)
    df = _normalize_missing_values_df(df)
    df = remover_assets_indesejados(df)
    return df


def adicionar_coluna_duplicada_final(
    wp: pd.DataFrame,
    wv: pd.DataFrame,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    col_names = []
    if wp is not None and wp.shape[1] > 0:
        col_names.extend(list(wp.columns))
    if wv is not None and wv.shape[1] > 0:
        col_names.extend(list(wv.columns))

    if not col_names:
        _append_log("[dup-final] wp/wv vazios → nada para duplicar.")
        return wp, wv

    try:
        cols_dt = sorted(pd.to_datetime(col_names))
    except Exception:
        _append_log("[dup-final] Não consegui interpretar colunas como datas → não duplica.")
        return wp, wv

    last_dt = cols_dt[-1].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(last_dt, last_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > last_dt]
    if not prox_list:
        _append_log(f"[dup-final] Não há próximo dia útil após {last_dt} → não duplica.")
        return wp, wv

    prox_dt = prox_list[0]
    prox_col = prox_dt.strftime("%Y-%m-%d")

    already_p = wp is not None and wp.shape[1] > 0 and prox_col in wp.columns
    already_v = wv is not None and wv.shape[1] > 0 and prox_col in wv.columns

    if already_p and already_v:
        _append_log(f"[dup-final] Coluna {prox_col} já existe em preços e valores → nenhuma duplicação feita.")
        return wp, wv

    if wp is not None and wp.shape[1] > 0 and not already_p and last_col in wp.columns:
        wp[prox_col] = wp[last_col]
        _append_log(f"[dup-final] (preço) Duplicado {last_col} -> {prox_col}")

    if wv is not None and wv.shape[1] > 0 and not already_v and last_col in wv.columns:
        wv[prox_col] = wv[last_col]
        _append_log(f"[dup-final] (valor) Duplicado {last_col} -> {prox_col}")

    return wp, wv


def salvar_wide(df: pd.DataFrame, path_parquet: str, path_csv: str, csv_ptbr_text: bool = True):
    df2 = df.copy()
    df2.index.name = "Assets"
    base = df2.reset_index().rename(columns={df2.reset_index().columns[0]: "Assets"})

    if csv_ptbr_text:
        out_txt = base.copy()

        cols_norm = []
        for c in out_txt.columns:
            if c == "Assets":
                cols_norm.append(c)
            else:
                try:
                    cols_norm.append(pd.to_datetime(c).strftime("%Y-%m-%d"))
                except Exception:
                    cols_norm.append(str(c))
        out_txt.columns = cols_norm

        for c in out_txt.columns:
            if c == "Assets":
                continue
            out_txt[c] = out_txt[c].map(_fmt_ptbr_2dec)

        out_txt.to_parquet(path_parquet, index=False)
        out_txt.to_csv(path_csv, index=False, encoding="utf-8")
    else:
        base.to_parquet(path_parquet, index=False)
        base.to_csv(path_csv, index=False, encoding="utf-8")


def mapear_asset(name: str, venc: str) -> str | None:
    if not venc:
        return None

    venc = venc.strip().upper()

    # ---------- DAP (IPCACoupon) ----------
    if name == "IPCACoupon":
        if len(venc) < 3:
            return None

        letra = venc[0]
        ano_str = venc[-2:]
        try:
            ano = int(ano_str)
        except ValueError:
            return None

        if ano % 2 == 0:
            if letra != "Q":
                return None
        else:
            if letra != "K":
                return None

        return f"DAP{ano_str}"

    # ---------- DI (filtrar anos permitidos) ----------
    if name == "DI1Day":
        sufixo = venc[-2:]
        try:
            ano = int(sufixo)
        except ValueError:
            return None

        if not (26 <= ano):
            _append_log(f"[mapear_asset] Ignorando {name} venc={venc} (ano {ano} fora da faixa 26+)")
            return None

        return f"DI_{sufixo}"

    # ---------- Dólar / WDO ----------
    if name in ("BusinessDollar", "WDOMiniFuture"):
        return "WDO1"

    # ---------- Treasury ----------
    if name == "USTNOTEFuture":
        return "TREASURY"

    return None


def _valor_por_ponto_dap(
    df_ipca: pd.DataFrame,
    data_ref: dt.date,
    df_long_dia: pd.DataFrame,
    df_ipca_prevista: pd.DataFrame,
) -> float:
    vi = None
    try:
        vi = df_long_dia.loc[df_long_dia["Name"] == "IPCACoupon", "ValorIndiceDia"].dropna().iloc[0]
    except Exception:
        vi = None

    if vi is not None and not pd.isna(vi):
        vpp = REAIS_POR_PONTO * float(vi)
        _append_log(f"[DAP] Valor por ponto via arquivo B3: Índice={vi:.4f} -> R$ {vpp:.6f}")
        return vpp

    ipca_previsto = obter_ipca_prevista_para_data(df_ipca_prevista, data_ref)
    vpp = calcular_valor_ponto_dap_para_data(
        df_ipca=df_ipca,
        data_ref=data_ref,
        ipca_previsto=ipca_previsto,
        reais_por_ponto=REAIS_POR_PONTO,
    )
    _append_log(f"[DAP] Valor por ponto via fallback IPEA + planilha: R$ {vpp:.6f}")
    return vpp


def construir_colunas_wide_duplas(
    df_long_dia: pd.DataFrame,
    data_ref: dt.date,
    df_ipca: pd.DataFrame,
    df_ipca_prevista: pd.DataFrame,
) -> tuple[pd.Series, pd.Series]:
    df = df_long_dia.copy()
    df["Asset"] = [mapear_asset(n, v) for n, v in zip(df["Name"], df["Vencimento"])]
    df = df[df["Asset"].notna()].copy()

    # PREÇO (sempre do Ajuste / PrecoAjusteAtual)
    s_preco = df.groupby("Asset")["PrecoAjusteAtual"].first()

    # VALOR: prioriza ValorAjusteContrato se existir e tiver dados
    usar_valor_ajuste = (
        PREFERIR_VALOR_AJUSTE_CONTRATO
        and "ValorAjusteContrato" in df.columns
        and df["ValorAjusteContrato"].notna().any()
    )

    if usar_valor_ajuste:
        s_valor = df.groupby("Asset")["ValorAjusteContrato"].first()
        _append_log(f"[wide] {data_ref}: wide_valor usando 'ValorAjusteContrato' (novo CSV)")
        origem_valor = "ValorAjusteContrato"
    else:
        s_valor = df.groupby("Asset")["Pontos"].first()
        _append_log(f"[wide] {data_ref}: wide_valor usando 'Pontos' (fallback)")
        origem_valor = "Pontos"

    # Regra especial DAP: só converte Pontos -> R$ quando estamos usando Pontos
    if origem_valor == "Pontos" and not df[df["Name"] == "IPCACoupon"].empty:
        valor_ponto_dap = _valor_por_ponto_dap(
            df_ipca=df_ipca,
            data_ref=data_ref,
            df_long_dia=df_long_dia,
            df_ipca_prevista=df_ipca_prevista,
        )
        for k in list(s_valor.index):
            if str(k).startswith("DAP"):
                val = pd.to_numeric(s_valor.loc[k], errors="coerce")
                if pd.notna(val):
                    s_valor.loc[k] = float(val) * valor_ponto_dap

    col_name = pd.to_datetime(data_ref).strftime("%Y-%m-%d")
    s_preco.name = col_name
    s_valor.name = col_name
    return s_preco, s_valor

# ==========================
# 6) Calendário B3 + backoff
# ==========================

def b3_calendar():
    return mcal.get_calendar("B3")


def b3_valid_days(start: dt.date, end: dt.date) -> list[dt.date]:
    v = b3_calendar().valid_days(start, end)
    return [d.date() for d in v]


def ultimo_dia_util_ANTES_de_hoje() -> dt.date:
    today = dt.date.today()
    v = b3_calendar().valid_days(today - dt.timedelta(days=20), today - dt.timedelta(days=1))
    return v[-1].date()

# ==========================
# 7) Fetch genérico com backoff (para dias históricos)
# ==========================

def buscar_dia_com_backoff(target_d: dt.date, df_ipca: pd.DataFrame) -> pd.DataFrame:
    dfs = []
    validos_back = b3_valid_days(target_d - dt.timedelta(days=60), target_d)[::-1]
    tentativas_max = max(BACKOFF_LIM, 15)

    for name in NAMES:
        ok = False
        tentativa = 0
        for prev_d in [target_d] + validos_back:
            try:
                csv_text = baixar_csv_bdi(name, prev_d)
                if name == "IPCACoupon":
                    try:
                        df_n = parse_ipcacoupon_special(csv_text, prev_d, name)
                    except Exception as e_special:
                        _append_log(f"[warn] IPCACoupon parser especial falhou ({e_special}); usando genérico.")
                        df_n = parse_ajustes(csv_text, prev_d, name)
                else:
                    df_n = parse_ajustes(csv_text, prev_d, name)

                if df_n is not None and not df_n.empty:
                    df_n["Data_Referencia"] = pd.to_datetime(target_d)
                    dfs.append(df_n)
                    ok = True
                    if prev_d != target_d:
                        _append_log(f"• {name}: {target_d} vazio → usando {prev_d} (backfill)")
                    break
            except Exception as e:
                _append_log(f"! {name} @ {prev_d}: falhou parse ({str(e)[:200]})")
            tentativa += 1
            if tentativa > tentativas_max:
                break

        if not ok:
            _append_log(f"! {name}: sem dados até {tentativas_max} DUs atrás para {target_d}")

    if dfs:
        df_all = pd.concat(dfs, ignore_index=True)
        # garante colunas padrão
        for c in LONG_COLS:
            if c not in df_all.columns:
                df_all[c] = pd.NA
        return df_all[LONG_COLS]

    return pd.DataFrame(columns=LONG_COLS)

# ==========================
# 8) “Dia exato” — coleta paralela com orçamento total (sem backfill)
# ==========================

def _try_parse_for_name(name: str, d: dt.date) -> pd.DataFrame:
    csv_text = baixar_csv_bdi(name, d)
    if name == "IPCACoupon":
        try:
            df_n = parse_ipcacoupon_special(csv_text, d, name)
        except Exception as e_special:
            _append_log(f"[warn] IPCACoupon especial falhou ({e_special}); usando genérico.")
            df_n = parse_ajustes(csv_text, d, name)
    else:
        df_n = parse_ajustes(csv_text, d, name)
    return df_n


def _fetch_one_name_exact(name: str, d: dt.date) -> pd.DataFrame | None:
    try:
        return _try_parse_for_name(name, d)
    except Exception as e:
        _append_log(f"[today-empty] {name} {d}: {e}")
        return None


def buscar_dia_EXATO_sem_backfill(target_d: dt.date) -> pd.DataFrame:
    _append_log(f"[today-check] Coleta EXATA do dia {target_d} (sem backfill).")
    dfs = []
    deadline = time.monotonic() + HTTP_TOTAL_BUDGET

    with ThreadPoolExecutor(max_workers=min(len(NAMES), 5)) as ex:
        futs = {ex.submit(_fetch_one_name_exact, n, target_d): n for n in NAMES}
        for fut in as_completed(futs):
            if time.monotonic() > deadline:
                _append_log("[today-timeout] Estourou orçamento total de tempo; seguindo sem esperar o restante.")
                break
            df_n = fut.result()
            if df_n is not None and not df_n.empty:
                dfs.append(df_n)

    if dfs:
        df_all = pd.concat(dfs, ignore_index=True)
        df_all["Data_Referencia"] = pd.to_datetime(target_d)
        for c in LONG_COLS:
            if c not in df_all.columns:
                df_all[c] = pd.NA
        _append_log(f"[today-ok] Dados encontrados para {target_d}: {len(df_all)} linhas.")
        return df_all[LONG_COLS]

    _append_log(f"[today-nodata] NENHUM instrumento com dados em {target_d}.")
    return pd.DataFrame(columns=LONG_COLS)

# ==========================
# 9) ffill em DAP25 e NTNB
# ==========================

def preencher_vazios_dap25_e_ntnb(wide_preco: pd.DataFrame) -> pd.DataFrame:
    if wide_preco is None or wide_preco.empty:
        return wide_preco

    wide_preco = _normalize_missing_values_df(wide_preco)

    idx = wide_preco.index.astype(str)
    targets = []

    if "DAP25" in wide_preco.index:
        targets.append("DAP25")

    ntb_mask = idx.str.contains("NTNB", case=False, na=False)
    targets.extend(list(wide_preco.index[ntb_mask]))

    seen = set()
    targets = [a for a in targets if not (a in seen or seen.add(a))]

    for asset in targets:
        if asset not in wide_preco.index:
            continue
        s = wide_preco.loc[asset].copy()
        s = _normalize_missing_values_df(pd.DataFrame(s).T).iloc[0].ffill()
        wide_preco.loc[asset] = s

    return wide_preco


def drop_tail_duplicate(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty or df.shape[1] < 2:
        return df

    try:
        cols_dt = sorted(pd.to_datetime(df.columns))
    except Exception:
        return df

    last_dt = cols_dt[-1].date()
    prev_dt = cols_dt[-2].date()
    last_col = cols_dt[-1].strftime("%Y-%m-%d")
    prev_col = cols_dt[-2].strftime("%Y-%m-%d")

    prox_list = b3_valid_days(prev_dt, prev_dt + dt.timedelta(days=10))
    prox_list = [d for d in prox_list if d > prev_dt]
    if not prox_list:
        return df

    prox_dt = prox_list[0]
    if prox_dt != last_dt:
        return df

    try:
        if df[last_col].equals(df[prev_col]):
            _append_log(f"[drop-dup] Removendo coluna duplicada {last_col} (cópia de {prev_col})")
            return df.drop(columns=[last_col])
    except Exception:
        return df

    return df

# ==========================
# 10) Pipeline principal
# ==========================

def main():
    # 1) carregar IPCA (histórico), IPCA previsto (planilha) e bases existentes
    df_ipca = carregar_ipca_ipeadata()
    df_ipca_prevista = carregar_ipca_prevista()

    wide_preco = ler_wide(PATH_PRECO)
    wide_valor = ler_wide(PATH_VALOR)

    wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
    wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

    # 2) definir range de datas históricas (até ontem)
    def _last_col_date(df):
        if df is None or df.empty or len(df.columns) == 0:
            return None
        try:
            return sorted(pd.to_datetime(df.columns))[-1].date()
        except Exception:
            return None

    last_preco = _last_col_date(wide_preco)
    last_valor = _last_col_date(wide_valor)

    if last_preco is None and last_valor is None:
        start_dt = dt.date(2025, 1, 2)
    else:
        candidates = [d for d in [last_preco, last_valor] if d is not None]
        start_dt = max(candidates)

    end_dt_hist = ultimo_dia_util_ANTES_de_hoje()
    dias_util = b3_valid_days(start_dt, end_dt_hist)
    _append_log(f"Atualizando de {start_dt} até {end_dt_hist} (DU B3: {len(dias_util)})")

    # 3) dias históricos (com backfill)
    for dref in dias_util:
        df_dia_all = buscar_dia_com_backoff(dref, df_ipca)

        if df_dia_all.empty:
            _append_log(f"{dref}: nenhum dado disponível — mantendo planilhas (sem atualização).")
            continue

        df_long = incrementar_base_ajuste(PATH_LONG, df_dia_all)
        _append_log(f"{dref}: base longa atualizada; total linhas = {len(df_long)}.")

        s_preco, s_valor = construir_colunas_wide_duplas(
            df_long_dia=df_dia_all,
            data_ref=dref,
            df_ipca=df_ipca,
            df_ipca_prevista=df_ipca_prevista,
        )

        if wide_preco.empty:
            wide_preco = pd.DataFrame(s_preco)
        else:
            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_preco[s_preco.name] = s_preco

        if wide_valor.empty:
            wide_valor = pd.DataFrame(s_valor)
        else:
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_valor[s_valor.name] = s_valor

        wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
        wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

        _append_log(f"{dref}: preços/valores atualizados.")

    # 4) tentar "dia exato" (ontem): se houver dados, adiciona; se não houver, duplicação final resolve
    today = dt.date.today() - dt.timedelta(days=1)
    is_today_du = today in set(b3_valid_days(today, today))

    if is_today_du:
        df_today = buscar_dia_EXATO_sem_backfill(today)
        if not df_today.empty:
            df_long = incrementar_base_ajuste(PATH_LONG, df_today)
            _append_log(f"{today}: base longa atualizada (dia exato); total linhas = {len(df_long)}.")

            s_preco, s_valor = construir_colunas_wide_duplas(
                df_long_dia=df_today,
                data_ref=today,
                df_ipca=df_ipca,
                df_ipca_prevista=df_ipca_prevista,
            )

            wide_preco = wide_preco.reindex(wide_preco.index.union(s_preco.index))
            wide_valor = wide_valor.reindex(wide_valor.index.union(s_valor.index))
            wide_preco[s_preco.name] = s_preco
            wide_valor[s_valor.name] = s_valor

            wide_preco = remover_assets_indesejados(_normalize_missing_values_df(wide_preco))
            wide_valor = remover_assets_indesejados(_normalize_missing_values_df(wide_valor))

            _append_log(f"{today}: preços/valores do dia exato adicionados.")
        else:
            _append_log(f"{today}: sem dados do dia exato — segue sem coluna (duplicação será feita no export final).")
    else:
        _append_log(f"{today}: não é dia útil B3 — não tenta dia exato.")

    # 5) ordenar colunas, adicionar coluna duplicada final e salvar
    def _order(df):
        if df is None or df.empty:
            return df
        try:
            cols_dt = sorted(pd.to_datetime(df.columns))
            return df[[c.strftime("%Y-%m-%d") for c in cols_dt]]
        except Exception:
            return df

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    wide_preco, wide_valor = adicionar_coluna_duplicada_final(wide_preco, wide_valor)

    wide_preco = _normalize_missing_values_df(wide_preco)
    wide_preco = preencher_vazios_dap25_e_ntnb(wide_preco)

    wide_preco = _order(wide_preco)
    wide_valor = _order(wide_valor)

    wide_preco = remover_assets_indesejados(wide_preco)
    wide_valor = remover_assets_indesejados(wide_valor)

    salvar_wide(wide_preco, PATH_PRECO, PATH_PRECO_CSV, csv_ptbr_text=True)
    salvar_wide(wide_valor, PATH_VALOR, PATH_VALOR_CSV, csv_ptbr_text=True)
    _append_log(f"Salvos: {PATH_PRECO} {wide_preco.shape} | {PATH_VALOR} {wide_valor.shape}")

    # 6) JSON pt-BR (texto garantido) — missing vira ""
    try:
        json_text = wide_to_ptbr_json_text(wide_preco)
        with open(PATH_JSON, "w", encoding="utf-8") as f:
            f.write(json_text)
        _append_log(f"Salvo JSON pt-BR de preços (texto): {PATH_JSON}")
    except Exception as e:
        _append_log(f"[warn] Falha ao gerar JSON pt-BR: {e}")


if __name__ == "__main__":
    main()



[2025-12-15 15:43:32] [IPCA_PREV] Carregado 182 linhas de inflação prevista de TaxasInflacaoDiariaPrevisao.xlsx
[2025-12-15 15:43:32] [drop-dup] Removendo coluna duplicada 2025-12-12 (cópia de 2025-12-11)
[2025-12-15 15:43:32] [drop-dup] Removendo coluna duplicada 2025-12-12 (cópia de 2025-12-11)
[2025-12-15 15:43:32] Atualizando de 2025-12-11 até 2025-12-12 (DU B3: 2)
[2025-12-15 15:43:33] [HTTP] IPCACoupon 2025-12-11 -> status=200 content-length=1142 t=0.33s
    [dump] CSV bruto salvo em: debug_b3_csv\IPCACoupon_2025-12-11.csv
[2025-12-15 15:43:33] [HTTP] md5=64cde4d5df4cb183d1ff3477b208c519 bytes=2128 (IPCACoupon 2025-12-11)
----[ RAW IPCACoupon 2025-12-11 | primeiras 22 de 22 linhas ]----
Vencimento;Contratos em aberto;Negócios realizados;Contratos negociados;Volume;Ajuste anterior;Preço de abertura;Preço mínimo;Preço máximo;Preço médio;Último preço;Ajuste;Variação em pontos;Última oferta de compra;Última oferta de venda
F26;63.017;9;327;59.775.888;99.103,08;10,27;10,27;10,27;10,27

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_21364\840657584.py:1101: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(dfs, ignore_index=True)
C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_21364\840657584.py:801: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comb = pd.concat([df_base, df_novo], ignore_index=True) if not df_base.empty else df_novo.copy()


SystemExit: Não há inflação prevista na planilha para a data 2025-12-11.

c:\Users\emanuel.cabral\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [25]:
import pandas as pd
dados = pd.read_parquet("df_preco_de_ajuste_atual_completo.parquet")
dados

,Assets,2025-01-02,2025-01-03,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,...,2025-11-27,2025-11-28,2025-12-01,2025-12-02,2025-12-03,2025-12-04,2025-12-05,2025-12-08,2025-12-09,2025-12-10
0,DAP25,"98.420,08","98.537,44","98.566,96","98.606,00","98.589,62","98.605,90","98.600,13","98.688,39","98.728,79",...,,,,,,,,,,
1,DAP26,"88.536,62","88.708,86","88.894,14","88.906,74","88.985,51","88.839,95","88.774,27","88.813,71","88.813,96",...,"93.454,03","93.483,38","93.491,92","93.566,07","93.636,91","93.613,30","93.462,30","93.533,55","93.564,15",
2,DAP27,"83.678,70","83.886,14","84.166,83","84.173,14","84.216,02","84.167,76","84.065,34","84.090,24","84.187,56",...,"88.754,02","88.700,76","88.677,31","88.794,48","88.876,30","88.864,62","88.731,36","88.830,40","88.802,18",
3,DAP28,"75.982,66","76.132,39","76.434,84","76.381,23","76.429,40","76.426,78","76.322,91","76.244,76","76.368,70",...,"81.312,68","81.266,57","81.220,71","81.417,37","81.543,25","81.537,58","81.190,08","81.185,19","81.090,51",
4,DAP29,"72.298,68","72.349,20","72.574,18","72.334,21","72.471,68","72.435,19","72.283,27","72.160,86","72.384,21",...,"77.467,89","77.404,76","77.366,43","77.561,24","77.707,03","77.791,32","77.239,20","77.238,16","77.128,00",
5,DAP30,"66.333,67","66.353,05","66.578,79","66.357,52","66.308,42","66.293,65","66.108,37","65.890,08","66.113,47",...,"70.995,04","71.015,70","70.959,48","71.180,23","71.370,55","71.499,26","70.873,95","70.803,17","70.519,86",
6,DAP32,"57.412,89","57.107,32","57.486,70","57.181,02","57.117,54","57.174,43","56.911,25","56.729,12","56.984,85",...,"61.862,71","61.861,20","61.821,37","62.069,43","62.299,03","62.471,24","61.758,45","61.738,07","61.319,18",
7,DAP33,"54.417,48","54.181,57","54.617,59","54.506,96","54.522,76","54.664,93","54.428,38","54.276,75","54.460,07",...,"58.908,05","58.843,50","58.779,14","59.039,45","59.239,52","59.440,15","58.724,86","58.701,30","58.316,44",
8,DAP35,"47.304,65","47.182,63","47.468,03","47.164,85","47.359,26","47.372,93","47.296,15","47.129,52","47.323,51",...,"52.045,68","51.878,01","51.869,70","52.202,87","52.446,21","52.644,50","51.723,63","51.760,66","51.370,28",
9,DAP40,"33.461,41","33.475,62","33.767,22","33.504,04","33.878,97","33.814,90","33.343,71","33.151,40","33.208,67",...,"37.170,97","37.054,16","37.064,14","37.455,92","37.722,77","37.810,18","37.230,90","37.139,42","36.846,77",


In [18]:
dados2 = pd.read_parquet("DashRisco\Dados\df_preco_de_ajuste_atual_completo.parquet")
dados2

,Assets,2025-01-02,2025-01-03,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,...,2025-11-26,2025-11-27,2025-11-28,2025-12-01,2025-12-02,2025-12-03,2025-12-04,2025-12-05,2025-12-08,2025-12-09
0,DAP25,"98.420,08","98.537,44","98.566,96","98.606,00","98.589,62","98.605,90","98.600,13","98.688,39","98.728,79",...,"100.000,00","100.000,00","100.000,00","100.000,00","100.000,00","100.000,00","100.000,00","100.000,00","100.000,00","100.000,00"
1,DAP26,"88.536,62","88.708,86","88.894,14","88.906,74","88.985,51","88.839,95","88.774,27","88.813,71","88.813,96",...,"93.418,70","93.454,03","93.483,38","93.491,92","93.566,07","93.636,91","93.613,30","93.462,30","93.533,55","93.533,55"
2,DAP27,"83.678,70","83.886,14","84.166,83","84.173,14","84.216,02","84.167,76","84.065,34","84.090,24","84.187,56",...,"88.748,61","88.754,02","88.700,76","88.677,31","88.794,48","88.876,30","88.864,62","88.731,36","88.830,40","88.830,40"
3,DAP28,"75.982,66","76.132,39","76.434,84","76.381,23","76.429,40","76.426,78","76.322,91","76.244,76","76.368,70",...,"81.379,39","81.312,68","81.266,57","81.220,71","81.417,37","81.543,25","81.537,58","81.190,08","81.185,19","81.185,19"
4,DAP29,"72.298,68","72.349,20","72.574,18","72.334,21","72.471,68","72.435,19","72.283,27","72.160,86","72.384,21",...,"77.444,97","77.467,89","77.404,76","77.366,43","77.561,24","77.707,03","77.791,32","77.239,20","77.238,16","77.238,16"
5,DAP30,"66.333,67","66.353,05","66.578,79","66.357,52","66.308,42","66.293,65","66.108,37","65.890,08","66.113,47",...,"71.020,64","70.995,04","71.015,70","70.959,48","71.180,23","71.370,55","71.499,26","70.873,95","70.803,17","70.803,17"
6,DAP32,"57.412,89","57.107,32","57.486,70","57.181,02","57.117,54","57.174,43","56.911,25","56.729,12","56.984,85",...,"61.921,92","61.862,71","61.861,20","61.821,37","62.069,43","62.299,03","62.471,24","61.758,45","61.738,07","61.738,07"
7,DAP33,"54.417,48","54.181,57","54.617,59","54.506,96","54.522,76","54.664,93","54.428,38","54.276,75","54.460,07",...,"58.932,04","58.908,05","58.843,50","58.779,14","59.039,45","59.239,52","59.440,15","58.724,86","58.701,30","58.701,30"
8,DAP35,"47.304,65","47.182,63","47.468,03","47.164,85","47.359,26","47.372,93","47.296,15","47.129,52","47.323,51",...,"51.962,96","52.045,68","51.878,01","51.869,70","52.202,87","52.446,21","52.644,50","51.723,63","51.760,66","51.760,66"
9,DAP40,"33.461,41","33.475,62","33.767,22","33.504,04","33.878,97","33.814,90","33.343,71","33.151,40","33.208,67",...,"37.059,52","37.170,97","37.054,16","37.064,14","37.455,92","37.722,77","37.810,18","37.230,90","37.139,42","37.139,42"


In [15]:
import pandas as pd
dados = pd.read_parquet("df_valor_ajuste_contrato.parquet")
dados

,Assets,2025-01-02,2025-01-03,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,...,2025-11-27,2025-11-28,2025-12-01,2025-12-02,2025-12-03,2025-12-04,2025-12-05,2025-12-08,2025-12-09,2025-12-10
0,DAP25,"100,19","175,57","19,91","36,78","-61,50","-3,60","-72,34","122,34","37,40",...,,,,,,,,,,
1,DAP26,"435,81","276,05","299,17","-6,91","110,39","-287,45","-172,47","39,15","-30,41",...,"-28,16","-22,61","-61,25","59,91","53,76","-120,67","-355,33","54,37","-20,42","-20,42"
2,DAP27,"711,74","340,01","469,84","-16,50","48,31","-113,33","-234,70","14,99","143,49",...,"-78,63","-171,01","-116,27","143,16","77,91","-94,75","-318,73","109,52","-125,03","-125,03"
3,DAP28,"569,66","240,30","510,95","-120,18","60,26","-29,81","-232,39","-165,16","193,45",...,"-204,18","-151,73","-151,45","295,82","165,20","-77,62","-707,74","-75,92","-241,33","-241,33"
4,DAP29,"1.002,80","65,74","374,91","-449,25","219,93","-88,58","-315,12","-242,28","371,29",...,"-35,09","-179,93","-134,38","295,56","205,08","91,39","-1.081,99","-65,58","-266,63","-266,63"
5,DAP30,"813,89","12,54","378,22","-414,13","-108,90","-48,02","-370,43","-410,24","373,54",...,"-118,09","-20,20","-162,11","348,65","292,41","178,51","-1.211,76","-188,93","-580,69","-580,69"
6,DAP32,"805,99","-560,23","653,54","-560,76","-131,38","82,08","-502,90","-342,90","434,11",...,"-170,94","-53,56","-124,36","406,50","372,31","266,18","-1.365,66","-88,48","-823,29","-823,29"
7,DAP33,"1.204,93","-435,82","754,87","-214,07","10,07","234,19","-454,08","-287,91","306,46",...,"-103,05","-167,32","-167,08","431,57","320,35","321,11","-1.367,78","-91,84","-758,04","-758,04"
8,DAP35,"497,29","-231,73","490,26","-553,03","329,17","8,65","-166,05","-312,06","327,88",...,"100,56","-351,73","-58,04","571,55","405,73","322,40","-1.741,27","25,65","-762,52","-762,52"
9,DAP40,"1.090,33","14,17","505,74","-477,63","653,75","-124,78","-857,20","-352,78","90,11",...,"168,37","-245,80","-12,13","691,76","461,18","130,09","-1.099,21","-199,39","-570,27","-570,27"


In [6]:
dados2 = pd.read_parquet("DashRisco\Dados\df_valor_ajuste_contrato.parquet")
dados2

,Assets,2025-01-02,2025-01-03,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,...,2025-11-26,2025-11-27,2025-11-28,2025-12-01,2025-12-02,2025-12-03,2025-12-04,2025-12-05,2025-12-08,2025-12-09
0,DAP25,"100,19","175,57","19,91","36,78","-61,50","-3,60","-72,34","122,34","37,40",...,None,None,None,None,None,None,None,None,None,None
1,DAP26,"435,81","276,05","299,17","-6,91","110,39","-287,45","-172,47","39,15","-30,41",...,"47,45","-28,16","-22,61","-61,24","59,90","53,75","-120,67","-355,32","54,36","54,36"
2,DAP27,"711,74","340,01","469,84","-16,50","48,31","-113,33","-234,70","14,99","143,49",...,"70,78","-78,63","-171,00","-116,26","143,15","77,90","-94,75","-318,72","109,52","109,52"
3,DAP28,"569,66","240,30","510,95","-120,18","60,26","-29,81","-232,39","-165,16","193,45",...,"131,20","-204,20","-151,73","-151,45","295,82","165,20","-77,62","-707,74","-75,92","-75,92"
4,DAP29,"1.002,80","65,74","374,91","-449,25","219,93","-88,58","-315,12","-242,28","371,29",...,"117,32","-35,09","-179,92","-134,38","295,56","205,07","91,39","-1.081,98","-65,57","-65,57"
5,DAP30,"813,89","12,54","378,22","-414,13","-108,90","-48,02","-370,43","-410,24","373,54",...,"209,30","-118,09","-20,19","-162,10","348,65","292,40","178,51","-1.211,76","-188,93","-188,93"
6,DAP32,"805,99","-560,23","653,54","-560,76","-131,38","82,08","-502,90","-342,90","434,11",...,"302,26","-170,95","-53,55","-124,35","406,49","372,30","266,18","-1.365,65","-88,48","-88,48"
7,DAP33,"1.204,93","-435,82","754,87","-214,07","10,07","234,19","-454,08","-287,91","306,46",...,"358,65","-103,05","-167,32","-167,08","431,56","320,35","321,10","-1.367,77","-91,83","-91,83"
8,DAP35,"497,29","-231,73","490,26","-553,03","329,17","8,65","-166,05","-312,06","327,88",...,"236,94","100,57","-351,73","-58,04","571,55","405,72","322,39","-1.741,26","25,65","25,65"
9,DAP40,"1.090,33","14,17","505,74","-477,63","653,75","-124,78","-857,20","-352,78","90,11",...,"408,02","168,39","-245,79","-12,12","691,75","461,18","130,09","-1.099,20","-199,38","-199,38"
